<a id='GoTop'></a>
# Digging Through Source of Hadrontherapy Geant4 Simulation

* **Main File** ['hadrontherapy.cc'](#hadrontherapy.cc) 
* **Header files in include/** sub-directory
    * [HadrontherapyGeometryController.hh](#HadrontherapyGeometryController.hh)
        * [HadrontherapyGeometryMessenger.hh](#HadrontherapyGeometryMessenger.hh)
        * [PassiveProtonBeamLine.hh](#PassiveProtonBeamLine.hh)
        * Other three beamlines
    * [HadrontherapyPhysicsList.hh](#HadrontherapyPhysicsList.hh)
    * [HadrontherapyActionInitialization.hh](#HadrontherapyActionInitialization.hh)
        - [HadrontherapyPrimaryGeneratorAction.hh](#HadrontherapyPrimaryGeneratorAction.hh)
            * Messenger
        - [HadrontherapyRunAction.hh](#HadrontherapyRunAction.hh)
            * Messenger
        - [HadrontherapyEventAction.hh](#HadrontherapyEventAction.hh) 
            * Messenger
        - [HadrontherapySteppingAction.hh](#HadrontherapySteppingAction.hh)
            * Messenger
    * [HadrontherapyInteractionParameters.hh](#HadrontherapyInteractionParameters.hh)
    * [HadrontherapyAnalysis.hh](#HadrontherapyAnalysis.hh)
    * [HadrontherapyMatrix.hh](#HadrontherapyMatrix.hh)
        - Also has the definition of **struct ion**
    * [HadrontherapyLet.hh](#HadrontherapyLet.hh)
        - Also has the definition of **struct ionLet**
    * [HadrontherapyRBE.hh](#HadrontherapyRBE.hh)
        * [HadrontherapyRBEAccumulable.hh](#HadrontherapyRBEAccumulable.hh)
* **Source files in src/** sub-directory
    * [HadrontherapyGeometryController.cc](#HadrontherapyGeometryController.cc) (See some notes about [Abstract class, virtual and pure-virtual methods ...](#AbstractVirtual)

        * [HadrontherapyGeometryMessenger.cc](#HadrontherapyGeometryMessenger.cc)
        * [PassiveProtonBeamLine.cc](#PassiveProtonBeamLine.cc)
        * Other three beamlines        
    * [HadrontherapyPhysicsList.cc](#HadrontherapyPhysicsList.cc)
    * [HadrontherapyActionInitialization.cc](#HadrontherapyActionInitialization.cc)
        - [HadrontherapyPrimaryGeneratorAction.cc](#HadrontherapyPrimaryGeneratorAction.cc)
            * Messenger
        - [HadrontherapyRunAction.cc](#HadrontherapyRunAction.cc)
            * Messenger
        - [HadrontherapyEventAction.cc](#HadrontherapyEventAction.cc) 
            * Messenger
        - [HadrontherapySteppingAction.cc](#HadrontherapySteppingAction.cc)
            * Messenger
    * [HadrontherapyInteractionParameters.cc](#HadrontherapyInteractionParameters.cc)
    * [HadrontherapyMatrix.cc](#HadrontherapyMatrix.cc)
        - [HadrontherapyAnalysis.cc](#HadrontherapyAnalysis.cc)
    * [HadrontherapyLet.cc](#HadrontherapyLet.cc)
    * [HadrontherapyRBE.cc](#HadrontherapyRBE.cc)
        * [HadrontherapyRBEAccumulable.cc](#HadrontherapyRBEAccumulable.cc)
    

### Some notes about Dose, LET, RBE etc calculations.
* About [Dose Calculations](#DoseCalculations)
* About [LET Calculations](#LETCalculations)
* About [RBE Calculations](#RBECalculations)

### Some other TidBits
* Use of ['using' keyword](#alias_usingKeyword) for aliases.
* [**Auto** keyword](#auto_keyword)
* [Abstract class, virtual and pure-virtual methods](#abstract_virtual) ...
* Only use of **dynamic_cast** (see [HadrontherapyRBEAccumulable.cc](#HadrontherapyRBEAccumulable.cc)): `HadrontherapyRBEAccumulable.cc:47:    const HadrontherapyRBEAccumulable& other = dynamic_cast<const HadrontherapyRBEAccumulable&>(rhs);`

In [1]:
!pwd

/Users/kpadhikari/KpInstalls/Geant4/ExamplesBld/HadronTherapyNew/Source/hadrontherapy


In object oriented program is a program consisting of interactive objects.

<a id='GoTop'></a><a id="hadrontherapy.cc"></a>
## Main File hadrontherapy.cc
* **Object and pointer creation of**
    * **G4UIExecutive**
        - To detect interactive mode (if no arguments) and define UI session
        - If no argument or .mac file is given, a G4UIExecutive object and its **pointer 'ui'** is created as **ui = new G4UIExecutive(argc, argv);**
    * **G4Timer** 
        - then Start() method is invoked.
    * **Random engine is set/initialized** (also for multithreading) using **CLHEP::RanluxEngine** and **G4Random**. 
        - **time(NULL)** is used for seed.
    * either **G4MTRunManager** (for multi-threading) or **G4RunManager**.
        * Please note that runManager is a singleton type see the class description in [G4RunManager.hh](https://gitlab.cern.ch/geant4/geant4/blob/bc13104726e8975c0e4a62bf0d859dcca77d4213/source/run/include/G4RunManager.hh) (or [this](http://www.apc.univ-paris7.fr/~franco/g4doxy4.10/html/_g4_run_manager_8hh_source.html)) and see [this code](#HadrontherapyGeometryController::registerGeometry) where we have a local pointer which is accessing the same object (first created here in the main()). The class description says the following:
            - G4RunManager is the only manager class in Geant4 kernel which the user MUST construct an object by him/herself in the main(). Also, G4RunManager is the only manager class in Geant4 kernel which the user CAN derive it to costomize the behavior of the run control. For this case, user should use protected methods provided in this class for procedures he/she does not want to change.
            - G4RunManager or the derived class of it MUST be a singleton. The user MUST NOT construct more than one object even if there are two different concrete implementations.
        * The method G4RunManager::GetRunManager() ([see G4RunManager.cc](http://www.apc.univ-paris7.fr/~franco/g4doxy4.10/html/_g4_run_manager_8cc_source.html#l00074)), just returns **fRunManager**, which is assigned 'this' insided the constructor G4RunManager::G4RunManager(). Now, if we go back to G4RunManager.hh, we see the **private static pointer** declared as follows: **static G4ThreadLocal G4RunManager* fRunManager;**, which ensures that we have that pointer residing in global scope (because it's static) and thus accessible from anywhere. 
    * [**HadrontherapyGeometryController**](#HadrontherapyGeometryController.hh) and [**HadrontherapyGeometryMessenger**](#HadrontherapyGeometryMessenger.hh) (takes the pointer of the former as input).
        - <span style="color:red">I have yet to understand the flow of the Messenger class, how it is triggered (by actions) and which function is exactly called first by the actions and so on!</span>**
    * **G4ScoringManager** (via GetScoringManager() because it is a **singleton object**, where constructor is private).
* **Initialize**
    * **Geometry:** geometryController->SetGeometry("default");
        - Inside SetGeometry method of [HadrontherapyGeometryController](#HadrontherapyGeometryController.cc) class, it calls another (private) method **registerGeometry(new [PassiveProtonBeamLine](PassiveProtonBeamLine.hh)());** for **default** option. Other options are 'Carbon', 'LaserDriven', 'TrentoLine'.
    * **the physics:**  runManager->SetUserInitialization(phys);
        - **(G4VModularPhysicsList* phys)** gets value either via PHYSLIST env variable (using G4PhysListFactory) or via an object of [HadrontherapyPhysicsList](#HadrontherapyPhysicsList.hh). 
    * **the Actions:**  runManager->SetUserInitialization(new [HadrontherapyActionInitialization](#HadrontherapyActionInitialization.hh));
        * Inside **BuildForMaster() const** method (meant for master thread in MT mode), an object of [HadrontherapyRunAction](#HadrontherapyRunAction.hh) is created and its pointer (pRunAction) is passed as **SetUserAction(pRunAction);**
        * Similarly, inside **Build() const** method, objects of [HadrontherapyPrimaryGeneratorAction](#HadrontherapyPrimaryGeneratorAction.hh), [HadrontherapyRunAction](#HadrontherapyRunAction.hh), [HadrontherapyEventAction](#HadrontherapyEventAction.hh) and [HadrontherapySteppingAction](#HadrontherapySteppingAction.hh) and pointer to each of them is passed to **SetUserAction(actionPointer)** method.         
    * **Command based scoring:** G4ScoringManager::GetScoringManager();
    * **Interaction data: stopping powers:**
        - HadrontherapyInteractionParameters* pInteraction = new [HadrontherapyInteractionParameters(true)](#HadrontherapyInteractionParameters.hh);
    * **Initialize analysis:** 
        - HadrontherapyAnalysis* analysis = [HadrontherapyAnalysis](#HadrontherapyAnalysis.hh)::GetInstance();
    * **Visualization:**
        - G4VisManager* visManager = new G4VisExecutive;
        - visManager -> Initialize();
* **Create and Use User Interface manager**
    * Get the pointer to the User Interface manager (kp: again singleton)
        * G4UImanager* UImanager = G4UImanager::GetUIpointer();
    * if (!ui) // **batch mode**
        * Pass **"/control/execute argv[1]"** command to UImanager:
            - UImanager->ApplyCommand("/control/execute" + fileName); 
    * else     // **interactive mode**
        * UImanager->ApplyCommand("/control/execute init_vis.mac");
        * ui->SessionStart();
        * delete ui;
* **Wrap up/Close out and Delete things**
    * delete visManager;
    * theTimer->Stop();
    * if ( HadrontherapyMatrix * pMatrix = [HadrontherapyMatrix](#HadrontherapyMatrix.hh)::GetInstance() ) pMatrix -> StoreDoseFluenceAscii();
    * if (HadrontherapyLet *let = [HadrontherapyLet](#HadrontherapyLet.hh)::GetInstance())
        if(let -> doCalculation)
        {
            let -> LetOutput(); 	// Calculate let
            let -> StoreLetAscii(); // Store it
        }
    * delete geometryMessenger;
    * delete geometryController;
    * delete pInteraction;
    * delete runManager;
    * delete analysis;
    * return 0;

[Go Top](#GoTop)    =================================
Actual code of the main file **hadrontherapy.cc**.
```cpp
//
// ********************************************************************
// * License and Disclaimer                                           *
// *                                                                  *
// * The  Geant4 software  is  copyright of the Copyright Holders  of *
// * the Geant4 Collaboration.  It is provided  under  the terms  and *
// * conditions of the Geant4 Software License,  included in the file *
// * LICENSE and available at  http://cern.ch/geant4/license .  These *
// * include a list of copyright holders.                             *
// *                                                                  *
// * Neither the authors of this software system, nor their employing *
// * institutes,nor the agencies providing financial support for this *
// * work  make  any representation or  warranty, express or implied, *
// * regarding  this  software system or assume any liability for its *
// * use.  Please see the license in the file  LICENSE  and URL above *
// * for the full disclaimer and the limitation of liability.         *
// *                                                                  *
// * This  code  implementation is the result of  the  scientific and *
// * technical work of the GEANT4 collaboration.                      *
// * By using,  copying,  modifying or  distributing the software (or *
// * any work based  on the software)  you  agree  to acknowledge its *
// * use  in  resulting  scientific  publications,  and indicate your *
// * acceptance of all terms of the Geant4 Software license.          *
// ********************************************************************
//
//      ----------------------------------------------------------------------------
//                              GEANT 4 - Hadrontherapy example
//      ----------------------------------------------------------------------------
//
//                                      MAIN AUTHOR
//                                  ====================
//                                  G.A.P. Cirrone(a)*
//
//                 *Corresponding author, email to pablo.cirrone@lns.infn.it
//
//                                  ACTUAL CONTRIBUTORS
//                                  ====================
//          G.A.P. Cirrone(a), Z. Mei(i), L. Pandola(a), G. Petringa(a), F. Romano (a,g)
//
//
//                      ==========>   PAST CONTRIBUTORS  <==========
//
//                      R. Calcagno(a), G.Danielsen (b), F.Di Rosa(a),
//                      S.Guatelli(c), A.Heikkinen(b), P.Kaitaniemi(b),
//                      A.Lechner(d), S.E.Mazzaglia(a), M.G.Pia(e),
//                      G.Russo(a,h), M.Russo(a), A. Tramontana (a),
//                      A.Varisano(a)
//
//              (a) Laboratori Nazionali del Sud of INFN, Catania, Italy
//              (b) Helsinki Institute of Physics, Helsinki, Finland
//              (c) University of Wallongong, Australia
//              (d) CERN, Geneve, Switzwerland
//              (e) INFN Section of Genova, Genova, Italy
//              (f) Physics and Astronomy Department, Univ. of Catania, Catania, Italy
//              (g) National Physics Laboratory, Teddington, UK
//              (h) CNR-IBFM, Italy
//              (i) Institute of Applied Electromagnetic Engineering(IAEE) 
//                  Huazhong University of Science and Technology(HUST), Wuhan, China
//
//
//                                          WEB
//                                      ===========
//       https://twiki.cern.ch/twiki/bin/view/Geant4/AdvancedExamplesHadrontherapy
//
// ----------------------------------------------------------------------------

#include "G4RunManager.hh"
#include "G4UImanager.hh"
#include "G4PhysListFactory.hh"
#include "G4VModularPhysicsList.hh"
#include "HadrontherapyEventAction.hh"
#include "HadrontherapyPhysicsList.hh"
#include "HadrontherapyDetectorSD.hh"
#include "HadrontherapyPrimaryGeneratorAction.hh"
#include "HadrontherapyRunAction.hh"
#include "HadrontherapyMatrix.hh"
#include "Randomize.hh"

#include "G4UImessenger.hh"
#include "globals.hh"
#include "HadrontherapySteppingAction.hh"
#include "HadrontherapyGeometryController.hh"
#include "HadrontherapyGeometryMessenger.hh"
#include "HadrontherapyInteractionParameters.hh"
#include "HadrontherapyLet.hh"

#include "G4ScoringManager.hh"
#include "G4ParallelWorldPhysics.hh"
#include <time.h>
#include "G4Timer.hh"

//************************MT*********************
#ifdef G4MULTITHREADED
#include "G4MTRunManager.hh"
#else
#include "G4RunManager.hh"
#endif

#include "HadrontherapyActionInitialization.hh"

//#ifdef G4VIS_USE
#include "G4VisExecutive.hh"
//#endif

//#ifdef G4UI_USE
#include "G4UIExecutive.hh"
//#endif

//////////////////////////////////////////////////////////////////////////////////////////////
int main(int argc ,char ** argv)
{
        G4UIExecutive* ui = 0;
    if ( argc == 1 ) {
        ui = new G4UIExecutive(argc, argv);
    }
    
    //Instantiate the G4Timer object, to monitor the CPU time spent for
    //the entire execution
    G4Timer* theTimer = new G4Timer();
    //Start the benchmark
    theTimer->Start();
    
    // Set the Random engine
    // The following guarantees random generation also for different runs
    // in multithread
    CLHEP::RanluxEngine defaultEngine( 1234567, 4 );
    G4Random::setTheEngine( &defaultEngine );
    G4int seed = time( NULL );
    G4Random::setTheSeed( seed );
    
#ifdef G4MULTITHREADED
    
    G4MTRunManager* runManager = new G4MTRunManager;
#else
    G4RunManager* runManager = new G4RunManager;
#endif
    
    // Geometry controller is responsible for instantiating the
    // geometries. All geometry specific m tasks are now in class
    // HadrontherapyGeometryController.
    HadrontherapyGeometryController *geometryController = new HadrontherapyGeometryController();
    
    // Connect the geometry controller to the G4 user interface
    HadrontherapyGeometryMessenger *geometryMessenger = new HadrontherapyGeometryMessenger(geometryController);

    /**
     *     //kp: G4ScoringManager seems to be a singleton object
     * Scoring manager seems to be a singleton object (there is only one instance).
     * The pointer to this object is available through the use of the method GetScoringManager();
     *
     * By design, constructors cannot be called to create a singleton object, because they are
     *     deliberately made private in the corresponding class definition. This is done in order
     *     to enforce having just one object being created, and this is done through a public
     *     static method which is sometimes also named as GetInstance(), which returns the pointer
     *     to the singleton object (the pointer is a static private member of the same class).
     *
     *  https://www.interviewsansar.com/2014/10/25/singleton-class-design-with-an-example/
     */

    G4ScoringManager *scoringManager = G4ScoringManager::GetScoringManager();
    scoringManager->SetVerboseLevel(1);
    
    // Initialize the default Hadrontherapy geometry
    geometryController->SetGeometry("default");
    
    // Initialize the physics
    G4PhysListFactory factory;
    G4VModularPhysicsList* phys = 0;
    G4String physName = "";
    
    // Physics List name defined via environment variable
    char* path = getenv("PHYSLIST");
    if (path) { physName = G4String(path); }
    
    if(physName != "" && factory.IsReferencePhysList(physName))
    {
        phys = factory.GetReferencePhysList(physName);
    }
    if (phys)
    {
        G4cout << "Going to register G4ParallelWorldPhysics" << G4endl;
        phys->RegisterPhysics(new G4ParallelWorldPhysics("DetectorROGeometry"));
    }
    else
    {
        G4cout << "Using HadrontherapyPhysicsList()" << G4endl;
        phys = new HadrontherapyPhysicsList();
    }
    
    // Initialisations of physics
    runManager->SetUserInitialization(phys);
    
    // Initialisation of the Actions
    runManager->SetUserInitialization(new HadrontherapyActionInitialization);
    
    // Initialize command based scoring
    G4ScoringManager::GetScoringManager();
    
    // Interaction data: stopping powers
    HadrontherapyInteractionParameters* pInteraction = new HadrontherapyInteractionParameters(true);
    
    // Initialize analysis
    HadrontherapyAnalysis* analysis = HadrontherapyAnalysis::GetInstance();
    

    
// Initialise the Visualisation
//#ifdef G4VIS_USE
    G4VisManager* visManager = new G4VisExecutive;
    visManager -> Initialize();
//#endif
    
    //** Get the pointer to the User Interface manager
    G4UImanager* UImanager = G4UImanager::GetUIpointer();
    
    if ( !ui ) {
        // batch mode
        G4String command = "/control/execute ";
        G4String fileName = argv[1];
        UImanager->ApplyCommand(command+fileName);
        
    }
    
    else {
        G4cout << "sono qua" << G4endl;

        //UImanager -> ApplyCommand("/control/execute macro/defaultMacro.mac");
        //UImanager -> ApplyCommand("/control/execute init_visCpdFromElsewhereAndModified.mac");//Works
        //UImanager -> ApplyCommand("/control/execute visCpdFromElsewhereAndModified2.mac"); //Works
        UImanager -> ApplyCommand("/control/execute macro/visualisationMacro.mac"); //works
        //UImanager -> ApplyCommand("/control/execute visCpdFromElsewhereAndModified.mac");//Works but no geom (dark panel)
        ui -> SessionStart();
        delete ui;
    }
    delete visManager;
 
    //Stop the benchmark here
    theTimer->Stop();
    
    G4cout << "The simulation took: " << theTimer->GetRealElapsed() << " s to run (real time)"
    << G4endl;
    
    // Job termination
    // Free the store: user actions, physics_list and detector_description are
    // owned and deleted by the run manager, so they should not be deleted
    // in the main() program !
    
    
        if ( HadrontherapyMatrix * pMatrix = HadrontherapyMatrix::GetInstance() )
    {
        // pMatrix -> TotalEnergyDeposit();
        pMatrix -> StoreDoseFluenceAscii();
        
    }
    
    if (HadrontherapyLet *let = HadrontherapyLet::GetInstance())
        if(let -> doCalculation)
        {
            let -> LetOutput(); 	// Calculate let
            let -> StoreLetAscii(); // Store it
        }
    
    delete geometryMessenger;
    delete geometryController;
    delete pInteraction;
    delete runManager;
    delete analysis;
    return 0;
    
}
```

[Go Top](#GoTop)<a id='HadrontherapyGeometryController.hh'></a>
## include/HadrontherapyGeometryController.hh
```cpp
// Hadrontherapy advanced example for Geant4
// See more at: https://twiki.cern.ch/twiki/bin/view/Geant4/AdvancedExamplesHadrontherapy
#ifndef HadrontherapyGeometryController_hh
#define HadrontherapyGeometryController_hh 1

#include "globals.hh"
#include "G4String.hh"
#include "G4VUserDetectorConstruction.hh"

/**
 * Controller for geometry selection
 *
 * This controller is called by the geometry messenger and used to
 * select the geometry. Each available geometry must have unique name
 * and it must be known by the geometry controller.
 */
class HadrontherapyGeometryController
{
public:
  HadrontherapyGeometryController();
  ~HadrontherapyGeometryController();

  /**
   * Select a geometry by name.
   */
  void SetGeometry(G4String);

private:
  void registerGeometry(G4VUserDetectorConstruction *detector);
};

#endif
```

[Go Top](#GoTop)<a id='HadrontherapyGeometryController.cc'></a>
## src/HadrontherapyGeometryController.cc
```cpp
// Hadrontherapy advanced example for Geant4
// See more at: https://twiki.cern.ch/twiki/bin/view/Geant4/AdvancedExamplesHadrontherapy

#include "HadrontherapyGeometryController.hh"
#include "HadrontherapyDetectorConstruction.hh"
#include "HadrontherapyInteractionParameters.hh"
#include "HadrontherapyDetectorROGeometry.hh"
#include "HadrontherapyTIFPAPassiveProtonBeamLine.hh"
#include "PassiveProtonBeamLine.hh"
#include "PassiveCarbonBeamLine.hh"
#include "LaserDrivenBeamLine.hh"
#include "G4RunManager.hh"
#include "G4VUserParallelWorld.hh"
#include "G4ThreeVector.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryController::HadrontherapyGeometryController()
{}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryController::~HadrontherapyGeometryController()
{}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyGeometryController::SetGeometry(G4String name)
{
    G4cout <<"Activating geometry " << name << G4endl;
    if(name == "default")
    {
      registerGeometry(new PassiveProtonBeamLine());
    }
    else if(name == "Carbon")
    {
      registerGeometry(new PassiveCarbonBeamLine());
    }
    else if(name == "LaserDriven")
    {
      registerGeometry(new LaserDrivenBeamLine());
    }
    
    else if(name == "TrentoLine")
    {
        registerGeometry(new TrentoPassiveProtonBeamLine());
    }
    else
    {
        G4cout <<"Unknown geometry: " << name << ". Geometry not changed." << G4endl;
    }
}
```
<a id="HadrontherapyGeometryController::registerGeometry"></a>
```cpp
/////////////////////////////////////////////////////////////////////////////
void HadrontherapyGeometryController::registerGeometry(G4VUserDetectorConstruction *detector)
{
	G4RunManager *runManager = G4RunManager::GetRunManager();
	runManager -> SetUserInitialization(detector);
	runManager -> GeometryHasBeenModified();
}
```
<a id='AbstractVirtual'></a>
KPA: Using the pointer of **Abstract class G4VUserDetectorConstruction** (note **G4V for G4-Virtual**) as the argument for the **registerGeometry(detector)**, I think, it is using **runtime polymorphism** to pass various different implementations/extensions of that abstract class such as PassiveProtonBeamLine, PassiveCarbonBeamLine etc.

[Go Top](#GoTop)<a id='abstract_virtual'></a>
## Abstract class, virtual and pure-virtual methods ...
* A class is abstract if it has at least one pure virtual function. ([Ref1](https://www.geeksforgeeks.org/pure-virtual-functions-and-abstract-classes/))
    * We can have pointers and references of abstract class type.
    * If we do not override the pure virtual function in derived class, then derived class also becomes abstract class.
    * An abstract class can have constructors.
    * Abstract classes are the base class which cannot be instantiated.[Ref2](https://www.programiz.com/cpp-programming/virtual-functions) (What?? Can have constructors but cannot be intantiated? What does that mean?)
* A **pure virtual function (or abstract function)** in C++ is a **virtual function** for which we don’t have implementation, we only declare it. A pure virtual function is declared by assigning 0 in declaration. 
* A **virtual function** is a member function in base class that you expect to redefine in derived classes. 
    * Declared with 'virtual' keyword in the front.
    * **Virtual functions** allow **runtime polymorphism**. The main thing to note is that the derived class’s function is called using a base class pointer. The idea is that virtual functions are called according to the type of the object instance pointed to or referenced, not according to the type of the pointer or reference.
    In other words, virtual functions are resolved late, at runtime.
    
```cpp
#include<iostream> 
using namespace std; 
  
class Base 
{ 
public: 
    virtual void show() = 0;   //pure virtual method
    virtual void show2() { cout<<"show2() in Base class."<<endl;  } //Just 'virtual method' with no 'pure' tag.
}; 
  
class Derived: public Base 
{ 
public: 
    void show() { cout << "In Derived \n"; } 
    void show2() {cout << "show2() in Derived class."<<endl; }
}; 
  
int main(void) 
{ 
    Base *bp = new Derived(); 
    bp->show(); 
    return 0; 
}
```
### Comparison with Java
In Java, a class can be made abstract by using abstract keyword. Similarly a function can be made pure virtual or abstract by using abstract keyword. See
Abstract Classes in Java for more details.

### Interface vs Abstract Classes:
An interface does not have implementation of any of its methods, it can be considered as a collection of method declarations. In C++, an interface can be simulated by making all methods as pure virtual. In Java, there is a separate keyword for interface. 

[Go Top](#GoTop)<a id='HadrontherapyGeometryMessenger.hh'></a>
## include/HadrontherapyGeometryMessenger.hh

```cpp
#ifndef HadrontherapyGeometryMessenger_h
#define HadrontherapyGeometryMessenger_h 1

#include "globals.hh"
#include "G4UImessenger.hh"
#include "G4String.hh"

class HadrontherapyGeometryController;
class G4UIdirectory;
class G4UIcmdWithADoubleAndUnit;
class G4UIcmdWithAString;

class HadrontherapyGeometryMessenger: public G4UImessenger
{
public:
  HadrontherapyGeometryMessenger(HadrontherapyGeometryController* );
  ~HadrontherapyGeometryMessenger();
    
  void SetNewValue(G4UIcommand*, G4String);
    
private:

  // Pointer to the detector component
  HadrontherapyGeometryController* hadrontherapyGeometryController;

  G4UIdirectory *changeTheGeometryDir;      ///> UI directory for the geometry control
  G4UIcmdWithAString *changeTheGeometryCmd;//, *changeTheDetectorCmd; ///> Select the geometry and the detector
};
#endif
```

[Go Top](#GoTop)<a id='HadrontherapyGeometryMessenger.cc'></a>
## src/HadrontherapyGeometryMessenger.cc
```cpp
#include "HadrontherapyGeometryMessenger.hh"
#include "HadrontherapyGeometryController.hh"
#include "G4UIdirectory.hh"
#include "G4UIcmdWithADoubleAndUnit.hh"
#include "G4UIcmdWithAString.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryMessenger::HadrontherapyGeometryMessenger(HadrontherapyGeometryController* controller)
:hadrontherapyGeometryController(controller)

{
    changeTheGeometryDir = new G4UIdirectory("/geometrySetup/");
    changeTheGeometryDir -> SetGuidance("Geometry setup");
    
    changeTheGeometryCmd = new G4UIcmdWithAString("/geometrySetup/selectGeometry",this);
    changeTheGeometryCmd -> SetGuidance("Select the geometry you wish to use");
    changeTheGeometryCmd -> SetParameterName("Geometry",false);
    changeTheGeometryCmd -> AvailableForStates(G4State_PreInit);
    
    /*  changeTheDetectorCmd = new G4UIcmdWithAString("/geometrySetup/selectDetector",this);
    changeTheDetectorCmd -> SetGuidance("Select the detector you wish to use");
    changeTheDetectorCmd -> SetParameterName("Detector",false);
    changeTheDetectorCmd -> AvailableForStates(G4State_PreInit);*/
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyGeometryMessenger::~HadrontherapyGeometryMessenger()
{
    delete changeTheGeometryDir;
    delete changeTheGeometryCmd;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyGeometryMessenger::SetNewValue(G4UIcommand* command,G4String newValue)
{

 if( command == changeTheGeometryCmd )
    {
        hadrontherapyGeometryController -> SetGeometry (newValue);
    }
}
```

[Go Top](#GoTop)<a id='HadrontherapyPhysicsList.hh'></a>
## include/HadrontherapyPhysicsList.hh
```cpp
#ifndef HadrontherapyPhysicsList_h
#define HadrontherapyPhysicsList_h 1

#include "G4VModularPhysicsList.hh"
#include "G4EmConfigurator.hh"
#include "globals.hh"

class G4VPhysicsConstructor;
class HadrontherapyStepMax;
class HadrontherapyPhysicsListMessenger;

class HadrontherapyPhysicsList: public G4VModularPhysicsList
{
public:
    
    HadrontherapyPhysicsList();
    virtual ~HadrontherapyPhysicsList();
    
    void ConstructParticle();
    void SetCutForGamma(G4double);
    void SetCutForElectron(G4double);
    void SetCutForPositron(G4double);
    void SetDetectorCut(G4double cut);
    void AddPhysicsList(const G4String& name);
    void ConstructProcess();
    void AddStepMax();
    void AddPackage(const G4String& name);
    
private:
    
    G4EmConfigurator em_config;
    
    G4double cutForGamma;
    G4double cutForElectron;
    G4double cutForPositron;
    G4bool locIonIonInelasticIsRegistered;
    G4bool radioactiveDecayIsRegistered;
    G4String      emName;
    G4VPhysicsConstructor* emPhysicsList;
    G4VPhysicsConstructor* decay_List;
    G4VPhysicsConstructor* radioactiveDecay_List;
    
    std::vector<G4VPhysicsConstructor*>  hadronPhys;
        
    HadrontherapyPhysicsListMessenger* pMessenger;
};

#endif
```


[Go Top](#GoTop)<a id='HadrontherapyPhysicsList.cc'></a>
## src/HadrontherapyPhysicsList.cc
```cpp
#include "G4SystemOfUnits.hh"
#include "G4RunManager.hh"
#include "G4Region.hh"
#include "G4RegionStore.hh"
#include "HadrontherapyPhysicsList.hh"
#include "HadrontherapyPhysicsListMessenger.hh"
#include "HadrontherapyStepMax.hh"
#include "G4PhysListFactory.hh"
#include "G4VPhysicsConstructor.hh"
#include "G4HadronPhysicsQGSP_BIC_HP.hh"
#include "G4HadronPhysicsQGSP_BIC.hh"
#include "G4EmStandardPhysics_option4.hh"
#include "G4EmStandardPhysics.hh"
#include "G4EmExtraPhysics.hh"
#include "G4StoppingPhysics.hh"
#include "G4DecayPhysics.hh"
#include "G4HadronElasticPhysics.hh"
#include "G4HadronElasticPhysicsHP.hh"
#include "G4RadioactiveDecayPhysics.hh"
#include "G4IonBinaryCascadePhysics.hh"
#include "G4DecayPhysics.hh"
#include "G4NeutronTrackingCut.hh"
#include "G4LossTableManager.hh"
#include "G4UnitsTable.hh"
#include "G4ProcessManager.hh"
#include "G4IonFluctuations.hh"
#include "G4IonParametrisedLossModel.hh"
#include "G4EmProcessOptions.hh"
#include "G4ParallelWorldPhysics.hh"
#include "G4EmLivermorePhysics.hh"
#include "G4AutoDelete.hh"
#include "G4HadronPhysicsQGSP_BIC_AllHP.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapyPhysicsList::HadrontherapyPhysicsList() : G4VModularPhysicsList()
{
    G4LossTableManager::Instance();
    defaultCutValue = 1.*mm;
    cutForGamma     = defaultCutValue;
    cutForElectron  = defaultCutValue;
    cutForPositron  = defaultCutValue;
    
    pMessenger = new HadrontherapyPhysicsListMessenger(this);
    SetVerboseLevel(1);
    decay_List = new G4DecayPhysics();
    // Elecromagnetic physics
    //
    emPhysicsList = new G4EmStandardPhysics_option4();
    
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyPhysicsList::~HadrontherapyPhysicsList()
{
    delete pMessenger;
    delete emPhysicsList;
    delete decay_List;
    //delete radioactiveDecay_List;
    hadronPhys.clear();
    for(size_t i=0; i<hadronPhys.size(); i++)
    {
        delete hadronPhys[i];
    }
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyPhysicsList::ConstructParticle()
{
    decay_List -> ConstructParticle();
    
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyPhysicsList::ConstructProcess()
{
    // Transportation
    //
    AddTransportation();
    
    decay_List -> ConstructProcess();
    emPhysicsList -> ConstructProcess();
    
    
    //em_config.AddModels();
    
    // Hadronic physics
    //
    for(size_t i=0; i < hadronPhys.size(); i++)
    {
        hadronPhys[i] -> ConstructProcess();
    }
    
    // step limitation (as a full process)
    //
    AddStepMax();
    
    //Parallel world sensitivity
    //
    G4ParallelWorldPhysics* pWorld = new G4ParallelWorldPhysics("DetectorROGeometry");
    pWorld->ConstructProcess();
    
    return;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyPhysicsList::AddPhysicsList(const G4String& name)
{
    if (verboseLevel>1) {
        G4cout << "PhysicsList::AddPhysicsList: <" << name << ">" << G4endl;
    }
    if (name == emName) return;
    
    ///////////////////////////////////
    //   ELECTROMAGNETIC MODELS
    ///////////////////////////////////
    if (name == "standard_opt4") {
        emName = name;
        delete emPhysicsList;
        hadronPhys.clear();
        emPhysicsList = new G4EmStandardPhysics_option4();
        G4RunManager::GetRunManager() -> PhysicsHasBeenModified();
        G4cout << "THE FOLLOWING ELECTROMAGNETIC PHYSICS LIST HAS BEEN ACTIVATED: G4EmStandardPhysics_option4" << G4endl;
        
        ////////////////////////////////////////
        //   ELECTROMAGNETIC + HADRONIC MODELS
        ////////////////////////////////////////
        
    }  else if (name == "HADRONTHERAPY_1") {
        
        AddPhysicsList("standard_opt4");
        hadronPhys.push_back( new G4DecayPhysics());
        hadronPhys.push_back( new G4RadioactiveDecayPhysics());
        hadronPhys.push_back( new G4IonBinaryCascadePhysics());
        hadronPhys.push_back( new G4EmExtraPhysics());
        hadronPhys.push_back( new G4HadronElasticPhysicsHP());
        hadronPhys.push_back( new G4StoppingPhysics());
        hadronPhys.push_back( new G4HadronPhysicsQGSP_BIC_HP());
        hadronPhys.push_back( new G4NeutronTrackingCut());
        
        G4cout << "HADRONTHERAPY_1 PHYSICS LIST has been activated" << G4endl;
    }
    
    else if (name == "HADRONTHERAPY_2") {
        // HP models are switched off
        AddPhysicsList("standard_opt4");
        hadronPhys.push_back( new G4DecayPhysics());
        hadronPhys.push_back( new G4RadioactiveDecayPhysics());
        hadronPhys.push_back( new G4IonBinaryCascadePhysics());
        hadronPhys.push_back( new G4EmExtraPhysics());
        hadronPhys.push_back( new G4HadronElasticPhysics());
        hadronPhys.push_back( new G4StoppingPhysics());
        hadronPhys.push_back( new G4HadronPhysicsQGSP_BIC());
        hadronPhys.push_back( new G4NeutronTrackingCut());
        
        G4cout << "HADRONTHERAPY_2 PHYSICS LIST has been activated" << G4endl;    }
    else {
        G4cout << "PhysicsList::AddPhysicsList: <" << name << ">"
        << " is not defined"
        << G4endl;
    }
    
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyPhysicsList::AddStepMax()
{
    // Step limitation seen as a process
    // This process must exist in all threads.
    //
    HadrontherapyStepMax* stepMaxProcess  = new HadrontherapyStepMax();
    G4AutoDelete::Register( stepMaxProcess );
    
    auto particleIterator = GetParticleIterator();
    particleIterator->reset();
    while ((*particleIterator)()){
        G4ParticleDefinition* particle = particleIterator->value();
        G4ProcessManager* pmanager = particle->GetProcessManager();
        
        if (stepMaxProcess->IsApplicable(*particle) && pmanager)
        {
            pmanager ->AddDiscreteProcess(stepMaxProcess);
        }
    }
}
```

[Go Top](#GoTop)<a id='HadrontherapyActionInitialization.hh'></a>
## include/HadrontherapyActionInitialization.hh
```cpp
#ifndef HadrontherapyActionInitialization_h
#define HadrontherapyActionInitialization_h 1

#include "G4VUserActionInitialization.hh"

class HadrontherapyDetectorConstruction;
class G4GeneralParticleSource;

class HadrontherapyActionInitialization : public G4VUserActionInitialization
{
  public:
    HadrontherapyActionInitialization(/*HadrontherapyDetectorConstruction*/);
    virtual ~HadrontherapyActionInitialization();

    virtual void BuildForMaster() const;
    virtual void Build() const;

  private:
   // G4VUserDetectorConstruction* fDetectorConstruction;
  //  G4GeneralParticleSource *masterGPS;

    
    
};

#endif
```

[Go Top](#GoTop)<a id='HadrontherapyActionInitialization.cc'></a>
## src/HadrontherapyActionInitialization.cc
```cpp
#include "HadrontherapyActionInitialization.hh"
#include "HadrontherapyPrimaryGeneratorAction.hh"
#include "HadrontherapyRunAction.hh"
#include "HadrontherapySteppingAction.hh"
#include "HadrontherapyDetectorConstruction.hh"
#include "G4GeneralParticleSource.hh"

#include "HadrontherapyEventAction.hh"
#include "G4RunManager.hh"

//....oooOO0OOooo........oooOO0OOooo........oooOO0OOooo........oooOO0OOooo......

HadrontherapyActionInitialization::HadrontherapyActionInitialization(/*G4VUserDetectorConstruction* detConstruction*/)
: G4VUserActionInitialization()//,
 // fDetectorConstruction(detConstruction)
{}

//....oooOO0OOooo........oooOO0OOooo........oooOO0OOooo........oooOO0OOooo......

HadrontherapyActionInitialization::~HadrontherapyActionInitialization()
{//delete masterGPS;
}

//....oooOO0OOooo........oooOO0OOooo........oooOO0OOooo........oooOO0OOooo......

void HadrontherapyActionInitialization::BuildForMaster() const
{
	// In MT mode, to be clearer, the RunAction class for the master thread might be
	// different than the one used for the workers.
	// This RunAction will be called before and after starting the
	// workers.
	// For more details, please refer to :
	// https://twiki.cern.ch/twiki/bin/view/Geant4/Geant4MTForApplicationDevelopers
	//
	// HadrontherapyRunAction* runAction= new HadrontherapyRunAction(fDetectorConstruction);
	// SetUserAction(runAction);
  HadrontherapyRunAction* pRunAction = new HadrontherapyRunAction();
  SetUserAction(pRunAction);
}

//....oooOO0OOooo........oooOO0OOooo........oooOO0OOooo........oooOO0OOooo......

void HadrontherapyActionInitialization::Build() const
{       
 // Initialize the primary particles
  HadrontherapyPrimaryGeneratorAction *pPrimaryGenerator = new HadrontherapyPrimaryGeneratorAction();
  SetUserAction(pPrimaryGenerator);
	
  // Optional UserActions: run, event, stepping
  HadrontherapyRunAction* pRunAction = new HadrontherapyRunAction();
  SetUserAction(pRunAction);

	
  HadrontherapyEventAction* pEventAction = new HadrontherapyEventAction();
  SetUserAction(pEventAction);
	
  HadrontherapySteppingAction* steppingAction = new HadrontherapySteppingAction(pRunAction); 
  SetUserAction(steppingAction);  

}
```

[Go Top](#GoTop)<a id='HadrontherapyPrimaryGeneratorAction.hh'></a>
## include/HadrontherapyPrimaryGeneratorAction.hh
```cpp
#ifndef HadrontherapyPrimaryGeneratorAction_h
#define HadrontherapyPrimaryGeneratorAction_h 1

#include "G4VUserPrimaryGeneratorAction.hh"
#include "globals.hh"
#include "HadrontherapyPrimaryGeneratorMessenger.hh"
#include "G4RunManager.hh"
#include "G4ParticleGun.hh"

class G4GeneralParticleSource;
class G4Event;

class HadrontherapyPrimaryGeneratorMessenger;
class HadrontherapyPrimaryGeneratorAction : public G4VUserPrimaryGeneratorAction
{
    public:
    HadrontherapyPrimaryGeneratorAction();
    ~HadrontherapyPrimaryGeneratorAction();
    
    public:
    // Methods to change the parameters of primary particle generation
    // interactively
    void GeneratePrimaries(G4Event*);
    static G4bool ReadFile;
    
    inline void setNewSource(G4bool Varbool){NewSource= Varbool;};
    G4String PathSource;
    G4bool Readfile;
    G4bool NewSource;
    inline void setCalculatedPhaseSpaceFileIN(G4String val){calculatedPhaseSpaceFileIN=val;}
    
    
    private:
    void SetDefaultPrimaryParticle();
    
    
    G4String calculatedPhaseSpaceFileIN;
    void setGunCalculatedPhaseSpace();
    
    HadrontherapyPrimaryGeneratorMessenger *PrimaryGeneratorMessenger;
    G4ParticleGun *particleGuns;
    
    
    private:
    G4GeneralParticleSource* particleGun;
    G4double sigmaX;
    std::ofstream ofs;
    
};

#endif
```

[Go Top](#GoTop)<a id='HadrontherapyPrimaryGeneratorAction.cc'></a>
## include/HadrontherapyPrimaryGeneratorAction.cc
```cpp
#include "HadrontherapyPrimaryGeneratorAction.hh"
#include "HadrontherapyPrimaryGeneratorMessenger.hh"

#include "HadrontherapyMatrix.hh"
#include "HadrontherapyDetectorSD.hh"
#include "G4SystemOfUnits.hh"
#include "G4Event.hh"
#include "G4ParticleGun.hh"
#include "G4GeneralParticleSource.hh"
#include "G4ParticleTable.hh"
#include "G4ParticleDefinition.hh"
#include "Randomize.hh"
#include "G4IonTable.hh"


#include "G4VUserPrimaryGeneratorAction.hh"
#include "G4ParticleTable.hh"

#include "G4Event.hh"
#include "G4Timer.hh"

#include "G4RunManager.hh"



/////////////////////////////////////////////////////////////////////////////
HadrontherapyPrimaryGeneratorAction::HadrontherapyPrimaryGeneratorAction()
{
    PrimaryGeneratorMessenger = new HadrontherapyPrimaryGeneratorMessenger(this);
    SetDefaultPrimaryParticle();
    particleGun = new G4GeneralParticleSource();
    
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyPrimaryGeneratorAction::~HadrontherapyPrimaryGeneratorAction()
{
    delete PrimaryGeneratorMessenger;
    delete  particleGun;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyPrimaryGeneratorAction::SetDefaultPrimaryParticle()
{
    
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyPrimaryGeneratorAction::GeneratePrimaries(G4Event* anEvent)
{

    if(NewSource==true)
    {
        std::ifstream in(calculatedPhaseSpaceFileIN);
        G4double e, xpos, ypos, zpos,dirx,diry,dirz;
        G4int PDG;
        G4ThreeVector pos,dir;
        
        if(in.eof())
        {
            G4Exception("HadrontherapyPrimaryGeneratorAction", "NoParticles", FatalException, "No more particles in the file");
        }
        
        while(!in.eof())
        {
            
            in >> e >> xpos >> ypos >>zpos >>dirx>>diry>>dirz >> PDG;
            dir= G4ThreeVector(dirx,diry,dirz);
            particleGun->GetCurrentSource()->GetEneDist()->SetMonoEnergy(e);
            
            particleGun->GetCurrentSource()->GetParticlePosition().setX(xpos);
            particleGun->GetCurrentSource()->GetParticlePosition().setY(ypos);
            particleGun->GetCurrentSource()->GetParticlePosition().setZ(zpos);
            particleGun->GetCurrentSource()->GetAngDist()->SetParticleMomentumDirection(dir);
            
            G4ParticleDefinition* particleDef = nullptr;
            if (PDG > 1000000000)
            {
                int a=(PDG-1000000000)-(((PDG-1000000000)/10)*10);
                if(a>0)
                {
                    PDG=PDG-a;
                    particleDef = G4IonTable::GetIonTable()->GetIon(PDG);
                    G4String Nome = particleDef->GetParticleName();
                }
                
                else
                {
                    particleDef = G4IonTable::GetIonTable()->GetIon(PDG);
                    G4String Nome = particleDef->GetParticleName();
                }
            }
            
            else
            {
                particleDef = G4ParticleTable::GetParticleTable()->FindParticle(PDG);
            }
            
            particleGun->GetCurrentSource()->SetParticleDefinition(particleDef);
            particleGun->GeneratePrimaryVertex(anEvent);
            
        }
        
        in.close();
        
    }
    else
    {
        particleGun->GeneratePrimaryVertex(anEvent);
    }
    
}
```

[Go Top](#GoTop)<a id='HadrontherapyRunAction.hh'></a>
## include/HadrontherapyRunAction.hh
```cpp
#ifndef HadrontherapyRunAction_h
#define HadrontherapyRunAction_h 1

#include "G4UserRunAction.hh"
#include "G4RunManager.hh"
#include "globals.hh"

#include "HadrontherapyRBEAccumulable.hh"

class G4Run;
class HadrontherapyAnalysisManager;
class HadrontherapyDetectorConstruction;
class HadrontherapyRunMessenger;
class HadrontherapyFactory;

class HadrontherapyRunAction : public G4UserRunAction
{
public:
  HadrontherapyRunAction();
  ~HadrontherapyRunAction();

public:
  void BeginOfRunAction(const G4Run*);
  void EndOfRunAction(const G4Run* );
  void SelectEnergy(G4int); 

  void AddEMProcess();
  // Counts the number of electromagnetic processes
  // of primary particles in the phantom

  void AddHadronicProcess();
  // Counts the number of hadronic processes 
  // of primary particles in the phantom

private:  
  G4int electromagnetic;
  G4int hadronic;

  HadrontherapyRBEAccumulable fRBEAccumulable;
};
#endif
```

[Go Top](#GoTop)<a id='auto_keyword'></a>
## 'auto' keyword

The line ```cpp auto analysisManager = G4AnalysisManager::Instance(); ``` inside **void HadrontherapyRunAction::EndOfRunAction(const G4Run*)** prompted me to search about **auto keyword in C++** which led me to [this](https://www.tutorialspoint.com/What-does-an-auto-keyword-do-in-Cplusplus) site.
    
Auto was a keyword that C++ "inherited" from C that had been there nearly forever, but virtually never used. All this changed with the introduction of auto to do type deduction from the context in C++11. Before C++ 11, each data type needs to be explicitly declared at compile time, limiting the values of an expression at runtime but after a new version of C++, many keywords are included which allows a programmer to leave the type deduction to the compiler itself.

With type inference capabilities, we can spend less time having to write out things compiler already knows. As all the types are deduced in compiler phase only, the time for compilation increases slightly but it does not affect the runtime of the program.

The auto keyword specifies that the type of the variable that is begin declared will automatically be deduced from its initializer and for functions if their return type is auto then that will be evaluated by return type expression at runtime.

For example,
```cpp
#include<iostream>
#incllude<vector>
using namespace std;

int main() {
   vector<int> vec(10);       // Auto deduce type to be iterator of a vector of ints.
   for(auto it = vec.begin(); it != vec.end(); vec ++)
   {
      cin >> *it;
   }
   return 0;
}
```

[Go Top](#GoTop)<a id='HadrontherapyRunAction.cc'></a>
## include/HadrontherapyRunAction.cc
```cpp
#include "HadrontherapyRunAction.hh"
#include "HadrontherapyEventAction.hh"
#include "HadrontherapyAnalysis.hh"
#include "G4Run.hh"
#include "G4RunManager.hh"
#include "G4UImanager.hh"
#include "G4ios.hh"
#include "HadrontherapyDetectorConstruction.hh"
#include "G4SDManager.hh"
#include "G4Timer.hh"
#include "HadrontherapyRunAction.hh"
#include "HadrontherapyMatrix.hh"
#include "HadrontherapyRBE.hh"
#include "G4UnitsTable.hh"
#include "G4SystemOfUnits.hh"

#include <G4AccumulableManager.hh>

/////////////////////////////////////////////////////////////////////////////
HadrontherapyRunAction::HadrontherapyRunAction()
{
    G4AccumulableManager* accumulableManager = G4AccumulableManager::Instance();
    accumulableManager->RegisterAccumulable(&fRBEAccumulable);
    
    // Create analysis manager
    // The choice of analysis technology is done via selectin of a namespace
    // in Analysis.hh
    auto analysisManager =G4AnalysisManager::Instance();
    G4cout << "Using " << analysisManager -> GetType() << G4endl;
    
    analysisManager->SetVerboseLevel(1);
    analysisManager->SetFirstHistoId(1);
    
    // Comment out the following line to generate an N-tuple
    analysisManager-> SetFirstNtupleId(2);
    
    // Creating the histograms of primary kinetic
    // energy (Ekin) and of the energy deposited (Edep)
    // in the first voxel/slice of the water phantom
    analysisManager -> CreateH1("Ekin","Ekin the voxel", 400,20*MeV, 60*MeV);
    analysisManager -> CreateH1("Edep","Edep the voxel", 200, -10, 10*MeV);
    
    // Example of how to create an Ntuple (comment-out, if needed)
    //analysisManager->CreateNtuple("NYUPLA", "Edep and TrackL");
    //analysisManager->CreateNtupleDColumn("Ekin");
    
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyRunAction::~HadrontherapyRunAction()
{
    delete G4AnalysisManager::Instance();
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyRunAction::BeginOfRunAction(const G4Run* aRun)
{
    
    // Get analysis manager
    auto analysisManager = G4AnalysisManager::Instance();
    
    // Open an output file
    //
    G4String fileName = "Hadrontherapy";
    analysisManager->OpenFile(fileName);
    
    G4AccumulableManager* accumulableManager = G4AccumulableManager::Instance();
    accumulableManager->Reset();

    G4RunManager::GetRunManager()-> SetRandomNumberStore(true);
    G4cout << "Run " << aRun -> GetRunID() << " starts ..." << G4endl;

    HadrontherapyRBE *rbe = HadrontherapyRBE::GetInstance();
    if (rbe->IsCalculationEnabled() && IsMaster() && rbe->GetVerboseLevel() > 0)
    {
        rbe->PrintParameters();
    }
    
    electromagnetic = 0;
    hadronic = 0;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyRunAction::EndOfRunAction(const G4Run*)
{
    auto analysisManager = G4AnalysisManager::Instance();
    
    //G4cout << " Summary of Run " << aRun -> GetRunID() <<" :"<< G4endl;
    //G4cout << "Number of electromagnetic processes of primary particles in the phantom:"
    // 	   << electromagnetic << G4endl;
    //G4cout << "Number of hadronic processes of primary particles in the phantom:"
    //	   << hadronic << G4endl;
    G4AccumulableManager* accumulableManager = G4AccumulableManager::Instance();
    accumulableManager->Merge();

    // Tell the RBE class what we have accumulated...
    HadrontherapyRBE *rbe = HadrontherapyRBE::GetInstance();
    if (rbe->IsCalculationEnabled() && IsMaster())
    {
        if (rbe->IsAccumulationEnabled())
        {
            rbe->AddAlphaNumerator(fRBEAccumulable.GetAlphaNumerator());
            rbe->AddBetaNumerator(fRBEAccumulable.GetBetaNumerator());
            rbe->AddDenominator(fRBEAccumulable.GetDenominator());
            rbe->AddEnergyDeposit(fRBEAccumulable.GetEnergyDeposit());
        }
        else
        {
            rbe->SetAlphaNumerator(fRBEAccumulable.GetAlphaNumerator());
            rbe->SetBetaNumerator(fRBEAccumulable.GetBetaNumerator());
            rbe->SetDenominator(fRBEAccumulable.GetDenominator());
            rbe->SetEnergyDeposit(fRBEAccumulable.GetEnergyDeposit());
        }

        rbe->StoreAlphaAndBeta();
        rbe->StoreRBE();
    }
    
    analysisManager->Write();
    analysisManager->CloseFile();

}
/////////////////////////////////////////////////////////////////////////////
void HadrontherapyRunAction::AddEMProcess()
{
    electromagnetic += 1;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyRunAction::AddHadronicProcess()
{
    hadronic += 1;
}
```

[Go Top](#GoTop)<a id='HadrontherapyEventAction.hh'></a>
## include/HadrontherapyEventAction.hh
```cpp
#ifndef HadrontherapyEventAction_h
#define HadrontherapyEventAction_h 1

#include "G4UserEventAction.hh"
#include "globals.hh"

class HadrontherapyMatrix;
class HadrontherapyEventActionMessenger;

class HadrontherapyEventAction : public G4UserEventAction
{
public:
  HadrontherapyEventAction();
  ~HadrontherapyEventAction();

public:
  void BeginOfEventAction(const G4Event*);
  void EndOfEventAction(const G4Event*);

  void SetPrintModulo(G4int val)
  {
    printModulo = val;
  };

  void SetDrawFlag(G4String val)
  {
    drawFlag = val;
  };

private: 
  G4String drawFlag; //Visualisation flag
  G4int hitsCollectionID;
  //HadrontherapyMatrix *matrix; 
  G4int printModulo;  
  HadrontherapyEventActionMessenger* pointerEventMessenger;
};

//kp: It seems, this was forgotten to be deleted once it was used
//    as a template to write this class.
class B4aEventAction : public G4UserEventAction
{
public:
    B4aEventAction();
    virtual ~B4aEventAction();
    
    virtual void  BeginOfEventAction(const G4Event* event);
    virtual void    EndOfEventAction(const G4Event* event);
    
    void AddAbs(G4double de, G4double dl);
    void AddGap(G4double de, G4double dl);
    
private:
    G4double  fEnergyAbs;
    G4double  fEnergyGap;
    G4double  fTrackLAbs;
    G4double  fTrackLGap;
};

// inline functions

inline void B4aEventAction::AddAbs(G4double de, G4double dl) {
    fEnergyAbs += de;
    fTrackLAbs += dl;
}

inline void B4aEventAction::AddGap(G4double de, G4double dl) {
    fEnergyGap += de;
    fTrackLGap += dl;
}


#endif
```

[Go Top](#GoTop)<a id='HadrontherapyEventAction.cc'></a>
## include/HadrontherapyEventAction.cc
```cpp
#include "G4SystemOfUnits.hh"
#include "G4Event.hh"
#include "G4EventManager.hh"
#include "G4HCofThisEvent.hh"
#include "G4VHitsCollection.hh"
#include "G4SDManager.hh"
#include "G4VVisManager.hh"

#include "HadrontherapyEventAction.hh"
#include "HadrontherapyDetectorHit.hh"
#include "HadrontherapyDetectorSD.hh"
#include "HadrontherapyDetectorConstruction.hh"
#include "HadrontherapyMatrix.hh"
#include "HadrontherapyEventActionMessenger.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapyEventAction::HadrontherapyEventAction() :
drawFlag("all" ),printModulo(10), pointerEventMessenger(0)
{
    hitsCollectionID = -1;
    pointerEventMessenger = new HadrontherapyEventActionMessenger(this);
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyEventAction::~HadrontherapyEventAction()
{
    delete pointerEventMessenger;
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyEventAction::BeginOfEventAction(const G4Event*)
{
    G4SDManager* pSDManager = G4SDManager::GetSDMpointer();
    if(hitsCollectionID == -1)
        hitsCollectionID = pSDManager -> GetCollectionID("HadrontherapyDetectorHitsCollection");
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyEventAction::EndOfEventAction(const G4Event* evt)
{
    if(hitsCollectionID < 0)
        return;
    G4HCofThisEvent* HCE = evt -> GetHCofThisEvent();
    
    // Clear voxels hit list
    HadrontherapyMatrix* matrix = HadrontherapyMatrix::GetInstance();
    if (matrix) matrix -> ClearHitTrack();
    
    if(HCE)
    {
        HadrontherapyDetectorHitsCollection* CHC = (HadrontherapyDetectorHitsCollection*)(HCE -> GetHC(hitsCollectionID));
        if(CHC)
        {
            if(matrix)
            {
                // Fill the matrix with the information: voxel and associated energy deposit
                // in the detector at the end of the event
                
                G4int HitCount = CHC -> entries();
                for (G4int h=0; h<HitCount; h++)
                {
                    G4int i = ((*CHC)[h]) -> GetXID();
                    G4int j = ((*CHC)[h]) -> GetYID();
                    G4int k = ((*CHC)[h]) -> GetZID();
                    G4double energyDeposit = ((*CHC)[h]) -> GetEdep();
                    matrix -> Fill(i, j, k, energyDeposit/MeV);
                }
            }
        }
    }
}
```

[Go Top](#GoTop)<a id='HHadrontherapySteppingAction.hh'></a>
## include/HadrontherapySteppingAction.hh
```cpp
#ifndef HadrontherapySteppingAction_h
#define HadrontherapySteppingAction_h 1

#include "G4UserSteppingAction.hh"
#include "G4Event.hh"
#include "G4EventManager.hh"
#include "G4ios.hh"
#include "globals.hh"

class HadrontherapyRunAction;
class HadrontherapySteppingMessenger;


#ifndef G4NOHIST
class HepTupleManager;
class HepHistogram;
#endif

class HadrontherapySteppingAction : public G4UserSteppingAction
{
public:
  HadrontherapySteppingAction(HadrontherapyRunAction*);
  ~HadrontherapySteppingAction();
  
  void UserSteppingAction(const G4Step*);

private:
  HadrontherapyRunAction* runAction;
};
#endif
```

[Go Top](#GoTop)<a id='HadrontherapySteppingAction.cc'></a>
## src/HadrontherapySteppingAction.cc
```cpp
#include "G4SteppingManager.hh"
#include "G4TrackVector.hh"
#include "HadrontherapySteppingAction.hh"
#include "G4ios.hh"
#include "G4SteppingManager.hh"
#include "G4Track.hh"
#include "G4Step.hh"
#include "G4StepPoint.hh"
#include "G4TrackStatus.hh"
#include "G4TrackVector.hh"
#include "G4ParticleDefinition.hh"
#include "G4ParticleTypes.hh"
#include "G4UserEventAction.hh"
#include "G4TransportationManager.hh"
#include "G4VSensitiveDetector.hh"
#include "HadrontherapyRunAction.hh"
#include "HadrontherapyMatrix.hh"
#include "G4SystemOfUnits.hh"

/////////////////////////////////////////////////////////////////////////////
HadrontherapySteppingAction::HadrontherapySteppingAction( HadrontherapyRunAction *run)
{
    runAction = run;
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapySteppingAction::~HadrontherapySteppingAction()
{
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapySteppingAction::UserSteppingAction(const G4Step* aStep)
{
    
    // The followings are calls to usefuls information retrieved at the step level
    // Please, comment out them if want to use
    
   // G4Track* theTrack = aStep->GetTrack();
    
    G4StepPoint* PreStep = aStep->GetPreStepPoint();
    G4StepPoint* PostStep = aStep->GetPostStepPoint();
    
    G4TouchableHandle touchPreStep = PreStep->GetTouchableHandle();
    G4TouchableHandle touchPostStep = PostStep->GetTouchableHandle();
    
    //G4double PreStepX =PreStep->GetPosition().x();
    //G4double PreStepY =PreStep->GetPosition().y();
    //G4double PreStepZ =PreStep->GetPosition().z();
    
    //G4double PostStepX =PostStep->GetPosition().x();
    //G4double PostStepY =PostStep->GetPosition().y();
    //G4double PostStepZ  =PostStep->GetPosition().z();
    
    //To get the current volume:
    G4VPhysicalVolume* volumePre = touchPreStep->GetVolume();
    //G4VPhysicalVolume* volumePost =touchPostStep->GetVolume();
    
    //To get its name:
    G4String namePre = volumePre->GetName();

    
    // positions in the global coordinate system:
    //G4ThreeVector posPreStep  = PreStep->GetPosition();
    //G4ThreeVector posPostStep = PostStep->GetPosition();
    
    //G4int eventNum = G4RunManager::GetRunManager() -> GetCurrentEvent() -> GetEventID();
    
    //G4double parentID =aStep->GetTrack()->GetParentID();
    //G4double trackID =aStep->GetTrack()->GetTrackID();
    
    G4double eKin = aStep -> GetPreStepPoint() -> GetKineticEnergy();
    
    G4double PosX = aStep->GetTrack()->GetPosition().x();
    G4double PosY = aStep->GetTrack()->GetPosition().y();
    G4double PosZ = aStep->GetTrack()->GetPosition().z();
    
    G4String volume=  aStep->GetTrack()->GetVolume()->GetName();
    G4Track* theTrack = aStep->GetTrack();
    
    //G4String material= aStep -> GetTrack() -> GetMaterial() -> GetName();
    //G4cout << "material   " << material << G4endl;
    //G4String volume=  aStep->GetTrack()->GetVolume()->GetName();
    //G4String pvname= pv-> GetName();
    
    G4String particleName = aStep->GetTrack()->GetDefinition()->GetParticleName();

    G4double momentumX =  aStep->GetTrack()->GetMomentumDirection().x();
    G4double momentumY =  aStep->GetTrack()->GetMomentumDirection().y();
    G4double momentumZ =  aStep->GetTrack()->GetMomentumDirection().z();
    
    
    G4ParticleDefinition *particleDef = theTrack -> GetDefinition();
    G4int pdg = particleDef ->GetPDGEncoding();
    
    if(namePre == "VirtualLayer")
    {
        std::ofstream WriteDataIn("Virtual_Layer.txt", std::ios::app);
        WriteDataIn
        
        <<   eKin             <<" " //  1
        <<   PosX             <<" " //  2
        <<   PosY             <<" " //  3
        <<   PosZ             <<" " //  4
        <<   momentumX        <<" " //  5
        <<   momentumY        <<" " //  6
        <<   momentumZ        <<" " //  7
        <<   pdg
        //<<   theTrack         << '\t' << "   "
        
        <<   G4endl;
        
        theTrack -> SetTrackStatus(fKillTrackAndSecondaries);
        
        
    }
    
    
    
  
}
```

[Go Top](#GoTop)<a id='HadrontherapyInteractionParameters.hh'></a>
## include/HadrontherapyInteractionParameters.hh
```cpp
#ifndef HadrontherapyInteractionParameters_H
#define HadrontherapyInteractionParameters_H 1

#include "G4EmCalculator.hh"
#include "G4NistMaterialBuilder.hh"
#include "G4NistElementBuilder.hh"

class HadrontherapyDetectorConstruction;
class HadrontherapyParameterMessenger; 
class G4ParticleDefinition;
class G4Material;

class HadrontherapyInteractionParameters : public G4EmCalculator 
{
public:

  HadrontherapyInteractionParameters(G4bool);
  ~HadrontherapyInteractionParameters();

  // Get data for Mass SP    
  // G4NistMaterialBuilder class materials
  // User must provide: material kinetic energy lower limit, kinetic energy upper limit, number of points to retrieve,
  // [particle], [output filename].

  G4bool GetStoppingTable (const G4String& vararg);
  G4double GetStopping (G4double energy,
			const G4ParticleDefinition*, 
			const G4Material*, 
			G4double density = 0.);

  void ListOfNistMaterials (const G4String& vararg);
  void BeamOn();
  bool ParseArg (const G4String& vararg);	

private:
  G4Material* GetNistMaterial(G4String material);
  G4NistElementBuilder* nistEle;
  G4NistMaterialBuilder* nistMat;
  std::ofstream outfile;
  std::ostream data;
  G4Material* Pmaterial;
  HadrontherapyParameterMessenger* pMessenger; 
  bool beamFlag;

  G4double kinEmin, kinEmax, npoints;
  G4String particle, material, filename; 
  G4double dedxtot, density;
  std::vector<G4double> energy;
  std::vector<G4double> massDedx;

};
#endif
```

[Go Top](#GoTop)<a id='HadrontherapyInteractionParameters.cc'></a>
## include/HadrontherapyInteractionParameters.cc
```cpp
#include <fstream>
#include <iostream>
#include <sstream>
#include <cmath>
#include <vector>

#include "HadrontherapyInteractionParameters.hh"
#include "HadrontherapyParameterMessenger.hh"
#include "HadrontherapyDetectorConstruction.hh"

#include "globals.hh"
#include "G4SystemOfUnits.hh"
#include "G4UnitsTable.hh"
#include "G4UImanager.hh"
#include "G4RunManager.hh"
#include "G4LossTableManager.hh"
#include "G4Material.hh"
#include "G4MaterialCutsCouple.hh"
#include "G4ParticleDefinition.hh"
#include "G4ParticleTable.hh"
#include "G4NistManager.hh"
#include "G4Element.hh"
#include "G4StateManager.hh"

HadrontherapyInteractionParameters::HadrontherapyInteractionParameters(G4bool wantMessenger): 
    nistEle(new G4NistElementBuilder(0)),										  
    nistMat(new G4NistMaterialBuilder(nistEle, 0)),									  
    data(G4cout.rdbuf()), 
    pMessenger(0),
    beamFlag(false)

{
    if (wantMessenger) pMessenger = new HadrontherapyParameterMessenger(this); 
}

HadrontherapyInteractionParameters::~HadrontherapyInteractionParameters()
{
    if (pMessenger) delete pMessenger; 
    delete nistMat; 
    delete nistEle; 
}

G4double HadrontherapyInteractionParameters::GetStopping (G4double ene, 
	                                                  const G4ParticleDefinition* pDef, 
						          const G4Material* pMat,
							  G4double dens)
{
    if (dens) return ComputeTotalDEDX(ene, pDef, pMat)/dens;
    return ComputeTotalDEDX(ene, pDef, pMat);
}
bool HadrontherapyInteractionParameters::GetStoppingTable(const G4String& vararg)
{
	// Check arguments
	if ( !ParseArg(vararg)) return false;
	// Clear previous energy & mass sp vectors
        energy.clear(); 
        massDedx.clear();
	// log scale 
	if (kinEmin != kinEmax && npoints >1)
	{
	    G4double logmin = std::log10(kinEmin);
	    G4double logmax = std::log10(kinEmax); 
	    G4double en;
	    // uniform log space
            for (G4double c = 0.; c < npoints; c++)
	       {
		    en = std::pow(10., logmin + ( c*(logmax-logmin)  / (npoints - 1.)) );  
		    energy.push_back(en/MeV);
		    dedxtot =  ComputeTotalDEDX (en, particle, material);
	            massDedx.push_back ( (dedxtot / density)/(MeV*cm2/g) );
	       }
	}
	else // one point only
	{
	    energy.push_back(kinEmin/MeV);
	    dedxtot =  ComputeTotalDEDX (kinEmin, particle, material);
	    massDedx.push_back ( (dedxtot / density)/(MeV*cm2/g) );
	}

    G4cout.precision(6);  
    data <<  "MeV             " << "MeV*cm2/g      " << particle << " (into " << 
	    material << ", density = " << G4BestUnit(density,"Volumic Mass") << ")" << G4endl;
    data << G4endl;
    data << std::left << std::setfill(' ');
    for (size_t i=0; i<energy.size(); i++){
		data << std::setw(16) << energy[i] << massDedx[i] << G4endl;
	}
    outfile.close();
    // This will plot

// Info to user
    G4String ofName = (filename == "") ? "User terminal": filename;
    G4cout << "User choice:\n";
    G4cout << "Kinetic energy lower limit= "<< G4BestUnit(kinEmin,"Energy") << 
	      ", Kinetic energy upper limit= " << G4BestUnit(kinEmax,"Energy") << 
	         ", npoints= "<< npoints << ", particle= \"" << particle << 
		 "\", material= \"" << material << "\", filename= \""<< 
		 ofName << "\"" << G4endl;
    return true;
}
// Search for user material choice inside G4NistManager database
G4Material* HadrontherapyInteractionParameters::GetNistMaterial(G4String mat)
{
    Pmaterial = G4NistManager::Instance()->FindOrBuildMaterial(mat);
    if (Pmaterial) density = Pmaterial -> GetDensity(); 
    return Pmaterial;
}
// Parse arguments line
bool HadrontherapyInteractionParameters::ParseArg(const G4String& vararg)
{
  kinEmin = kinEmax = npoints = 0.;
  particle = material = filename = "";
  // set internal variables
  std::istringstream strParam(vararg);
  // TODO here check for number and parameters consistency 
  strParam >> std::skipws >> material >> kinEmin >> kinEmax >> npoints >> particle >> filename;
  // npoints must be an integer!
  npoints = std::floor(npoints); 

// Check that  kinEmax >= kinEmin > 0 &&  npoints >= 1 
// TODO NIST points and linear scale
   if (kinEmax == 0. && kinEmin > 0. ) kinEmax = kinEmin;
   if (kinEmax == 0. && kinEmin == 0. ) kinEmax = kinEmin = 1.*MeV;
   if (kinEmax < kinEmin) 
   {
     G4cout << "WARNING: kinEmin must not exceed kinEmax!" << G4endl;
     G4cout << "Usage: /parameter/command  material EkinMin EKinMax nPoints [particle] [output filename]" << G4endl; 	
     return false;
   }
  if (npoints < 1) npoints = 1;

    // check if element/material is into database
    if (!GetNistMaterial(material) )
	   {
	     G4cout << "WARNING: material \"" << material << "\" doesn't exist in NIST elements/materials"
	 	       " table [$G4INSTALL/source/materials/src/G4NistMaterialBuilder.cc]" << G4endl; 
	     G4cout << "Use command \"/parameter/nist\" to see full materials list" << G4endl; 
	     return false;
	   }
    // Check for particle
    if (particle == "") particle = "proton"; // default to "proton"
    else if ( !FindParticle(particle) )
	   {
		G4cout << "WARNING: Particle \"" << particle << "\" isn't supported." << G4endl;
		G4cout << "Try the command \"/particle/list\" to get full supported particles list." << G4endl;
		G4cout << "If you are interested in an ion that isn't in this list you must give it to the particle gun."
		          "\nTry the commands:\n/gun/particle ion"
			  "\n/gun/ion <atomic number> <mass number> <[charge]>" << G4endl << G4endl;
		return false;
	   }
    // start physics by forcing a G4RunManager::BeamOn(): 
    BeamOn();
    // Set output file
    if( filename != "" ) 
       {
          outfile.open(filename,std::ios_base::trunc); // overwrite existing file
          data.rdbuf(outfile.rdbuf());
       }
    else data.rdbuf(G4cout.rdbuf());	// output is G4cout!                
    return true;
}
// Force physics tables build
void HadrontherapyInteractionParameters::BeamOn()
{
    // first check if RunManager is above G4State_Idle 
    G4StateManager* mState = G4StateManager::GetStateManager();
    G4ApplicationState  aState = mState -> GetCurrentState(); 
    if ( aState <= G4State_Idle && beamFlag == false)
	 {
	    G4cout << "Issuing a G4RunManager::beamOn()... "; 
	    G4cout << "Current Run State is " << mState -> GetStateString( aState ) << G4endl; 
	    G4RunManager::GetRunManager() -> BeamOn(0);
	    beamFlag = true;
	 }

}
// print a list of Nist elements and materials
void HadrontherapyInteractionParameters::ListOfNistMaterials(const G4String& vararg)
{
/*
 $G4INSTALL/source/materials/src/G4NistElementBuilder.cc
 You can also construct a new material by the ConstructNewMaterial method:
 see $G4INSTALL/source/materials/src/G4NistMaterialBuilder.cc
*/
    // Get simplest full list
     if (vararg =="list")
	  {
	    const std::vector<G4String>& vec =  nistMat -> GetMaterialNames(); 
	    for (size_t i=0; i<vec.size(); i++)
	     {
	        G4cout << std::setw(12) << std::left << i+1 << vec[i] << G4endl;
	     }
	    G4cout << G4endl;
          }
    else if (vararg =="all" || vararg =="simple" || vararg =="compound" || vararg =="hep" )
	{
		nistMat -> ListMaterials(vararg);
	}
}
```

[Go Top](#GoTop)<a id='HadrontherapyAnalysis.hh'></a>
## include/HadrontherapyAnalysis.hh
```cpp
#ifndef Analysis_h
#define Analysis_h 1

//#include "g4root.hh"  //kp: http://www.apc.univ-paris7.fr/~franco/g4doxy/html/g4root_8hh-source.html
#include "g4csv.hh"     //kp: http://www.apc.univ-paris7.fr/~franco/g4doxy/html/g4csv_8hh-source.html
//#include "g4xml.hh"

#endif
```
I thought that was all about HadrontherapyAnalysis class, but it turns out this class has been declared and defined in [HadrontherapyMatrix.hh](#HadrontherapyMatrix.hh)/[.cc](#HadrontherapyMatrix.cc).

[Go Top](#GoTop)<a id='HadrontherapyAnalysis.cc'></a>
## src/HadrontherapyAnalysis.cc
There is no file of this name, because as we see above, there is no class or any definition.

#### More on HadrontherapyAnalysis.cc
To my surprise, there is no file called HadrontherapyAnalysis.cc in src directory, whereas there is HadrontherapyAnalysis.hh (see above) in include directory. This header file has the following code.
```cpp
#ifndef Analysis_h
#define Analysis_h 1

//#include "g4root.hh"
#include "g4csv.hh"
//#include "g4xml.hh"

#endif
```
So, may be we can control what output to produce by enabling the corresponding line here. For example, to produce ROOT output files, we should enable the “#include “g4root.hh” file.

But, one thing still was not clear to me initially - how is it possible to call a line as follows (in [hadrontherapy.cc](#hadrontherapy.cc)):

    // Initialize analysis
    HadrontherapyAnalysis* analysis = HadrontherapyAnalysis::GetInstance();
    
After some digging through, I found that the class HadrontherapyAnalysis is defined in [HadrontherapyMatrix.hh](#HadrontherapyMatrix.hh)/[.cc](#HadrontherapyMatrix.cc).


[Go Top](#GoTop)<a id='HadrontherapyMatrix.hh'></a>
## include/HadrontherapyMatrix.hh
This file has the declarations/definitions of **struct ion** as well as two classes **HadrontherapyAnalysis** and **HadrontherapyMatrix**.
```cpp
#ifndef HadrontherapyMatrix_H
#define HadrontherapyMatrix_H 1
#include <G4ParticleDefinition.hh>
#include "globals.hh"
#include <vector>
#include <fstream>
//#include "g4csv.hh"


#ifndef HADRONTHERAPYANALYSISMANAGER_HH
#define HADRONTHERAPYANALYSISMANAGER_HH 1

class HadrontherapyAnalysisFileMessenger;

/**
 * A class for connecting the simulation to an analysis package.
 */
class HadrontherapyAnalysis
{
private:
    /**
     * Analysis manager is a singleton object (there is only one instance).
     * The pointer to this object is available through the use of the method GetInstance();
     *
     * @see GetInstance
     */
    HadrontherapyAnalysis();
    
    
    
public:
    ~HadrontherapyAnalysis();
    
    /**
     * Get the pointer to the analysis manager.
     */
    static HadrontherapyAnalysis* GetInstance();
    
    
    
    static HadrontherapyAnalysis* instance;
    HadrontherapyAnalysisFileMessenger* fMess;
    
};

#endif

// The information: energy deposit and position in the phantom
// is stored in a matrix

// type struct useful to store nucludes data


struct ion
{ 
  G4bool isPrimary;   // true if particle is primary
  G4int PDGencoding;  // Particle data group id for the particle
  //G4String extName; //  AZ[excitation energy]: like He3[1277.4], He4[0.0], Li7[231.4], ...
  G4String name;   	 // simple name without excitation energy: He3, He4, Li7, ...
  std::string::size_type len; 	 // name length
  G4int Z; 		 // atomic number
  G4int A; 		 // mass number
  G4double *dose; 	 // pointer to dose matrix
  unsigned int    *fluence;  // pointer to fluence matrix
  //friend G4bool operator<(const ion& a, const ion& b) {return (a.Z == b.Z) ? b.A < a.A : b.Z < a.Z ;}
  G4bool operator<(const ion& a) const{return (this->Z == a.Z) ? this-> A < a.A : this->Z < a.Z ;}
};

class HadrontherapyMatrix 
{
private:
  HadrontherapyMatrix(G4int numberOfVoxelAlongX, 
		      G4int numberOfVoxelAlongY, 
		      G4int numberOfVoxelAlongZ,
		      G4double massOfVoxel); //< this is supposed to be a singleton


public:

  ~HadrontherapyMatrix();
  // Get object instance only
  static HadrontherapyMatrix* GetInstance();
  // Make & Get instance
  static HadrontherapyMatrix* GetInstance(G4int nX, G4int nY, G4int nZ, G4double mass);

  static G4bool secondary;
  // Full list of generated nuclides
    
    
  void PrintNuclides(); 
  // Hit array marker (useful to avoid multiple counts of fluence)
  void ClearHitTrack();
  G4int* GetHitTrack(G4int i, G4int j, G4int k);

  // All the elements of the matrix are initialised to zero
  void Initialize(); 
  void Clear();
  // Fill DOSE/fluence matrix for particle: 
  // if fluence parameter is true then fluence at voxel (i, j, k) is increased 
  // else energyDeposit fill the dose matrix for voxel (i,j,k) 
  G4bool Fill(G4int, G4ParticleDefinition* particleDef, G4int i, G4int j, G4int k, G4double energyDeposit, G4bool fluence=false); 

  // Fill TOTAL DOSE matrix for primary particles only 
  void Fill(G4int i, G4int j, G4int k, G4double energyDeposit);
  // The matrix is filled with the energy deposit 
  // in the element corresponding to the voxel of the phantom where
  // the energy deposit was registered
  
  // Store the information of the matrix in a ntuple and in 
  // a 1D Histogram
  //void TotalEnergyDeposit();
   
  // Store single matrix data to filename 
  void StoreMatrix(G4String file, void* data,size_t psize);
  // Store all fluence data to filenames
  void StoreFluenceData();
  // Store all dose data to filenames
  void StoreDoseData();

  // Store all data (except the total dose) to ONE filename
  void StoreDoseFluenceAscii(G4String filename = "");
  

    
  inline G4int Index(G4int i, G4int j, G4int k) { return (i * numberOfVoxelAlongY + j) * numberOfVoxelAlongZ + k; } 
  // Get a unique index from  a three dimensional one 

  G4double * GetMatrix(){return matrix;}

  G4int GetNvoxel(){return numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ;}
  // Total number of voxels read only access  
  G4int GetNumberOfVoxelAlongX(){return numberOfVoxelAlongX;}
  G4int GetNumberOfVoxelAlongY(){return numberOfVoxelAlongY;}
  G4int GetNumberOfVoxelAlongZ(){return numberOfVoxelAlongZ;}
private:

  static HadrontherapyMatrix* instance;
  G4int numberOfVoxelAlongX;
  G4int numberOfVoxelAlongY;
  G4int numberOfVoxelAlongZ;
  G4double massOfVoxel;

  G4double* matrix;
  G4int* hitTrack;
  G4String stdFile, filename;
  std::ofstream ofs;

  // Dose&fluence data store 
  std::vector <ion> ionStore;
  // want secondary particles?
  G4double doseUnit;
};
#endif
```

[Go Top](#GoTop)  [Dose Calculations](#DoseCalculations) <a id='HadrontherapyMatrix.cc'></a> 
## src/HadrontherapyMatrix.cc
```cpp
#include <fstream>
#include <iostream>
#include <sstream>
#include <iomanip>

#include "HadrontherapyMatrix.hh"
#include "HadrontherapyPrimaryGeneratorAction.hh"
#include "globals.hh"
#include "G4SystemOfUnits.hh"
#include "G4RunManager.hh"
#include "G4ParticleGun.hh"
#include "HadrontherapySteppingAction.hh"
#include "HadrontherapyAnalysisFileMessenger.hh"
#include "G4SystemOfUnits.hh"
#include <time.h>

HadrontherapyAnalysis* HadrontherapyAnalysis::instance = 0;
/////////////////////////////////////////////////////////////////////////////

HadrontherapyAnalysis::HadrontherapyAnalysis()
{
    fMess = new HadrontherapyAnalysisFileMessenger(this);
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyAnalysis::~HadrontherapyAnalysis()
{
    delete fMess;
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyAnalysis* HadrontherapyAnalysis::GetInstance(){
    
  if (instance == 0) instance = new HadrontherapyAnalysis;
    return instance;
}

HadrontherapyMatrix* HadrontherapyMatrix::instance = NULL;
G4bool HadrontherapyMatrix::secondary = false;


// Only return a pointer to matrix
HadrontherapyMatrix* HadrontherapyMatrix::GetInstance()
{
    return instance;
}

/////////////////////////////////////////////////////////////////////////////
// This STATIC method delete (!) the old matrix and rewrite a new object returning a pointer to it
// TODO A check on the parameters is required!
HadrontherapyMatrix* HadrontherapyMatrix::GetInstance(G4int voxelX, G4int voxelY, G4int voxelZ, G4double mass)
{
    if (instance) delete instance;
    instance = new HadrontherapyMatrix(voxelX, voxelY, voxelZ, mass);
    instance -> Initialize();
    return instance;
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyMatrix::HadrontherapyMatrix(G4int voxelX, G4int voxelY, G4int voxelZ, G4double mass):
stdFile("Dose.out"),
doseUnit(gray)
{
    // Number of the voxels of the phantom
    // For Y = Z = 1 the phantom is divided in slices (and not in voxels)
    // orthogonal to the beam axis
    numberOfVoxelAlongX = voxelX;
    numberOfVoxelAlongY = voxelY;
    numberOfVoxelAlongZ = voxelZ;
    massOfVoxel = mass;
    
    
    // Create the dose matrix
    matrix = new G4double[numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ];
    if (matrix)
    {
        G4cout << "HadrontherapyMatrix: Memory space to store physical dose into " <<
        numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ <<
        " voxels has been allocated " << G4endl;
    }
    
    
    else G4Exception("HadrontherapyMatrix::HadrontherapyMatrix()", "Hadrontherapy0005", FatalException, "Can't allocate memory to store physical dose!");
    
    
    // Hit voxel (TrackID) marker
    // This array mark the status of voxel, if a hit occur, with the trackID of the particle
    // Must be initialized
    
    hitTrack = new G4int[numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ];
    ClearHitTrack();
}

/////////////////////////////////////////////////////////////////////////////
HadrontherapyMatrix::~HadrontherapyMatrix()
{
    delete[] matrix;
    delete[] hitTrack;
    Clear();
}

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyMatrix::Clear()
{
    for (size_t i=0; i<ionStore.size(); i++)
    {
        delete[] ionStore[i].dose;
        delete[] ionStore[i].fluence;
    }
    ionStore.clear();
}

/////////////////////////////////////////////////////////////////////////////
// Initialise the elements of the matrix to zero

void HadrontherapyMatrix::Initialize()
{
    // Clear ions store
    Clear();
    // Clear dose
    for(int i=0;i<numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ;i++)
    {
        matrix[i] = 0;
    }
}

/////////////////////////////////////////////////////////////////////////////
// Print generated nuclides list

/////////////////////////////////////////////////////////////////////////////
void HadrontherapyMatrix::PrintNuclides()
{
    for (size_t i=0; i<ionStore.size(); i++)
    {
        G4cout << ionStore[i].name << G4endl;
    }
}

/////////////////////////////////////////////////////////////////////////////
// Clear Hit voxel (TrackID) markers

void HadrontherapyMatrix::ClearHitTrack()
{
    for(G4int i=0; i<numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ; i++) hitTrack[i] = 0;
}


// Return Hit status
G4int* HadrontherapyMatrix::GetHitTrack(G4int i, G4int j, G4int k)
{
    return &(hitTrack[Index(i,j,k)]);
}



/////////////////////////////////////////////////////////////////////////////
// Dose methods...
// Fill DOSE/fluence matrix for secondary particles:
// If fluence parameter is true (default value is FALSE) then fluence at voxel (i, j, k) is increased.
// The energyDeposit parameter fill the dose matrix for voxel (i,j,k)
/////////////////////////////////////////////////////////////////////////////
G4bool HadrontherapyMatrix::Fill(G4int trackID,
                                 G4ParticleDefinition* particleDef,
                                 G4int i, G4int j, G4int k,
                                 G4double energyDeposit,
                                 G4bool fluence)
{
    
    if ( (energyDeposit <=0. && !fluence) || !secondary) return false;
    
    // Get Particle Data Group particle ID
    G4int PDGencoding = particleDef -> GetPDGEncoding();
    PDGencoding -= PDGencoding%10;
    
    // Search for already allocated data...
    for (size_t l=0; l < ionStore.size(); l++)
    {
        if (ionStore[l].PDGencoding == PDGencoding )
        {   // Is it a primary or a secondary particle?
            
            if ( (trackID ==1 && ionStore[l].isPrimary) || (trackID !=1 && !ionStore[l].isPrimary))
            {
                if (energyDeposit > 0.)
                    
                    ionStore[l].dose[Index(i, j, k)] += energyDeposit;
                
                // Fill a matrix per each ion with the fluence
                
                if (fluence) ionStore[l].fluence[Index(i, j, k)]++;
                return true;
            }
        }
    }
    
    G4int Z = particleDef-> GetAtomicNumber();
    G4int A = particleDef-> GetAtomicMass();
    G4String fullName = particleDef -> GetParticleName();
    G4String name = fullName.substr (0, fullName.find("[") ); // cut excitation energy
    
    // Let's put a new particle in our store...
   
    
    ion newIon =
    {
        (trackID == 1) ? true:false,
        PDGencoding,
        name,
        name.length(),
        Z,
        A,
        new G4double[numberOfVoxelAlongX * numberOfVoxelAlongY * numberOfVoxelAlongZ],
        new unsigned int[numberOfVoxelAlongX * numberOfVoxelAlongY * numberOfVoxelAlongZ]
    };
    
    
    // Initialize data
    if (newIon.dose && newIon.fluence)
    {
        for(G4int q=0; q<numberOfVoxelAlongX*numberOfVoxelAlongY*numberOfVoxelAlongZ; q++)
        {
            newIon.dose[q] = 0.;
            newIon.fluence[q] = 0;
        }
        
        if (energyDeposit > 0.) newIon.dose[Index(i, j, k)] += energyDeposit;
        if (fluence) newIon.fluence[Index(i, j, k)]++;
        
        ionStore.push_back(newIon);
        return true;
    }
    
    else // XXX Out of memory! XXX
    
    {
        return false;
    }
    
}

/////////////////////////////////////////////////////////////////////////////
/////////////////////////////////////////////////////////////////////////////
// Methods to store data to filenames...
////////////////////////////////////////////////////////////////////////////
////////////////////////////////////////////////////////////////////////////
//
// General method to store matrix data to filename
void HadrontherapyMatrix::StoreMatrix(G4String file, void* data, size_t psize)
{
    if (data)
    {
        ofs.open(file, std::ios::out);
        if (ofs.is_open())
        {
            for(G4int i = 0; i < numberOfVoxelAlongX; i++)
                for(G4int j = 0; j < numberOfVoxelAlongY; j++)
                    for(G4int k = 0; k < numberOfVoxelAlongZ; k++)
                    {
                        G4int n = Index(i, j, k);
                        
                        if (psize == sizeof(unsigned int))
                        {
                            unsigned int* pdata = (unsigned int*)data;
                            
                            if (pdata[n])
                                
                                ofs << i << '\t' << j << '\t' << k << '\t' << pdata[n] << G4endl;
                            
                        }
                       
                        else if (psize == sizeof(G4double))
                        
                        {
                            G4double* pdata = (G4double*)data;
                            if (pdata[n]) ofs << i << '\t' << j << '\t' << k << '\t' << pdata[n] << G4endl;
                        }
                    }
            
            ofs.close();
        }
    }
}

/////////////////////////////////////////////////////////////////////////////
// Store fluence per single ion in multiple files
void HadrontherapyMatrix::StoreFluenceData()
{
    for (size_t i=0; i < ionStore.size(); i++){
        StoreMatrix(ionStore[i].name + "_Fluence.out", ionStore[i].fluence, sizeof(unsigned int));
    }
}

/////////////////////////////////////////////////////////////////////////////
// Store dose per single ion in multiple files
void HadrontherapyMatrix::StoreDoseData()
{
    
    for (size_t i=0; i < ionStore.size(); i++){
        StoreMatrix(ionStore[i].name + "_Dose.out", ionStore[i].dose, sizeof(G4double));
    }
}


////////////////////////////////////////////////////////////////////////
// Store dose into a single file
// or in histograms. Please note that this function is called via
// messenger commands
// defined in the HadrontherapyAnalysisFileMessenger.cc class file
void HadrontherapyMatrix::StoreDoseFluenceAscii(G4String file)
{
#define width 15L
    filename = (file=="") ? stdFile:file;
    
    // Sort like periodic table
    
    std::sort(ionStore.begin(), ionStore.end());
    G4cout << "Dose is being written to " << filename << G4endl;
    ofs.open(filename, std::ios::out);
    
    
    if (ofs.is_open())
    {
        // Write the voxels index and the list of particles/ions
        
        ofs << std::setprecision(6) << std::left <<
        "i\tj\tk\t";
        
        
        // Total dose
        ofs << std::setw(width) << "Dose(Gy)";
        
        
        if (secondary)
        {
            for (size_t l=0; l < ionStore.size(); l++)
            {
                G4String a = (ionStore[l].isPrimary) ? "_1":""; // is it a primary?
                ofs << std::setw(width) << ionStore[l].name + a <<
                std::setw(width) << ionStore[l].name  + a;
                
                
            }
            ofs << G4endl;
        }
        
        // Write data
        for(G4int i = 0; i < numberOfVoxelAlongX; i++)
            for(G4int j = 0; j < numberOfVoxelAlongY; j++)
                for(G4int k = 0; k < numberOfVoxelAlongZ; k++)
                {
                    G4int n = Index(i, j, k);
                    
                    if (matrix[n])
                    {
                        ofs << G4endl;
                        ofs << i << '\t' << j << '\t' << k << '\t';
                        
                        // Total dose
                        ofs << std::setw(width) << (matrix[n]/massOfVoxel)/doseUnit;
                       
                        
                        if (secondary)
                        {
                            for (size_t l=0; l < ionStore.size(); l++)
                            {
                                // Fill ASCII file rows
                                ofs << std::setw(width) << ionStore[l].dose[n]/massOfVoxel/doseUnit <<
                                std::setw(width) << ionStore[l].fluence[n];
                                
                            }
                        }
                    }
                }
        ofs.close();
    }
    
    

}
//////////////////////////////////////////////////////////////////////////////
void HadrontherapyMatrix::Fill(G4int i, G4int j, G4int k,
                               G4double energyDeposit)
{
    if (matrix)
        matrix[Index(i,j,k)] += energyDeposit;
    
    // Store the energy deposit in the matrix element corresponding 
    // to the phantom voxel  
}



```

[Go Top](#GoTop)<a id='HadrontherapyLet.hh'></a>
## include/HadrontherapyLet.hh
```cpp
#ifndef HadrontherapyLet_h
#define HadrontherapyLet_h 1
#endif

#include "G4ParticleDefinition.hh"
#include "globals.hh"
#include <fstream>
#include <vector>
#include <string>

#include "g4csv.hh"
#include "HadrontherapyMatrix.hh"
struct ionLet
{
    G4bool isPrimary;        // True if particle is primary
    G4int PDGencoding;      // Particle data group id for the particle
    G4String fullName;      // AZ[excitation energy]: like He3[1277.4], He4[0.0], Li7[231.4], ...
    G4String name;          // simple name without excitation energy: He3, He4, Li7, ...
    G4int Z;                // atomic number
    G4int A;            // mass number
    G4double *letDN , *letDD, *letTN , *letTD; // Track averaged LET and Dose averaged LET
    //friend G4bool operator<(const ionLet& a, const ionLet& b) {return (a.Z == b.Z) ? b.A < a.A : b.Z < a.Z ;}
    G4bool operator<(const ionLet& a) const{return (this->Z == a.Z) ? this-> A < a.A : this->Z < a.Z ;}
    // For isotopes sort by the mass number, else sort by the atomic one.
};

class G4Material;
class HadrontherapyMatrix;
class HadrontherapyPrimaryGeneratorAction;
class HadrontherapyInteractionParameters;
class HadrontherapyDetectorConstruction;

class HadrontherapyLet
{
private:
    HadrontherapyLet(HadrontherapyDetectorConstruction*);
    
public:
    ~HadrontherapyLet();
    static HadrontherapyLet* GetInstance(HadrontherapyDetectorConstruction*);
    static HadrontherapyLet* GetInstance();
    static G4bool doCalculation;
    void Initialize();
    void Clear();
    
    void Fill(G4int i, G4int j, G4int k, G4double DE, G4double DX);
    void FillEnergySpectrum (G4int trackID,
                             G4ParticleDefinition* particleDef,
                             G4double ekinMean,
                             G4Material* mat,
                             G4double DE,
                             G4double DEEletrons,
                             G4double DX,
                             G4int i, G4int j, G4int k);
    void LetOutput();
    void StoreLetAscii();
    
    
private:
    static HadrontherapyLet *instance;
    HadrontherapyPrimaryGeneratorAction* pPGA;
    // Detector material
    G4Material* detectorMat;
    G4double density;
    G4String filename;
    
    std::ofstream ofs;
    std::ofstream stopFile;
    HadrontherapyMatrix *matrix;
    G4int nVoxels, numberOfVoxelAlongX, numberOfVoxelAlongY, numberOfVoxelAlongZ ;
    G4double primaryEnergy, energyLimit, binWidth;
    G4int nBins;
    G4double  nT, dT, nD, dD;
    G4double  nSecondaryT, nSecondaryD, dSecondaryT, dSecondaryD;
    G4double  nPrimaryT, nPrimaryD, dPrimaryT, dPrimaryD ;
    G4double *secondaryLetT,  *secondaryLetD, *totalLetT, *DtotalLetT , *DtotalLetD, *totalLetD;
    G4String nome_file;
    
    std::vector<ionLet> ionLetStore;
};
```

[Go Top](#GoTop)<a id='HadrontherapyLet.cc'></a>
## src/HadrontherapyLet.cc
```cpp
#include "HadrontherapyDetectorConstruction.hh"
#include "HadrontherapyLet.hh"

#include "HadrontherapyMatrix.hh"
#include "HadrontherapyInteractionParameters.hh"
#include "HadrontherapyPrimaryGeneratorAction.hh"
#include "HadrontherapyMatrix.hh"
#include "G4RunManager.hh"
#include "G4SystemOfUnits.hh"
#include <cmath>

HadrontherapyLet* HadrontherapyLet::instance = NULL;
G4bool HadrontherapyLet::doCalculation = false;

HadrontherapyLet* HadrontherapyLet::GetInstance(HadrontherapyDetectorConstruction *pDet)
{
    if (instance) delete instance;
    instance = new HadrontherapyLet(pDet);
    return instance;
}

HadrontherapyLet* HadrontherapyLet::GetInstance()
{
    return instance;
}

HadrontherapyLet::HadrontherapyLet(HadrontherapyDetectorConstruction* pDet)
:filename("Let.out"),matrix(0) // Default output filename
{
    
    matrix = HadrontherapyMatrix::GetInstance();
    
    if (!matrix)
        G4Exception("HadrontherapyLet::HadrontherapyLet",
                    "Hadrontherapy0005", FatalException,
                    "HadrontherapyMatrix not found. Firstly create an instance of it.");
    
    nVoxels = matrix -> GetNvoxel();
    
    numberOfVoxelAlongX = matrix -> GetNumberOfVoxelAlongX();
    numberOfVoxelAlongY = matrix -> GetNumberOfVoxelAlongY();
    numberOfVoxelAlongZ = matrix -> GetNumberOfVoxelAlongZ();
    
    G4RunManager *runManager = G4RunManager::GetRunManager();
    pPGA = (HadrontherapyPrimaryGeneratorAction*)runManager -> GetUserPrimaryGeneratorAction();
    // Pointer to the detector material
    detectorMat = pDet -> GetDetectorLogicalVolume() -> GetMaterial();
    density = detectorMat -> GetDensity();
    // Instances for Total LET
    totalLetD =      new G4double[nVoxels];
    DtotalLetD =     new G4double[nVoxels];
    totalLetT =      new G4double[nVoxels];
    DtotalLetT =     new G4double[nVoxels];
    
}

HadrontherapyLet::~HadrontherapyLet()
{
    Clear();
    delete [] totalLetD;
    delete [] DtotalLetD;
    delete [] totalLetT;
    delete [] DtotalLetT;
}

// Fill energy spectrum for every voxel (local energy spectrum)
void HadrontherapyLet::Initialize()
{
    for(G4int v=0; v < nVoxels; v++) totalLetD[v] = DtotalLetD[v] = totalLetT[v] = DtotalLetT[v] = 0.;
    Clear();
}
/**
 * Clear all stored data
 */
void HadrontherapyLet::Clear()
{
    for (size_t i=0; i < ionLetStore.size(); i++)
    {
        delete [] ionLetStore[i].letDN; // Let Dose Numerator
        delete [] ionLetStore[i].letDD; // Let Dose Denominator
        delete [] ionLetStore[i].letTN; // Let Track Numerator
        delete [] ionLetStore[i].letTD; // Let Track Denominator
    }
    ionLetStore.clear();
}
void  HadrontherapyLet::FillEnergySpectrum(G4int trackID,
                                           G4ParticleDefinition* particleDef,
                                           G4double ekinMean,
                                           G4Material* mat,
                                           G4double DE,
                                           G4double DEEletrons,
                                           G4double DX,
                                           G4int i, G4int j, G4int k)
{
    if (DE <= 0. || DX <=0. ) return; // calculate only energy deposit
    if (!doCalculation) return;
    
    // atomic number
    G4int Z = particleDef -> GetAtomicNumber();
    if (Z<1) return; // calculate only protons and ions
    
    G4int PDGencoding = particleDef -> GetPDGEncoding();
    PDGencoding -= PDGencoding%10; // simple Particle data group id  without excitation level
    
    G4int voxel = matrix -> Index(i,j,k);
    
    // ICRU stopping power calculation
    G4EmCalculator emCal;
    // use the mean kinetic energy of ions in a step to calculate ICRU stopping power
    G4double Lsn = emCal.ComputeElectronicDEDX(ekinMean, particleDef, mat);
    
    
    // Total LET calculation...
    totalLetD[voxel]  += (DE + DEEletrons) * Lsn; // total dose Let Numerator, including secondary electrons energy deposit
    DtotalLetD[voxel] += DE + DEEletrons;         // total dose Let Denominator, including secondary electrons energy deposit
    totalLetT[voxel]  += DX * Lsn;                // total track Let Numerator
    DtotalLetT[voxel] += DX;                      // total track Let Denominator
    
    // store primary ions and secondary ions
    size_t l;
    for (l=0; l < ionLetStore.size(); l++)
    {
        // judge species of the current particle and store it
        if (ionLetStore[l].PDGencoding == PDGencoding)
            if ( ((trackID ==1) && (ionLetStore[l].isPrimary)) || ((trackID !=1) && (!ionLetStore[l].isPrimary)))
                break;
    }
    
    if (l == ionLetStore.size()) // Just another type of ion/particle for our store...
    {
        // mass number
        G4int A = particleDef -> GetAtomicMass();
        
        // particle name
        G4String fullName = particleDef -> GetParticleName();
        G4String name = fullName.substr (0, fullName.find("[") ); // cut excitation energy [x.y]
        
        ionLet ion =
        {
            (trackID == 1) ? true:false, // is it the primary particle?
            PDGencoding,
            fullName,
            name,
            Z,
            A,
            new G4double[nVoxels], // Let Dose Numerator
            new G4double[nVoxels],  // Let Dose Denominator
            new G4double[nVoxels], // Let Track Numerator
            new G4double[nVoxels],  // Let Track Denominator
        };
        
        // Initialize let and other parameters
        for(G4int v=0; v < nVoxels; v++)
        {
            ion.letDN[v] = ion.letDD[v] = ion.letTN[v] = ion.letTD[v] = 0.;
        }
        
        
        ionLetStore.push_back(ion);
    }
    
    // calculate ions let and store them
    ionLetStore[l].letDN[voxel] += (DE + DEEletrons)* Lsn; // ions dose Let Numerator, including secondary electrons energy deposit
    ionLetStore[l].letDD[voxel] += DE + DEEletrons;        // ions dose Let Denominator, including secondary electrons energy deposit
    ionLetStore[l].letTN[voxel] += DX* Lsn;                // ions track Let Numerator
    ionLetStore[l].letTD[voxel] += DX;                     // ions track Let Denominator
    
}




void HadrontherapyLet::LetOutput()
{
    for(G4int v=0; v < nVoxels; v++)
    {
        // compute total let
        if (DtotalLetD[v]>0.) totalLetD[v] = totalLetD[v]/DtotalLetD[v];
        if (DtotalLetT[v]>0.) totalLetT[v] = totalLetT[v]/DtotalLetT[v];
    }
    
    // Sort ions by A and then by Z ...
    std::sort(ionLetStore.begin(), ionLetStore.end());
    
    
    // Compute Let Track and Let Dose for ions
    
    for(G4int v=0; v < nVoxels; v++)
    {
        
        for (size_t ion=0; ion < ionLetStore.size(); ion++)
        {
            // compute ions let
            if (ionLetStore[ion].letDD[v] >0.) ionLetStore[ion].letDN[v] = ionLetStore[ion].letDN[v] / ionLetStore[ion].letDD[v];
            if (ionLetStore[ion].letTD[v] >0.) ionLetStore[ion].letTN[v] = ionLetStore[ion].letTN[v] / ionLetStore[ion].letTD[v];
        } // end loop over ions
    }
} // end loop over voxels



void HadrontherapyLet::StoreLetAscii()
{
#define width 15L
    
    if(ionLetStore.size())
    {
        ofs.open(filename, std::ios::out);
        if (ofs.is_open())
        {
            
            // Write the voxels index and total Lets and the list of particles/ions
            ofs << std::setprecision(6) << std::left <<
            "i\tj\tk\t";
            ofs <<  std::setw(width) << "LDT";
            ofs <<  std::setw(width) << "LTT";
            
            for (size_t l=0; l < ionLetStore.size(); l++) // write ions name
            {
                G4String a = (ionLetStore[l].isPrimary) ? "_1_D":"_D";
                ofs << std::setw(width) << ionLetStore[l].name  + a ;
                G4String b = (ionLetStore[l].isPrimary) ? "_1_T":"_T";
                ofs << std::setw(width) << ionLetStore[l].name  + b ;
            }
            ofs << G4endl;
            
            // Write data
            
            G4AnalysisManager*  LetFragmentTuple = G4AnalysisManager::Instance();
            
            LetFragmentTuple->SetVerboseLevel(1);
            LetFragmentTuple->SetFirstHistoId(1);
            LetFragmentTuple->SetFirstNtupleId(1);
            LetFragmentTuple ->OpenFile("Let");
            
            
            LetFragmentTuple ->CreateNtuple("coordinate", "Let");
            
            
            LetFragmentTuple ->CreateNtupleIColumn("i");//0
            LetFragmentTuple ->CreateNtupleIColumn("j");//1
            LetFragmentTuple ->CreateNtupleIColumn("k");//2
            LetFragmentTuple ->CreateNtupleDColumn("TotalLetD");//3
            LetFragmentTuple ->CreateNtupleDColumn("TotalLetT");//4
            LetFragmentTuple ->CreateNtupleIColumn("A");//5
            LetFragmentTuple ->CreateNtupleIColumn("Z");//6
            LetFragmentTuple ->CreateNtupleDColumn("IonLETD");//7
            LetFragmentTuple ->CreateNtupleDColumn("IonLETT");//8
            LetFragmentTuple ->FinishNtuple();
            
            
            for(G4int i = 0; i < numberOfVoxelAlongX; i++)
                for(G4int j = 0; j < numberOfVoxelAlongY; j++)
                    for(G4int k = 0; k < numberOfVoxelAlongZ; k++)
                    {
                        LetFragmentTuple->FillNtupleIColumn(1,0, i);
                        LetFragmentTuple->FillNtupleIColumn(1,1, j);
                        LetFragmentTuple->FillNtupleIColumn(1,2, k);
                        
                        G4int v = matrix -> Index(i, j, k);
                        
                        // write total Lets and voxels index
                        ofs << G4endl;
                        ofs << i << '\t' << j << '\t' << k << '\t';
                        ofs << std::setw(width) << totalLetD[v]/(keV/um);
                        ofs << std::setw(width) << totalLetT[v]/(keV/um);
                        
                        
                        // write ions Lets
                        for (size_t l=0; l < ionLetStore.size(); l++)
                        {
                            
                            // Write only not identically null data line
                            if(ionLetStore[l].letDN)
                            {
                                // write ions Lets
                                ofs << std::setw(width) << ionLetStore[l].letDN[v]/(keV/um) ;
                                ofs << std::setw(width) << ionLetStore[l].letTN[v]/(keV/um);
                            }
                        }
                        
                        LetFragmentTuple->FillNtupleDColumn(1,3, totalLetD[v]/(keV/um));
                        LetFragmentTuple->FillNtupleDColumn(1,4, totalLetT[v]/(keV/um));
                        
                        
                        for (size_t ll=0; ll < ionLetStore.size(); ll++)
                        {
                            
                            
                            LetFragmentTuple->FillNtupleIColumn(1,5, ionLetStore[ll].A);
                            LetFragmentTuple->FillNtupleIColumn(1,6, ionLetStore[ll].Z);
                            
                            
                            LetFragmentTuple->FillNtupleDColumn(1,7, ionLetStore[ll].letDN[v]/(keV/um));
                            LetFragmentTuple->FillNtupleDColumn(1,8, ionLetStore[ll].letTN[v]/(keV/um));
                            LetFragmentTuple->AddNtupleRow(1);
                        }
                    }
            ofs.close();
            
            LetFragmentTuple->Write();
            LetFragmentTuple->CloseFile();
        }
        
    }
    
}

```

[Go Top](#GoTop)<a id='HadrontherapyRBE.hh'></a>, [Go to 'About RBE Calculation'](#RBECalculations)
## include/HadrontherapyRBE.hh
```cpp
#ifndef HadrontherapyRBE_H
#define HadrontherapyRBE_H 1

#include "globals.hh"
#include <vector>
#include <valarray>
#include <map>
#include "G4Pow.hh"

class G4GenericMessenger;

/**
 * @brief Main class of the RBE calculation.
 *
 * The calculation has to be explicitly enabled
 * (use macro command)
 *
 * Available macro commands:
 *
 *  - /rbe/calculation 0/1 : enable or disable RBE calculation
 *  - /rbe/verbose 0/1/2 : level of screen output detail
 *  - /rbe/loadLemTable [path] : read a CSV file with alphas, betas, ...
 *  - /rbe/cellLine [name] : select one of the cell lines from the data file
 *  - /rbe/doseScale [number] : factor to make the survival/RBE calculation correct
 *  - /rbe/accumulate 0/1 : enable or disable data summing over multiple runs
 *  - /rbe/reset : clear accumulated data back to 0.
 */
class HadrontherapyRBE
{
public:
    virtual ~HadrontherapyRBE();

    // Make a new & get instance pointer
    static HadrontherapyRBE* CreateInstance(G4int nX, G4int nY, G4int nZ, G4double massOfVoxel);
    static HadrontherapyRBE* GetInstance();

    // If this is false (set with macro), nothing happens
    G4bool IsCalculationEnabled() const { return fCalculationEnabled; }

    // If this is true, dose (and alpha/beta parameters) are accumulated over multiple runs
    G4bool IsAccumulationEnabled() const { return fAccumulate; }

    // Initialization of data from a CSV file
    void LoadLEMTable(G4String path);

    // Select the cell and update the pointer
    void SetCellLine(G4String name);

    // Calculate alpha and beta for single deposition, {0,0} if not applicable
    std::tuple<G4double, G4double> GetHitAlphaAndBeta(G4double E, G4int Z);

    // Parameter setting
    void SetDoseScale(G4double scale);
    void SetCalculationEnabled(G4bool enabled);
    void SetAccumulationEnabled(G4bool accumulate);

    // Verbosity for output
    void SetVerboseLevel(G4int level) { fVerboseLevel = level; }
    G4int GetVerboseLevel() const { return fVerboseLevel; }
    
    // Alias for matrix type
    using array_type = std::valarray<G4double>;

    // Calculation
    void ComputeAlphaAndBeta();
    void ComputeRBE();

    // Update the class with accumulated data
    // (To be used from HadrontherapyRBEAccumulable)
    void SetAlphaNumerator(const array_type alpha);
    void SetBetaNumerator(const array_type beta);
    void SetEnergyDeposit(const array_type eDep);
    void SetDenominator(const array_type denom);
    
    // Accumulation variants necessary for multi-run sumation
    void AddAlphaNumerator(const array_type alpha);
    void AddBetaNumerator(const array_type beta);
    void AddEnergyDeposit(const array_type eDep);
    void AddDenominator(const array_type denom);
    
    // Clear accumulated data
    void Reset();

    // Output to text files (called at the end of run)
    void StoreAlphaAndBeta();
    void StoreRBE();

    // Information about voxels
    size_t GetNumberOfVoxelsAlongX() const { return fNumberOfVoxelsAlongX; }
    size_t GetNumberOfVoxelsAlongY() const { return fNumberOfVoxelsAlongY; }
    size_t GetNumberOfVoxelsAlongZ() const { return fNumberOfVoxelsAlongZ; }

    // Some basic output to the screen
    void PrintParameters();

protected:
    inline G4int Index(G4int i, G4int j, G4int k) {return (i * fNumberOfVoxelsAlongY + j) * fNumberOfVoxelsAlongZ + k;}

    // Interpolation
    // G4int GetRowVecEnergy();
    // G4bool NearLookup(G4double E, G4double DE);
    // G4bool LinearLookup(G4double E, G4double DE, G4int Z);
    // void interpolation_onE(G4int k,G4int m, G4int indexE, G4double E, G4int Z);
    // G4bool interpolation_onLET1_onLET2_onE(G4int k,G4int m, G4int indexE, G4double E, G4double LET);
    // void InitDynamicVec(std::vector<G4double> &vecEnergy, G4int matrix_energy);

    // Messenger initialization
    void CreateMessenger();

private:
    HadrontherapyRBE(G4int numberOfVoxelX, G4int numberOfVoxelY, G4int numberOfVoxelZ, G4double massOfVoxel);

    G4GenericMessenger* fMessenger;
    G4Pow* g4pow = G4Pow::GetInstance();

    static HadrontherapyRBE* instance;
    G4int fVerboseLevel { 1 };

    // Parameters for calculation
    G4double fAlphaX { 0.0 };
    G4double fBetaX { 0.0 };
    G4double fDoseCut { 0.0 };
    G4double fDoseScale { 1.0 };

    // Output paths (TODO: Change to analysis tools)
    G4String fAlphaBetaPath { "AlphaAndBeta.out" };
    G4String fRBEPath { "RBE.out" };

    // Voxelization
    G4int fNumberOfVoxelsAlongX, fNumberOfVoxelsAlongY, fNumberOfVoxelsAlongZ;
    G4int fNumberOfVoxels;
    G4double fMassOfVoxel;

    G4double* x; // TODO: Currently not used (that much)

    G4bool fCalculationEnabled { false };
    G4bool fAccumulate { false };

    // Matrices to be set when accumulated
    array_type fAlpha;
    array_type fBeta;
    array_type fDose; // Note: this is calculated from energyDeposit, massOfVoxel and doseScale
    
    array_type fAlphaNumerator;
    array_type fBetaNumerator;
    array_type fDenominator;

    // Matrices of calculated values
    array_type fLnS;
    array_type fSurvival;
    array_type fDoseX;
    array_type fRBE;

    // Available tables and associated values.
    using vector_type = std::map<G4int, std::vector<G4double>>;
    std::map<G4String, vector_type> fTablesEnergy;
    // std::map<G4String, vector_type> fTablesLet;
    std::map<G4String, vector_type> fTablesAlpha;
    std::map<G4String, vector_type> fTablesBeta;
    std::map<G4String, G4double> fTablesAlphaX;
    std::map<G4String, G4double> fTablesBetaX;
    std::map<G4String, G4double> fTablesDoseCut;

    // Selected tables and associated values.
    // (changed when the cell line is set)
    G4String fActiveCellLine = "";
    vector_type* fActiveTableEnergy { nullptr };
    // vector_type* fActiveTableLet { nullptr };
    vector_type* fActiveTableAlpha { nullptr };
    vector_type* fActiveTableBeta { nullptr };
    std::map<G4int, G4double> fMaxEnergies;
    std::map<G4int, G4double> fMinEnergies;
    G4int fMinZ;
    G4int fMaxZ;
};
#endif
```

[Go Top](#GoTop)<a id='HadrontherapyRBE.cc'></a>, [Go to 'About RBE Calculation'](#RBECalculations)
## src/HadrontherapyRBE.cc
```cpp
#include "HadrontherapyRBE.hh"
#include "HadrontherapyMatrix.hh"

#include "G4UnitsTable.hh"
#include "G4SystemOfUnits.hh"
#include "G4GenericMessenger.hh"
#include "G4Pow.hh"

#include <fstream>
#include <iostream>
#include <iomanip>
#include <cstdlib>
#include <algorithm>
#include <sstream>
#include <numeric>

#define width 15L

using namespace std;

// TODO: Perhaps we can use the material database???
const std::map<G4String, G4int> elements = {
    {"H", 1},
    {"He", 2},
    {"Li", 3},
    {"Be", 4},
    {"B", 5},
    {"C", 6},
    {"N", 7},
    {"O", 8},
    {"F", 9},
    {"Ne", 10}
};

HadrontherapyRBE* HadrontherapyRBE::instance = nullptr;

HadrontherapyRBE* HadrontherapyRBE::GetInstance()
{
    return instance;
}

HadrontherapyRBE* HadrontherapyRBE::CreateInstance(G4int voxelX, G4int voxelY, G4int voxelZ, G4double massOfVoxel)
{
    if (instance) delete instance;
    instance = new HadrontherapyRBE(voxelX, voxelY, voxelZ, massOfVoxel);
    return instance;
}
//kp: 8/7/19: Uses member initializer list (): https://www.learncpp.com/cpp-tutorial/8-5a-constructor-member-initializer-lists/
HadrontherapyRBE::HadrontherapyRBE(G4int voxelX, G4int voxelY, G4int voxelZ, G4double massOfVoxel)
    : fNumberOfVoxelsAlongX(voxelX),
      fNumberOfVoxelsAlongY(voxelY),
      fNumberOfVoxelsAlongZ(voxelZ),
      fNumberOfVoxels(voxelX * voxelY * voxelY),
      fMassOfVoxel(massOfVoxel)

{
    CreateMessenger();
    
    // x axis for 1-D plot
    // TODO: Remove
    x = new G4double[fNumberOfVoxelsAlongX];

    for (G4int i = 0; i < fNumberOfVoxelsAlongX; i++)
    {
        x[i] = 0;
    }

}

HadrontherapyRBE::~HadrontherapyRBE()
{
    delete[] x;
    delete fMessenger;
}

void HadrontherapyRBE::PrintParameters()
{
    G4cout << "RBE Cell line: " << fActiveCellLine << G4endl;
    G4cout << "RBE Dose threshold value: " << fDoseCut / gray << " gray" << G4endl;
    G4cout << "RBE Alpha X value: " << fAlphaX * gray << " 1/gray" << G4endl;
    G4cout << "RBE Beta X value: " << fBetaX * pow(gray, 2.0) << " 1/gray2" << G4endl;
}

/**
  * @short Split string into parts using a delimiter.
  *
  * @param line a string to be splitted
  * @param delimiter a string to be looked for
  *
  * Usage: Help function for reading CSV
  * Also remove spaces and quotes around.
  * Note: If delimiter is inside a string, the function fails!
  */
vector<G4String> split(const G4String& line, const G4String& delimiter)
{
    vector<G4String> result;

    size_t current = 0;
    size_t pos = 0;

    while(pos != G4String::npos)
    {
        pos = line.find(delimiter, current);
        G4String token = line.substr(current, pos - current);
        token = token.strip(G4String::both);
        token = token.strip(G4String::both, '\"');
        result.push_back(token);
        current = pos + delimiter.size();
    }
    return result;
}

void HadrontherapyRBE::LoadLEMTable(G4String path)
{
    // TODO: Check for presence? Perhaps we want to be able to load more

    ifstream in(path);
    if (!in)
    {
        stringstream ss;
        ss << "Cannot open LEM table input file: " << path;
        G4Exception("HadrontherapyRBE::LoadData", "WrongTable", FatalException, ss.str().c_str());
    }

    // Start with the first line
    G4String line;
    if (!getline(in, line))
    {
        stringstream ss;
        ss << "Cannot read header from the LEM table file: " << path;
        G4Exception("HadrontherapyRBE::LoadLEMTable", "CannotReadHeader", FatalException, ss.str().c_str());
    }
    vector<G4String> header = split(line, ",");

    // Find the order of columns
    std::vector<G4String> columns = { "alpha_x", "beta_x", "D_t", "specific_energy", "alpha", "beta", "cell", "particle"};
    std::map<G4String, int> columnIndices;
    for (auto columnName : columns)
    {
        auto pos = find(header.begin(), header.end(), columnName);
        if (pos == header.end())
        {
            stringstream ss;
            ss << "Column " << columnName << " not present in the LEM table file.";
            G4Exception("HadrontherapyRBE::LoadLEMTable", "ColumnNotPresent", FatalException, ss.str().c_str());
        }
        else
        {
            columnIndices[columnName] = distance(header.begin(), pos);
        }
    }

    // Continue line by line
    while (getline(in, line))
    {
        vector<G4String> lineParts = split(line, ",");
        G4String cellLine = lineParts[columnIndices["cell"]];
        G4int element = elements.at(lineParts[columnIndices["particle"]]);

        fTablesEnergy[cellLine][element].push_back(
            stod(lineParts[columnIndices["specific_energy"]]) * MeV
        );
        fTablesAlpha[cellLine][element].push_back(
            stod(lineParts[columnIndices["alpha"]])
        );
        /* fTablesLet[cellLine][element].push_back(
            stod(lineParts[columnIndices["let"]])
        ); */
        fTablesBeta[cellLine][element].push_back(
            stod(lineParts[columnIndices["beta"]])
        );

        fTablesAlphaX[cellLine] = stod(lineParts[columnIndices["alpha_x"]]) / gray;
        fTablesBetaX[cellLine] = stod(lineParts[columnIndices["beta_x"]]) / (gray * gray);
        fTablesDoseCut[cellLine] = stod(lineParts[columnIndices["D_t"]]) * gray;
    }

    // Sort the tables by energy
    // (https://stackoverflow.com/a/12399290/2692780)
    for (auto aPair : fTablesEnergy)
    {
        for (auto ePair : aPair.second)
        {
            vector<G4double>& tableEnergy = fTablesEnergy[aPair.first][ePair.first];
            vector<G4double>& tableAlpha = fTablesAlpha[aPair.first][ePair.first];
            vector<G4double>& tableBeta = fTablesBeta[aPair.first][ePair.first];

            vector<size_t> idx(tableEnergy.size());
            iota(idx.begin(), idx.end(), 0);
            sort(idx.begin(), idx.end(),
                [&tableEnergy](size_t i1, size_t i2) {return tableEnergy[i1] < tableEnergy[i2];});

            vector<vector<G4double>*> tables = {
                &tableEnergy, &tableAlpha, &tableBeta
            };
            for (vector<G4double>* table : tables)
            {
                vector<G4double> copy = *table;
                for (size_t i = 0; i < copy.size(); ++i)
                {
                    (*table)[i] = copy[idx[i]];
                }
                // G4cout << (*table)[0];
                // reorder(*table, idx);
                G4cout << (*table)[0] << G4endl;
            }
        }
    }

    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: read LEM data for the following cell lines and elements [number of points]: " << G4endl;
        for (auto aPair : fTablesEnergy)
        {
            G4cout << "- " << aPair.first << ": ";
            for (auto ePair : aPair.second)
            {
                G4cout << ePair.first << "[" << ePair.second.size() << "] ";
            }
            G4cout << G4endl;
        }
    }
}

void HadrontherapyRBE::SetCellLine(G4String name)
{
    if (fTablesEnergy.size() == 0)
    {
        G4Exception("HadrontherapyRBE::SetCellLine", "NoCellLine", FatalException, "Cannot select cell line, probably LEM table not loaded yet?");
    }
    if (fTablesEnergy.find(name) == fTablesEnergy.end())
    {
        stringstream str;
        str << "Cell line " << name << " not found in the LEM table.";
        G4Exception("HadrontherapyRBE::SetCellLine", "", FatalException, str.str().c_str());
    }
    else
    {
        fAlphaX = fTablesAlphaX[name];
        fBetaX = fTablesBetaX[name];
        fDoseCut = fTablesDoseCut[name];

        fActiveTableEnergy = &fTablesEnergy[name];
        fActiveTableAlpha = &fTablesAlpha[name];
        fActiveTableBeta = &fTablesBeta[name];

        fMinZ = 0;
        fMaxZ = 0;
        fMinEnergies.clear();
        fMaxEnergies.clear();

        for (auto energyPair : *fActiveTableEnergy)
        {
            if (!fMinZ || (energyPair.first < fMinZ)) fMinZ = energyPair.first;
            if (energyPair.first > fMaxZ) fMaxZ = energyPair.first;

            fMinEnergies[energyPair.first] = energyPair.second[0];
            fMaxEnergies[energyPair.first] = energyPair.second[energyPair.second.size() - 1];
        }
    }

    fActiveCellLine = name;

    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: cell line " << name << " selected." << G4endl;
    }
}

std::tuple<G4double, G4double> HadrontherapyRBE::GetHitAlphaAndBeta(G4double E, G4int Z)
{
    if (!fActiveTableEnergy)
    {
        G4Exception("HadrontherapyRBE::GetHitAlphaAndBeta", "NoCellLine", FatalException, "No cell line selected. Please, do it using the /rbe/cellLine command.");
    }

    // Check we are in the tables
    if ((Z < fMinZ) || (Z > fMaxZ))
    {
        if (fVerboseLevel > 1)
        {
            stringstream str;
            str << "Alpha & beta calculation supported only for ";
            str << fMinZ << " <= Z <= " << fMaxZ << ", but " << Z << " requested.";
            G4Exception("HadrontherapyRBE::GetHitAlphaAndBeta", "", JustWarning, str.str().c_str());
        }
        return make_tuple<G4double, G4double>( 0.0, 0.0 ); //out of table!
    }
    if ((E < fMinEnergies[Z]) || (E >= fMaxEnergies[Z]))
    {
        if (fVerboseLevel > 2)
        {
            G4cout << "RBE hit: Z=" << Z << ", E=" << E << " => out of LEM table";
            if (fVerboseLevel > 3)
            {
                G4cout << " (" << fMinEnergies[Z] << " to " << fMaxEnergies[Z] << " MeV)";
            }
            G4cout << G4endl;
        }
        return make_tuple<G4double, G4double>( 0.0, 0.0 ); //out of table!
    }

    std::vector<G4double>& vecEnergy = (*fActiveTableEnergy)[Z];
    std::vector<G4double>& vecAlpha = (*fActiveTableAlpha)[Z];
    std::vector<G4double>& vecBeta = (*fActiveTableBeta)[Z];

    // Find the row in energy tables
    const auto eLarger = upper_bound(begin(vecEnergy), end(vecEnergy), E);
    const G4int lower = distance(begin(vecEnergy), eLarger) - 1;
    const G4int upper = lower + 1;

    // Interpolation
    const G4double energyLower = vecEnergy[lower];
    const G4double energyUpper = vecEnergy[upper];
    const G4double energyFraction = (E - energyLower) / (energyUpper - energyLower);

    //linear interpolation along E
    const G4double alpha = ((1 - energyFraction) * vecAlpha[lower] + energyFraction * vecAlpha[upper]);
    const G4double beta = ((1 - energyFraction) * vecBeta[lower] + energyFraction * vecBeta[upper]);
    if (fVerboseLevel > 2)
    {
        G4cout << "RBE hit: Z=" << Z << ", E=" << E << " => alpha=" << alpha << ", beta=" << beta << G4endl;
    }

    return make_tuple(alpha, beta);
}

// Zaider & Rossi alpha & Beta mean
void HadrontherapyRBE::ComputeAlphaAndBeta()
{
    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: Computing alpha and beta..." << G4endl;
    }
    fAlpha = fAlphaNumerator / (fDenominator * gray);
    
    fBeta = pow(fBetaNumerator / fDenominator * gray, 2.0);
    
    //g4pow -> powN(fBetaNumerator / fDenominator * gray, 2)
}

void HadrontherapyRBE::ComputeRBE()
{
    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: Computing survival and RBE..." << G4endl;
    }
    G4double smax = fAlphaX + 2 * fBetaX * fDoseCut;

    // Prepare matrices that were not initialized yet
    fLnS.resize(fNumberOfVoxels);
    fDoseX.resize(fNumberOfVoxels);

    for (G4int i = 0; i < fNumberOfVoxels; i++)
    {
        if (std::isnan(fAlpha[i]) || std::isnan(fBeta[i]))
        {
            fLnS[i] = 0.0;
            fDoseX[i] = 0.0;
        }
        else if (fDose[i] <= fDoseCut)
        {
            fLnS[i] = -(fAlpha[i] * fDose[i]) - (fBeta[i] * (pow(fDose[i], 2.0))) ;
            fDoseX[i] = sqrt((-fLnS[i] / fBetaX) + pow((fAlphaX / (2 * fBetaX)), 2.0)) - (fAlphaX / (2 * fBetaX));
        }
        else
        {
            G4double ln_Scut = -(fAlpha[i] * fDoseCut) - (fBeta[i] * (pow((fDoseCut), 2.0)));
            fLnS[i] = ln_Scut - ((fDose[i] - fDoseCut) * smax);

            // TODO: CHECK THIS!!!
            fDoseX[i] = ( (-fLnS[i] + ln_Scut) / smax ) + fDoseCut;
        }
    }
    fRBE = fDoseX / fDose; //kpa: matrix division operation
    fSurvival = exp(fLnS);
}

void HadrontherapyRBE::SetDenominator(const HadrontherapyRBE::array_type denom)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Setting denominator..."  << G4endl;
    }
    fDenominator = denom;
}

void HadrontherapyRBE::AddDenominator(const HadrontherapyRBE::array_type denom)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Adding denominator...";
    }
    if (fDenominator.size())
    {
        fDenominator += denom;
    }
    else
    {
        if (fVerboseLevel > 1)
        {
            G4cout << " (created empty array)";
        }
        fDenominator = denom;
    }
    G4cout << G4endl;
}

void HadrontherapyRBE::SetAlphaNumerator(const HadrontherapyRBE::array_type alpha)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Setting alpha numerator..."  << G4endl;
    }
    fAlphaNumerator = alpha;
}

void HadrontherapyRBE::SetBetaNumerator(const HadrontherapyRBE::array_type beta)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Setting beta numerator..." << G4endl;
    }
    fBetaNumerator = beta;
}

void HadrontherapyRBE::AddAlphaNumerator(const HadrontherapyRBE::array_type alpha)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Adding alpha numerator...";
    }
    if (fAlphaNumerator.size())
    {
        fAlphaNumerator += alpha;
    }
    else
    {
        if (fVerboseLevel > 1)
        {
            G4cout << " (created empty array)";
        }
        fAlphaNumerator = alpha;
    }
    G4cout << G4endl;
}

void HadrontherapyRBE::AddBetaNumerator(const HadrontherapyRBE::array_type beta)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Adding beta...";
    }
    if (fBetaNumerator.size())
    {
        fBetaNumerator += beta;
    }
    else
    {
        if (fVerboseLevel > 1)
        {
            G4cout << " (created empty array)";
        }
        fBetaNumerator = beta;
    }
    G4cout << G4endl;
}

void HadrontherapyRBE::SetEnergyDeposit(const std::valarray<G4double> eDep)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Setting dose..." << G4endl;
    }
    fDose = eDep * (fDoseScale / fMassOfVoxel);
}

void HadrontherapyRBE::AddEnergyDeposit(const std::valarray<G4double> eDep)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Adding dose... (" << eDep.size() << " points)" << G4endl;
    }
    if (fDose.size())
    {
        fDose += eDep * (fDoseScale / fMassOfVoxel);
    }
    else
    {
        if (fVerboseLevel > 1)
        {
            G4cout << " (created empty array)";
        }
        fDose = eDep * (fDoseScale / fMassOfVoxel);
    }
}

void HadrontherapyRBE::StoreAlphaAndBeta()
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Writing alpha and beta..." << G4endl;
    }
    ofstream ofs(fAlphaBetaPath);

    ComputeAlphaAndBeta();

    if (ofs.is_open())
    {
        ofs << "alpha" << std::setw(width) << "beta " << std::setw(width) << "depth(slice)" << G4endl;
        for (G4int i = 0; i < fNumberOfVoxelsAlongX * fNumberOfVoxelsAlongY * fNumberOfVoxelsAlongZ; i++)
            ofs << fAlpha[i]*gray << std::setw(15L) << fBeta[i]*pow(gray, 2.0) << std::setw(15L) << i << G4endl;
    }
    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: Alpha and beta written to " << fAlphaBetaPath << G4endl;
    }
}

void HadrontherapyRBE::StoreRBE()
{
    ofstream ofs(fRBEPath);

    // TODO: only if not yet calculated. Or in RunAction???
    ComputeRBE();

    if (ofs.is_open())
    {
        ofs << "Dose(Gy)" << std::setw(width) << "ln(S) " << std::setw(width) << "Survival"  << std::setw(width) << "DoseB(Gy)" << std::setw(width) << "RBE" <<  std::setw(width) << "depth(slice)" << G4endl;

        for (G4int i = 0; i < fNumberOfVoxelsAlongX * fNumberOfVoxelsAlongY * fNumberOfVoxelsAlongZ; i++)

            ofs << (fDose[i] / gray) << std::setw(width) << fLnS[i] << std::setw(width) << fSurvival[i]
                << std::setw(width) << fDoseX[i] / gray << std::setw(width) << fRBE[i] << std::setw(width)  << i << G4endl;
    }
    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: RBE written to " << fRBEPath << G4endl;
    }
}

void HadrontherapyRBE::Reset()
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Reset(): ";
    }
    fAlphaNumerator = 0.0;
    fBetaNumerator = 0.0;
    fDenominator = 0.0;
    fDose = 0.0;
    if (fVerboseLevel > 1)
    {
        G4cout << fAlphaNumerator.size() << " points." << G4endl;
    }
}

void HadrontherapyRBE::CreateMessenger()
{
    fMessenger = new G4GenericMessenger(this, "/rbe/");
    fMessenger->SetGuidance("RBE calculation");

    fMessenger->DeclareMethod("calculation", &HadrontherapyRBE::SetCalculationEnabled)
            .SetGuidance("Whether to enable RBE calculation")
            .SetStates(G4State_PreInit, G4State_Idle)
            .SetToBeBroadcasted(false);

    fMessenger->DeclareMethod("verbose", &HadrontherapyRBE::SetVerboseLevel)
            .SetGuidance("Set verbosity level of RBE")
            .SetGuidance("0 = quiet")
            .SetGuidance("1 = important messages (~10 per run)")
            .SetGuidance("2 = debug")
            .SetToBeBroadcasted(false);

    fMessenger->DeclareMethod("loadLemTable", &HadrontherapyRBE::LoadLEMTable)
            .SetGuidance("Load a LEM table used in calculating alpha&beta")
            .SetStates(G4State_PreInit, G4State_Idle)
            .SetToBeBroadcasted(false);

    fMessenger->DeclareMethod("cellLine", &HadrontherapyRBE::SetCellLine)
            .SetGuidance("Set the cell line for alpha&beta calculation")
            .SetStates(G4State_PreInit, G4State_Idle)
            .SetToBeBroadcasted(false);

    fMessenger->DeclareMethod("doseScale", &HadrontherapyRBE::SetDoseScale)
            .SetGuidance("Set the scaling factor to calculate RBE with the real physical dose")
            .SetGuidance("If you don't set this, the RBE will be incorrect")
            .SetStates(G4State_PreInit, G4State_Idle)
            .SetToBeBroadcasted(false);

    fMessenger->DeclareMethod("accumulate", &HadrontherapyRBE::SetAccumulationEnabled)
            .SetGuidance("If false, reset the values at the beginning of each run.")
            .SetGuidance("If true, all runs are summed together")
            .SetStates(G4State_PreInit, G4State_Idle)
            .SetToBeBroadcasted(false);

    fMessenger->DeclareMethod("reset", &HadrontherapyRBE::Reset)
            .SetGuidance("Reset accumulated data (relevant only if accumulate mode is on)")
            .SetStates(G4State_PreInit, G4State_Idle)
            .SetToBeBroadcasted(false);
}

/*
G4bool HadrontherapyRBE::LinearLookup(G4double E, G4double LET, G4int Z)
{
    G4int j;
    G4int indexE;
    if ( E < vecEnergy[0] || E >= vecEnergy[GetRowVecEnergy() - 1]) return false; //out of table!

    // Search E
    for (j = 0; j < GetRowVecEnergy(); j++)
    {
        if (j + 1 == GetRowVecEnergy()) break;
        if (E >= vecEnergy[j] && E < vecEnergy[j + 1]) break;
    }

    indexE = j;

    G4int k = (indexE * column);

    G4int l = ((indexE + 1) * column);

    if (Z <= 8) //linear interpolation along E for calculate alpha and beta
    {
        interpolation_onE(k, l, indexE, E, Z);
    }
    else
    {

        return interpolation_onLET1_onLET2_onE(k, l, indexE, E, LET);

    }
    return true;
}
*/

/*
void HadrontherapyRBE::interpolation_onE(G4int k, G4int l, G4int indexE, G4double E, G4int Z)
{
    // k=(indexE*column) identifies the position of E1 known the value of E (identifies the group of 8 elements in the array at position E1)
    // Z-1 identifies the vector ion position relative to the group of 8 values ​​found

    k = k + (Z - 1);
    l = l + (Z - 1);

    //linear interpolation along E
    alpha = (((vecEnergy[indexE + 1] - E) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * vecAlpha[k]) + ((E - vecEnergy[indexE]) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * vecAlpha[l];

    beta = (((vecEnergy[indexE + 1] - E) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * vecBeta[k]) + ((E - vecEnergy[indexE]) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * vecBeta[l];

}

G4bool HadrontherapyRBE::interpolation_onLET1_onLET2_onE(G4int k, G4int l, G4int indexE, G4double E, G4double LET)
{
    G4double LET1_2, LET3_4, LET1_2_beta, LET3_4_beta;
    G4int indexLET1, indexLET2, indexLET3, indexLET4;
    size_t i;
    if ( (LET >= vecLET[k + column - 1] && LET >= vecLET[l + column - 1]) || (LET < vecLET[k] && LET < vecLET[l]) ) return false; //out of table!

    //Find the value of E1 is detected the value of LET among the 8 possible values ​​corresponding to E1
    for (i = 0; i < column - 1; i++)
    {

        if ( (i + 1 == column - 1) || (LET < vecLET[k]) ) break;

        else if (LET >= vecLET[k] && LET < vecLET[k + 1]) break;
        k++;

    }
    indexLET1 = k;
    indexLET2 = k + 1;

    //Find the value of E2 is detected the value of LET among the 8 possible values ​​corresponding to E2
    for (i = 0; i < column - 1; i++)
    {

        if (i + 1 == column - 1) break;
        if (LET >= vecLET[l] && LET < vecLET[l + 1]) break; // time to interpolate
        l++;

    }

    indexLET3 = l;
    indexLET4 = l + 1;

    //Interpolation between LET1 and LET2 on E2 position
    LET1_2 = (((vecLET[indexLET2] - LET) / (vecLET[indexLET2] - vecLET[indexLET1])) * vecAlpha[indexLET1]) + ((LET - vecLET[indexLET1]) / (vecLET[indexLET2] - vecLET[indexLET1])) * vecAlpha[indexLET2];

    LET1_2_beta = (((vecLET[indexLET2] - LET) / (vecLET[indexLET2] - vecLET[indexLET1])) * vecBeta[indexLET1]) + ((LET - vecLET[indexLET1]) / (vecLET[indexLET2] - vecLET[indexLET1])) * vecBeta[indexLET2];

    //Interpolation between LET3 and LET4 on E2 position
    LET3_4 = (((vecLET[indexLET4] - LET) / (vecLET[indexLET4] - vecLET[indexLET3])) * vecAlpha[indexLET3]) + ((LET - vecLET[indexLET3]) / (vecLET[indexLET4] - vecLET[indexLET3])) * vecAlpha[indexLET4];
    LET3_4_beta = (((vecLET[indexLET4] - LET) / (vecLET[indexLET4] - vecLET[indexLET3])) * vecBeta[indexLET3]) + ((LET - vecLET[indexLET3]) / (vecLET[indexLET4] - vecLET[indexLET3])) * vecBeta[indexLET4];

    //Interpolation along E between LET1_2 and LET3_4
    alpha = (((vecEnergy[indexE + 1] - E) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * LET1_2) + ((E - vecEnergy[indexE]) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * LET3_4;
    beta = (((vecEnergy[indexE + 1] - E) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * LET1_2_beta) + ((E - vecEnergy[indexE]) / (vecEnergy[indexE + 1] - vecEnergy[indexE])) * LET3_4_beta;


    return true;
}
**/

/* void HadrontherapyRBE::SetThresholdValue(G4double dosecut)
{
    fDoseCut = dosecut;
}

void HadrontherapyRBE::SetAlphaX(G4double alphaX)
{
    fAlphaX = alphaX;
}

void HadrontherapyRBE::SetBetaX(G4double betaX)
{
    fBetaX = betaX;
}*/

void HadrontherapyRBE::SetCalculationEnabled(G4bool enabled)
{
    fCalculationEnabled = enabled;
}

void HadrontherapyRBE::SetAccumulationEnabled(G4bool accumulate)
{
    fAccumulate = accumulate;
    // The accumulation should start now, not taking into account old data
    Reset();
}

/*
void HadrontherapyRBE::SetLEMTablePath(G4String path)
{
    // fLEMTablePath = path;
    LoadLEMTable(path);
}
*/

void HadrontherapyRBE::SetDoseScale(G4double scale)
{
    fDoseScale = scale;
}

// Nearest neighbor interpolation
/*
G4bool HadrontherapyRBE::NearLookup(G4double E, G4double DE)
{

    size_t j = 0, step = 77;

    // Check bounds
    if (E < vecE[0] || E > vecE.back() || DE < vecDE[0] || DE > vecDE[step - 1]) return false; //out of table!

    // search for Energy... simple linear search. This take approx 1 us per single search on my sempron 2800+ laptop
    for (; j < vecE.size(); j += step)
    {
        if (E <= vecE[j]) break;
    }
    if (j == vecE.size()) j -= step;
    if (j == vecE.size() && E > vecE[j]) return false; // out of table!


    // find nearest (better interpolate)
    if ( j > 0 && ((vecE[j] - E) > (E - vecE[j - 1])) ) {j = j - step;}
    bestE = vecE[j];


    // search for stopping power... simple linear search
    for (; vecE[j] == bestE && j < vecE.size(); j++)
    {
        if (DE <= vecDE[j]) break;
    }
    if (j == vecE.size() &&  DE > vecDE[j])  return false;// out of table!

    if (j == 0 && (E < vecE[j] || DE < vecDE[j]) ) return false;// out of table!

    if ( (vecDE[j] - DE) > (DE - vecDE[j - 1]) ) {j = j - 1;}
    bestDE = vecDE[j];

    this -> alpha = vecAlpha[j];
    this -> beta  = vecBeta[j];

    return true;
}
*/
```

[Go Top](#GoTop)<a id='HadrontherapyRBEAccumulable.hh'></a>, [Go to 'About RBE Calculation'](#RBECalculations)
## include/HadrontherapyRBEAccumulable.hh
```cpp
#ifndef HADRONTHERAPYRBEACCUMULABLE_HH
#define HADRONTHERAPYRBEACCUMULABLE_HH

#include <G4VAccumulable.hh>

#include <valarray>

/**
 * @brief Accumulable of RBE-related data (that must be thread-local).
 *
 * It keeps the sum of alpha and beta numerators/denominator, as well as energy deposits.
 * The class is closely tied with the singleton HadrontherapyRBE that is used both
 * to calculate alphas and betas, and also to store results.
 *
 * This is implemented as a customized G4VAccumulable with non-scalar data.
 *
 * @note There are two levels of merging (accumulation):
 *   1) From more threads in one run (G4VAccumulable merging is applied)
 *   2) (Optional) inter-run merging of data (implemented in HadrontherapyRBE).
 *
 * @note std::valarray is used (instead of C arrays or std::vectors)
 *    to accumulate data for its logical simplicity.
 */
class HadrontherapyRBEAccumulable : public G4VAccumulable
{
public:
    HadrontherapyRBEAccumulable();
    HadrontherapyRBEAccumulable(const HadrontherapyRBEAccumulable& other) = default;

    // G4VAccumulable virtual methods
    void Merge(const G4VAccumulable &rhs) override;
    void Reset() override;

    // Store information from a single step
    void Accumulate(G4double E, G4double energyDeposit, G4double dX, G4int Z, G4int i, G4int j, G4int k);
```

<a id='alias_usingKeyword'></a>
```cpp
// Type alias for numerical arrays
    using array_type = std::valarray<G4double>;

    // Access to stored data (to be called on the merged data)
    const array_type GetEnergyDeposit() const;
    const array_type GetAlphaNumerator() const { return fAlphaNumerator; }
    const array_type GetBetaNumerator() const { return fBetaNumerator; }
    const array_type GetDenominator() const { return fDenominator; }

    // Verbosity, shared with HadrontherapyRBE
    G4int GetVerboseLevel() const;

private:
    /** @brief Helper function to get the 1D index in the 3D array */
    inline G4int GetIndex(G4int i, G4int j, G4int k) const {return (i * fVoxelsAlongY + j) * fVoxelsAlongZ + k; }

    // Apply configuration from the HadrontherapyRBE class and prepare matrices
    void Initialize();
    G4bool fInitialized { false };

    array_type fAlphaNumerator;
    array_type fBetaNumerator;
    array_type fDenominator;
    array_type fEnergyDeposit;

    // How many voxels do we have?
    // ...along each axis
    size_t fVoxelsAlongX;
    size_t fVoxelsAlongY;
    size_t fVoxelsAlongZ;
    // ...and in total
    size_t fVoxels;
};

#endif // HADRONTHERAPYRBEACCUMULABLE_HH
```

[Go Top](#GoTop)<a id='HadrontherapyRBEAccumulable.cc'></a>, [Go to 'About RBE Calculation'](#RBECalculations)
## src/HadrontherapyRBEAccumulable.cc
This code also uses the only **dynamic_cast** in the entire **hadrontherapy** code.
```
KPAd's FunPrompt $ grep -in dynamic_cast Source/hadrontherapy/src/*
Source/hadrontherapy/src/HadrontherapyRBEAccumulable.cc:47:    const HadrontherapyRBEAccumulable& other = dynamic_cast<const HadrontherapyRBEAccumulable&>(rhs);
KPAd's FunPrompt $ grep -in dynamic_cast Source/hadrontherapy/include/*
KPAd's FunPrompt $
```
====================

```cpp
#include "HadrontherapyRBEAccumulable.hh"
#include "HadrontherapyRBE.hh"

#include <tuple>
#include <G4SystemOfUnits.hh>

using namespace std;

HadrontherapyRBEAccumulable::HadrontherapyRBEAccumulable()
    : G4VAccumulable("RBE")
{

}

void HadrontherapyRBEAccumulable::Merge(const G4VAccumulable& rhs)
{
    if (GetVerboseLevel() > 1)
    {
        G4cout << "HadrontherapyRBEAccumulable::Merge()" << G4endl;
    }
    const HadrontherapyRBEAccumulable& other = dynamic_cast<const HadrontherapyRBEAccumulable&>(rhs);
    fAlphaNumerator += other.fAlphaNumerator;
    fDenominator += other.fDenominator;
    fBetaNumerator += other.fBetaNumerator;
    fEnergyDeposit += other.fEnergyDeposit;
}

void HadrontherapyRBEAccumulable::Reset()
{
    if (GetVerboseLevel() > 1)
    {
        G4cout << "HadrontherapyRBEAccumulable::Reset()" << G4endl;
    }
    if (fInitialized)
    {
        fAlphaNumerator = 0.0;
        fBetaNumerator = 0.0;
        fDenominator = 0.0;
        fEnergyDeposit = 0.0;
    }
    else
    {
        Initialize();
    }
}

void HadrontherapyRBEAccumulable::Accumulate(G4double E, G4double energyDeposit, G4double dX, G4int Z, G4int i, G4int j, G4int k)
{
    if (!fInitialized)
    {
        G4Exception("HadrontherapyRBEAccumulable::Accumulate", "NotInitialized", FatalException, "Accumulable not initialized. Must be a programming error.");
    }
    if (GetVerboseLevel() > 2)
    {
        G4cout << "HadrontherapyRBEAccumulable::Accumulate() in " << i << ", " << j << ", " << k << G4endl;
    }
    if (energyDeposit <= 0)
    {
        return;
    }
    size_t n = GetIndex(i, j, k);
    fEnergyDeposit[n] += energyDeposit;

    if ((Z >= 1) && (dX > 0) && (E > 0)) // TODO: Verify this condition
    {
        tuple<G4double, G4double> alpha_beta = HadrontherapyRBE::GetInstance()->GetHitAlphaAndBeta(E, Z);
        fDenominator[n] += energyDeposit;
        fAlphaNumerator[n] += get<0>(alpha_beta) * energyDeposit;
        fBetaNumerator[n] += sqrt(get<1>(alpha_beta)) * energyDeposit;
    }
}

const HadrontherapyRBEAccumulable::array_type HadrontherapyRBEAccumulable::GetEnergyDeposit() const
{
    return fEnergyDeposit;
}

G4int HadrontherapyRBEAccumulable::GetVerboseLevel() const
{
    return HadrontherapyRBE::GetInstance()->GetVerboseLevel();
}

void HadrontherapyRBEAccumulable::Initialize()
{
    if (GetVerboseLevel() > 1)
    {
        G4cout << "HadrontherapyRBEAccumulable::Initialize(): ";
    }
    auto rbe = HadrontherapyRBE::GetInstance();

    fVoxelsAlongX = rbe->GetNumberOfVoxelsAlongX();
    fVoxelsAlongY = rbe->GetNumberOfVoxelsAlongY();
    fVoxelsAlongZ = rbe->GetNumberOfVoxelsAlongZ();
    fVoxels = fVoxelsAlongX * fVoxelsAlongY * fVoxelsAlongZ;

    if (GetVerboseLevel() > 1)
    {
        G4cout << fVoxels << " voxels." << G4endl;
    }

    fAlphaNumerator = array_type(0.0, fVoxels);
    fBetaNumerator = array_type(0.0, fVoxels);
    fDenominator = array_type(0.0, fVoxels);
    fEnergyDeposit = array_type(0.0, fVoxels);
    fInitialized = true;
}
```

[Go Top](#GoTop)<a id='PassiveProtonBeamLine.hh'></a>
## include/PassiveProtonBeamLine.hh
```cpp
#ifndef PassiveProtonBeamLine_H
#define PassiveProtonBeamLine_H 1

#include "globals.hh"
#include "G4VUserDetectorConstruction.hh"
#include "G4Box.hh"
#include "G4Tubs.hh"
#include "G4VisAttributes.hh"
#include "G4LogicalVolume.hh"

class G4VPhysicalVolume;
class HadrontherapyDetectorConstruction;
class HadrontherapyModulator;
class PassiveProtonBeamLineMessenger;
class HadrontherapyDetectorROGeometry;

class PassiveProtonBeamLine : public G4VUserDetectorConstruction
{
public:

    PassiveProtonBeamLine();
    ~PassiveProtonBeamLine();
  // static G4bool doCalculation;
    
    G4VPhysicalVolume* Construct();
    //***************************** PW **************NON SERVE*************************
    
    static PassiveProtonBeamLine* GetInstance();
    
    //***************************** PW **************NON SERVE*************************
    
    void HadrontherapyBeamLineSupport();
    // Definition of the beam line support
    
    void HadrontherapyBeamScatteringFoils();
    // Definition of the first scattering foil,
    // of the Kapton window, of the stopper
    
    void HadrontherapyRangeShifter();
    // This defines the "range shifter". Is is a slab
    // (usually of PMMA" acting as energy degrader
    // of primary beam
    
    void HadrontherapyBeamCollimators();
    // Definition of the first collimator, of the range shifter,
    // of the second collimator, of the first and second
    // collimator modulators
    
    void HadrontherapyBeamMonitoring();
    // Definition of three monitor chambers
    
    void HadrontherapyMOPIDetector();
    // Construct the MOPI on-line detector
    
    void HadrontherapyBeamNozzle();
    // Definition of the beam noozle
    
    void HadrontherapyBeamFinalCollimator();
    // Definition of the final collimator
    
    // The following methods allow to change parameters
    // of some beam line components
    
    void SetRangeShifterXPosition(G4double value);
    // This method allows to move the Range Shifter along
    // the X axis
    
    void SetRangeShifterXSize(G4double halfSize);
    // This method allows to change the size of the range shifter along
    // the X axis
    
    void SetFirstScatteringFoilXSize(G4double);
    // This method allows to change the size of the first scattering foil
    // along the X axis
    
    void SetSecondScatteringFoilXSize(G4double);
    // This method allows to change the size of the second scattering foil
    // along the X axis
    
    void SetOuterRadiusStopper(G4double);
    // This method allows to change the size of the outer radius of the stopper
    
    void SetInnerRadiusFinalCollimator(G4double);
    // This method allows to change the size of the inner radius of the
    // final collimator
    
    void SetRSMaterial(G4String);
    // This method allows to change the material
    // of the range shifter
    
    void SetModulatorAngle(G4double angle);
    // This method allows moving the modulator through UI commands
    
    
private:
    static PassiveProtonBeamLine* instance;
    //passive proton line dimensions
    void SetDefaultDimensions();
    void ConstructPassiveProtonBeamLine();
    
    HadrontherapyModulator* modulator; // Pointer to the modulator
    // geometry component
    PassiveProtonBeamLineMessenger* passiveMessenger;
    G4VPhysicalVolume* physicalTreatmentRoom;
    HadrontherapyDetectorConstruction* hadrontherapyDetectorConstruction;
    
    
    G4Material* kapton;
	
    G4double vacuumZoneXSize;
    G4double vacuumZoneYSize;
    G4double vacuumZoneZSize;
    G4double vacuumZoneXPosition;
    
    G4double firstScatteringFoilXSize;
    G4double firstScatteringFoilYSize;
    G4double firstScatteringFoilZSize;
    G4double firstScatteringFoilXPosition;
    
    G4double kaptonWindowXSize;
    G4double kaptonWindowYSize;
    G4double kaptonWindowZSize;
    G4double kaptonWindowXPosition;
    
    G4double innerRadiusStopper;
    G4double heightStopper;
    G4double startAngleStopper;
    G4double spanningAngleStopper;
    G4double stopperXPosition;
    G4double stopperYPosition;
    G4double stopperZPosition;
    G4double outerRadiusStopper;
    
    G4double secondScatteringFoilXSize;
    G4double secondScatteringFoilYSize;
    G4double secondScatteringFoilZSize;
    G4double secondScatteringFoilXPosition;
    G4double secondScatteringFoilYPosition;
    G4double secondScatteringFoilZPosition;
    
    G4double rangeShifterXSize;
    G4double rangeShifterYSize;
    G4double rangeShifterZSize;
    G4double rangeShifterXPosition;
    G4double rangeShifterYPosition;
    G4double rangeShifterZPosition;
    
    
    G4VPhysicalVolume* physiBeamLineSupport;
    G4VPhysicalVolume* physiBeamLineCover;
    G4VPhysicalVolume* physiBeamLineCover2;
    G4Box* firstScatteringFoil;
    G4VPhysicalVolume* physiFirstScatteringFoil;
    G4VPhysicalVolume* physiKaptonWindow;
    
    G4Tubs* solidStopper;
    G4VPhysicalVolume* physiStopper;
    G4LogicalVolume* logicStopper;
    
    G4Box* secondScatteringFoil;
    G4VPhysicalVolume* physiSecondScatteringFoil;
    G4VPhysicalVolume* physiFirstCollimator;
    G4VPhysicalVolume* physiHoleFirstCollimator;
    G4Box* solidRangeShifterBox;
    G4LogicalVolume* logicRangeShifterBox;
    G4VPhysicalVolume* physiRangeShifterBox;
    G4VPhysicalVolume* physiSecondCollimator;
    G4VPhysicalVolume* physiHoleSecondCollimator;
    
    G4VPhysicalVolume* physiFirstCollimatorModulatorBox;
    G4VPhysicalVolume* physiHoleFirstCollimatorModulatorBox;
    
    G4VPhysicalVolume* physiSecondCollimatorModulatorBox;
    G4VPhysicalVolume* physiHoleSecondCollimatorModulatorBox;
    
    // MOPI Detector
    // Mother volume
    G4VPhysicalVolume* physiMOPIMotherVolume;
    G4LogicalVolume* logicMOPIMotherVolume;
    G4Box* solidMOPIMotherVolume;
    
    G4double MOPIMotherVolumeXSize;
    G4double MOPIMotherVolumeYSize;
    G4double MOPIMotherVolumeZSize;
    G4double MOPIMotherVolumeXPosition;
    G4double MOPIMotherVolumeYPosition;
    G4double MOPIMotherVolumeZPosition;
    
    // First Kapton layer
    G4double MOPIFirstKaptonLayerXSize;
    G4double MOPIFirstKaptonLayerYSize;
    G4double MOPIFirstKaptonLayerZSize;
    G4double MOPIFirstKaptonLayerXPosition;
    G4double MOPIFirstKaptonLayerYPosition;
    G4double MOPIFirstKaptonLayerZPosition;
    G4Box* solidMOPIFirstKaptonLayer;
    G4LogicalVolume* logicMOPIFirstKaptonLayer;
    G4VPhysicalVolume* physiMOPIFirstKaptonLayer;
    
    // First Aluminum layer
    G4double MOPIFirstAluminumLayerXSize;
    G4double MOPIFirstAluminumLayerYSize;
    G4double MOPIFirstAluminumLayerZSize;
    G4double MOPIFirstAluminumLayerXPosition;
    G4double MOPIFirstAluminumLayerYPosition;
    G4double MOPIFirstAluminumLayerZPosition;
    G4Box* solidMOPIFirstAluminumLayer;
    G4LogicalVolume* logicMOPIFirstAluminumLayer;
    G4VPhysicalVolume* physiMOPIFirstAluminumLayer;
    
    // First Air Gap
    G4double MOPIFirstAirGapXSize;
    G4double MOPIFirstAirGapYSize;
    G4double MOPIFirstAirGapZSize;
    G4double MOPIFirstAirGapXPosition;
    G4double MOPIFirstAirGapYPosition;
    G4double MOPIFirstAirGapZPosition;
    G4Box* solidMOPIFirstAirGap;
    G4LogicalVolume* logicMOPIFirstAirGap;
    G4VPhysicalVolume* physiMOPIFirstAirGap;
    
    // Cathode
    G4double MOPICathodeXSize;
    G4double MOPICathodeYSize;
    G4double MOPICathodeZSize;
    G4double MOPICathodeXPosition;
    G4double MOPICathodeYPosition;
    G4double MOPICathodeZPosition;
    G4Box* solidMOPICathode;
    G4LogicalVolume* logicMOPICathode;
    G4VPhysicalVolume* physiMOPICathode;
    
    G4VisAttributes* redWire;
    
    // First Air Gap
    G4double MOPISecondAirGapXSize;
    G4double MOPISecondAirGapYSize;
    G4double MOPISecondAirGapZSize;
    G4double MOPISecondAirGapXPosition;
    G4double MOPISecondAirGapYPosition;
    G4double MOPISecondAirGapZPosition;
    G4Box* solidMOPISecondAirGap;
    G4LogicalVolume* logicMOPISecondAirGap;
    G4VPhysicalVolume* physiMOPISecondAirGap;
    
    // First Aluminum layer
    G4double MOPISecondAluminumLayerXSize;
    G4double MOPISecondAluminumLayerYSize;
    G4double MOPISecondAluminumLayerZSize;
    G4double MOPISecondAluminumLayerXPosition;
    G4double MOPISecondAluminumLayerYPosition;
    G4double MOPISecondAluminumLayerZPosition;
    G4Box* solidMOPISecondAluminumLayer;
    G4LogicalVolume* logicMOPISecondAluminumLayer;
    G4VPhysicalVolume* physiMOPISecondAluminumLayer;
    
    // Second Kapton layer
    G4double MOPISecondKaptonLayerXSize;
    G4double MOPISecondKaptonLayerYSize;
    G4double MOPISecondKaptonLayerZSize;
    G4double MOPISecondKaptonLayerXPosition;
    G4double MOPISecondKaptonLayerYPosition;
    G4double MOPISecondKaptonLayerZPosition;
    G4Box* solidMOPISecondKaptonLayer;
    G4LogicalVolume* logicMOPISecondKaptonLayer;
    G4VPhysicalVolume* physiMOPISecondKaptonLayer;
    
    G4double innerRadiusFinalCollimator;
    G4VPhysicalVolume* mother;
    
    G4VPhysicalVolume* physiFirstMonitorLayer1;
    G4VPhysicalVolume* physiFirstMonitorLayer2;
    G4VPhysicalVolume* physiFirstMonitorLayer3;
    G4VPhysicalVolume* physiFirstMonitorLayer4;
    G4VPhysicalVolume* physiSecondMonitorLayer1;
    G4VPhysicalVolume* physiSecondMonitorLayer2;
    G4VPhysicalVolume* physiSecondMonitorLayer3;
    G4VPhysicalVolume* physiSecondMonitorLayer4;
    G4VPhysicalVolume* physiNozzleSupport;
    G4VPhysicalVolume* physiHoleNozzleSupport;
    G4VPhysicalVolume* physiBrassTube;
    G4VPhysicalVolume* physiBrassTube2;
    G4VPhysicalVolume* physiBrassTube3;
    G4Tubs* solidFinalCollimator;
    G4VPhysicalVolume* physiFinalCollimator;
    
    G4VisAttributes* blue;
    G4VisAttributes* gray;
    G4VisAttributes* white;
    G4VisAttributes* red;
    G4VisAttributes* yellow;
    G4VisAttributes* green;
    G4VisAttributes* darkGreen;
    G4VisAttributes* darkOrange3;
    G4VisAttributes* skyBlue;
    
    G4Material* rangeShifterMaterial;
    G4Material* beamLineSupportMaterial;
    G4Material* vacuumZoneMaterial;
    G4Material* firstScatteringFoilMaterial;
    G4Material* kaptonWindowMaterial;
    G4Material* stopperMaterial;
    G4Material* secondScatteringFoilMaterial;
    G4Material* firstCollimatorMaterial;
    G4Material* holeFirstCollimatorMaterial;
    G4Material* modulatorBoxMaterial;
    G4Material* holeModulatorBoxMaterial;
    G4Material* layer1MonitorChamberMaterial;
    G4Material* layer2MonitorChamberMaterial;
    G4Material* layer3MonitorChamberMaterial;
    G4Material* layer4MonitorChamberMaterial;
    G4Material* MOPIMotherVolumeMaterial;
    G4Material* MOPIFirstKaptonLayerMaterial;
    G4Material* MOPIFirstAluminumLayerMaterial;
    G4Material* MOPIFirstAirGapMaterial;
    G4Material* MOPICathodeMaterial;
    G4Material* MOPISecondAirGapMaterial;
    G4Material* MOPISecondAluminumLayerMaterial;
    G4Material* MOPISecondKaptonLayerMaterial;
    G4Material* nozzleSupportMaterial;
    G4Material* holeNozzleSupportMaterial;
    
    G4Material* brassTubeMaterial;
    G4Material* brassTube2Material;
    G4Material* brassTube3Material;
    G4Material* finalCollimatorMaterial;
    
    
    HadrontherapyDetectorROGeometry* RO;
    
    
};
#endif
```

[Go Top](#GoTop)<a id='PassiveProtonBeamLine.cc'></a>
## src/PassiveProtonBeamLine.cc
```cpp
#include "globals.hh"
#include "G4SystemOfUnits.hh"
#include "G4Box.hh"
#include "G4Tubs.hh"
#include "G4VisAttributes.hh"
#include "G4Colour.hh"
#include "G4RunManager.hh"
#include "G4LogicalVolume.hh"
#include "G4PVPlacement.hh"
#include "G4RotationMatrix.hh"
#include "G4NistManager.hh"
#include "G4NistElementBuilder.hh"
#include "HadrontherapyDetectorConstruction.hh"
#include "HadrontherapyModulator.hh"
#include "PassiveProtonBeamLine.hh"
#include "PassiveProtonBeamLineMessenger.hh"


//G4bool PassiveProtonBeamLine::doCalculation = false;
/////////////////////////////////////////////////////////////////////////////
PassiveProtonBeamLine::PassiveProtonBeamLine():
modulator(0), physicalTreatmentRoom(0),hadrontherapyDetectorConstruction(0),
physiBeamLineSupport(0), physiBeamLineCover(0), physiBeamLineCover2(0),
firstScatteringFoil(0), physiFirstScatteringFoil(0), physiKaptonWindow(0),
solidStopper(0), physiStopper(0), secondScatteringFoil(0), physiSecondScatteringFoil(0),
physiFirstCollimator(0), solidRangeShifterBox(0), logicRangeShifterBox(0),
physiRangeShifterBox(0), physiSecondCollimator(0), physiFirstCollimatorModulatorBox(0),
physiHoleFirstCollimatorModulatorBox(0), physiSecondCollimatorModulatorBox(0),
physiHoleSecondCollimatorModulatorBox(0), physiMOPIMotherVolume(0),
physiFirstMonitorLayer1(0), physiFirstMonitorLayer2(0), physiFirstMonitorLayer3(0),
physiFirstMonitorLayer4(0), physiSecondMonitorLayer1(0), physiSecondMonitorLayer2(0),
physiSecondMonitorLayer3(0), physiSecondMonitorLayer4(0), physiNozzleSupport(0), physiBrassTube(0), solidFinalCollimator(0), physiFinalCollimator(0)
{
    // Messenger to change parameters of the passiveProtonBeamLine geometry
    passiveMessenger = new PassiveProtonBeamLineMessenger(this);
    
    //***************************** PW ***************************************
    static G4String ROGeometryName = "DetectorROGeometry";
    RO = new HadrontherapyDetectorROGeometry(ROGeometryName);
    
    G4cout << "Going to register Parallel world...";
    RegisterParallelWorld(RO);
    G4cout << "... done" << G4endl;

}
/////////////////////////////////////////////////////////////////////////////
PassiveProtonBeamLine::~PassiveProtonBeamLine()
{
    delete passiveMessenger;
    delete hadrontherapyDetectorConstruction;

}

/////////////////////////////////////////////////////////////////////////////
G4VPhysicalVolume* PassiveProtonBeamLine::Construct()
{
    // Sets default geometry and materials
    SetDefaultDimensions();
    
    // Construct the whole Passive Beam Line
    ConstructPassiveProtonBeamLine();
    
    //***************************** PW ***************************************
    if (!hadrontherapyDetectorConstruction)
        
        //***************************** PW ***************************************
        
        // HadrontherapyDetectorConstruction builds ONLY the phantom and the detector with its associated ROGeometry
        hadrontherapyDetectorConstruction = new HadrontherapyDetectorConstruction(physicalTreatmentRoom);
    
    
    //***************************** PW ***************************************
    
    hadrontherapyDetectorConstruction->InitializeDetectorROGeometry(RO,hadrontherapyDetectorConstruction->GetDetectorToWorldPosition());
    
    //***************************** PW ***************************************

    return physicalTreatmentRoom;
}

// In the following method the DEFAULTS used in the geometry of
// passive beam line are provided
// HERE THE USER CAN CHANGE THE GEOMETRY CHARACTERISTICS OF BEAM
// LINE ELEMENTS, ALTERNATIVELY HE/SHE CAN USE THE MACRO FILE (IF A
// MESSENGER IS PROVIDED)
//
// DEFAULT MATERIAL ARE ALSO PROVIDED
// and COLOURS ARE ALSO DEFINED
// ----------------------------------------------------------
/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetDefaultDimensions()
{
    // Set of coulors that can be used
    white = new G4VisAttributes( G4Colour());
    white -> SetVisibility(true);
    white -> SetForceSolid(true);
	
    blue = new G4VisAttributes(G4Colour(0. ,0. ,1.));
    blue -> SetVisibility(true);
    blue -> SetForceSolid(true);
	
    gray = new G4VisAttributes( G4Colour(0.5, 0.5, 0.5 ));
    gray-> SetVisibility(true);
    gray-> SetForceSolid(true);
	
    red = new G4VisAttributes(G4Colour(1. ,0. ,0.));
    red-> SetVisibility(true);
    red-> SetForceSolid(true);
	
    yellow = new G4VisAttributes(G4Colour(1., 1., 0. ));
    yellow-> SetVisibility(true);
    yellow-> SetForceSolid(true);
	
    green = new G4VisAttributes( G4Colour(25/255. , 255/255. ,  25/255. ));
    green -> SetVisibility(true);
    green -> SetForceSolid(true);
	
    darkGreen = new G4VisAttributes( G4Colour(0/255. , 100/255. ,  0/255. ));
    darkGreen -> SetVisibility(true);
    darkGreen -> SetForceSolid(true);
    
    darkOrange3 = new G4VisAttributes( G4Colour(205/255. , 102/255. ,  000/255. ));
    darkOrange3 -> SetVisibility(true);
    darkOrange3 -> SetForceSolid(true);
	
    skyBlue = new G4VisAttributes( G4Colour(135/255. , 206/255. ,  235/255. ));
    skyBlue -> SetVisibility(true);
    skyBlue -> SetForceSolid(true);
    
    
    // VACUUM PIPE: first track of the beam line is inside vacuum;
    // The PIPE contains the FIRST SCATTERING FOIL and the KAPTON WINDOW
    G4double defaultVacuumZoneXSize = 100.0 *mm;
    vacuumZoneXSize = defaultVacuumZoneXSize;
    
    G4double defaultVacuumZoneYSize = 52.5 *mm;
    vacuumZoneYSize = defaultVacuumZoneYSize;
    
    G4double defaultVacuumZoneZSize = 52.5 *mm;
    vacuumZoneZSize = defaultVacuumZoneZSize;
    
    G4double defaultVacuumZoneXPosition = -3010.0 *mm;
    vacuumZoneXPosition = defaultVacuumZoneXPosition;
    
    // FIRST SCATTERING FOIL: a thin foil performing a first scattering
    // of the original beam
    G4double defaultFirstScatteringFoilXSize = 0.0075 *mm;
    firstScatteringFoilXSize = defaultFirstScatteringFoilXSize;
    
    G4double defaultFirstScatteringFoilYSize = 52.5   *mm;
    firstScatteringFoilYSize = defaultFirstScatteringFoilYSize;
    
    G4double defaultFirstScatteringFoilZSize = 52.5   *mm;
    firstScatteringFoilZSize = defaultFirstScatteringFoilZSize;
    
    G4double defaultFirstScatteringFoilXPosition = 0.0 *mm;
    firstScatteringFoilXPosition = defaultFirstScatteringFoilXPosition;
    
    // KAPTON WINDOW: it prmits the passage of the beam from vacuum to air
    G4double defaultKaptonWindowXSize = 0.010*mm;
    kaptonWindowXSize = defaultKaptonWindowXSize;
    
    G4double defaultKaptonWindowYSize = 5.25*cm;
    kaptonWindowYSize = defaultKaptonWindowYSize;
    
    G4double defaultKaptonWindowZSize = 5.25*cm;
    kaptonWindowZSize = defaultKaptonWindowZSize;
    
    G4double defaultKaptonWindowXPosition = 100.0*mm - defaultKaptonWindowXSize;
    kaptonWindowXPosition = defaultKaptonWindowXPosition;
    
    // STOPPER: is a small cylinder able to stop the central component
    // of the beam (having a gaussian shape). It is connected to the SECON SCATTERING FOIL
    // and represent the second element of the scattering system
    G4double defaultInnerRadiusStopper = 0.*cm;
    innerRadiusStopper = defaultInnerRadiusStopper;
    
    G4double defaultHeightStopper = 3.5*mm;
    heightStopper = defaultHeightStopper;
    
    G4double defaultStartAngleStopper = 0.*deg;
    startAngleStopper = defaultStartAngleStopper;
    
    G4double defaultSpanningAngleStopper = 360.*deg;
    spanningAngleStopper = defaultSpanningAngleStopper;
    
    G4double defaultStopperXPosition = -2705.0 *mm;
    stopperXPosition = defaultStopperXPosition;
    
    G4double defaultStopperYPosition = 0.*m;
    stopperYPosition = defaultStopperYPosition;
    
    G4double defaultStopperZPosition = 0.*m;
    stopperZPosition = defaultStopperZPosition;
    
    G4double defaultOuterRadiusStopper = 2 *mm;
    outerRadiusStopper = defaultOuterRadiusStopper;
    
    // SECOND SCATTERING FOIL: it is another thin foil and provides the
    // final diffusion of the beam. It represents the third element of the scattering
    // system;
    G4double defaultSecondScatteringFoilXSize = 0.0125 *mm;
    secondScatteringFoilXSize = defaultSecondScatteringFoilXSize;
    
    G4double defaultSecondScatteringFoilYSize = 52.5   *mm;
    secondScatteringFoilYSize = defaultSecondScatteringFoilYSize;
    
    G4double defaultSecondScatteringFoilZSize = 52.5   *mm;
    secondScatteringFoilZSize = defaultSecondScatteringFoilZSize;
    
    G4double defaultSecondScatteringFoilXPosition = defaultStopperXPosition + defaultHeightStopper + defaultSecondScatteringFoilXSize;
    secondScatteringFoilXPosition = defaultSecondScatteringFoilXPosition;
    
    G4double defaultSecondScatteringFoilYPosition =  0 *mm;
    secondScatteringFoilYPosition = defaultSecondScatteringFoilYPosition;
    
    G4double defaultSecondScatteringFoilZPosition =  0 *mm;
    secondScatteringFoilZPosition = defaultSecondScatteringFoilZPosition;
    
    // RANGE SHIFTER: is a slab of PMMA acting as energy degreader of
    // primary beam
    
    //Default material of the range shifter
    
    G4double defaultRangeShifterXSize = 5. *mm;
    rangeShifterXSize = defaultRangeShifterXSize;
    
    G4double defaultRangeShifterYSize = 176. *mm;
    rangeShifterYSize = defaultRangeShifterYSize;
    
    G4double defaultRangeShifterZSize = 176. *mm;
    rangeShifterZSize = defaultRangeShifterZSize;
    
    G4double defaultRangeShifterXPosition = -2393.0 *mm;
    rangeShifterXPosition = defaultRangeShifterXPosition;
    
    G4double defaultRangeShifterYPosition = 0. *mm;
    rangeShifterYPosition = defaultRangeShifterYPosition;
    
    G4double defaultRangeShifterZPosition = 0. *mm;
    rangeShifterZPosition = defaultRangeShifterZPosition;
    
    // MOPI DETECTOR: two orthogonal microstrip gas detectors developed
    // by the INFN Section of Turin in collaboration with some
    // of the author of this example. It permits the
    // on-line check of the beam simmetry via the signal
    // integration of the collected charge for each strip.
    
    // Mother volume of MOPI
    
    G4double defaultMOPIMotherVolumeXSize = 12127.0 *um;
    MOPIMotherVolumeXSize = defaultMOPIMotherVolumeXSize;
    
    G4double defaultMOPIMotherVolumeYSize = 40.0 *cm;
    MOPIMotherVolumeYSize = defaultMOPIMotherVolumeYSize;
    
    G4double defaultMOPIMotherVolumeZSize = 40.0 *cm;
    MOPIMotherVolumeZSize = defaultMOPIMotherVolumeZSize;
    
    G4double defaultMOPIMotherVolumeXPosition = -1000.0 *mm;
    MOPIMotherVolumeXPosition = defaultMOPIMotherVolumeXPosition;
    
    G4double defaultMOPIMotherVolumeYPosition = 0.0 *mm;
    MOPIMotherVolumeYPosition = defaultMOPIMotherVolumeYPosition;
    
    G4double defaultMOPIMotherVolumeZPosition = 0.0 *mm;
    MOPIMotherVolumeZPosition = defaultMOPIMotherVolumeZPosition;
    
    // First Kapton Layer of MOPI
    G4double defaultMOPIFirstKaptonLayerXSize = 35 *um;
    MOPIFirstKaptonLayerXSize = defaultMOPIFirstKaptonLayerXSize;
    
    G4double defaultMOPIFirstKaptonLayerYSize = 30 *cm;
    MOPIFirstKaptonLayerYSize = defaultMOPIFirstKaptonLayerYSize;
    
    G4double defaultMOPIFirstKaptonLayerZSize = 30 *cm;
    MOPIFirstKaptonLayerZSize = defaultMOPIFirstKaptonLayerZSize;
    
    G4double defaultMOPIFirstKaptonLayerXPosition = -(MOPIMotherVolumeXSize/2 - (MOPIFirstKaptonLayerXSize/2));
    MOPIFirstKaptonLayerXPosition = defaultMOPIFirstKaptonLayerXPosition;
    
    G4double defaultMOPIFirstKaptonLayerYPosition = 0.0 *mm;
    MOPIFirstKaptonLayerYPosition = defaultMOPIFirstKaptonLayerYPosition;
    
    G4double defaultMOPIFirstKaptonLayerZPosition = 0.0 *mm;
    MOPIFirstKaptonLayerZPosition = defaultMOPIFirstKaptonLayerZPosition;
    
    //First Aluminum  Layer of MOPI
    G4double defaultMOPIFirstAluminumLayerXSize = 15 *um;
    MOPIFirstAluminumLayerXSize = defaultMOPIFirstAluminumLayerXSize;
    
    G4double defaultMOPIFirstAluminumLayerYSize = 30 *cm;
    MOPIFirstAluminumLayerYSize = defaultMOPIFirstAluminumLayerYSize;
    
    G4double defaultMOPIFirstAluminumLayerZSize = 30 *cm;
    MOPIFirstAluminumLayerZSize = defaultMOPIFirstAluminumLayerZSize;
    
    G4double defaultMOPIFirstAluminumLayerXPosition =
    MOPIFirstKaptonLayerXPosition + MOPIFirstKaptonLayerXSize/2 + MOPIFirstAluminumLayerXSize/2;
    MOPIFirstAluminumLayerXPosition = defaultMOPIFirstAluminumLayerXPosition;
    
    G4double defaultMOPIFirstAluminumLayerYPosition = 0.0 *mm;
    MOPIFirstAluminumLayerYPosition = defaultMOPIFirstAluminumLayerYPosition;
    
    G4double defaultMOPIFirstAluminumLayerZPosition = 0.0 *mm;
    MOPIFirstAluminumLayerZPosition = defaultMOPIFirstAluminumLayerZPosition;
    
    // First Air gap of MOPI
    G4double defaultMOPIFirstAirGapXSize = 6000 *um;
    MOPIFirstAirGapXSize = defaultMOPIFirstAirGapXSize;
    
    G4double defaultMOPIFirstAirGapYSize = 30 *cm;
    MOPIFirstAirGapYSize = defaultMOPIFirstAirGapYSize;
    
    G4double defaultMOPIFirstAirGapZSize = 30 *cm;
    MOPIFirstAirGapZSize = defaultMOPIFirstAirGapZSize;
    
    G4double defaultMOPIFirstAirGapXPosition =
    MOPIFirstAluminumLayerXPosition + MOPIFirstAluminumLayerXSize/2 + MOPIFirstAirGapXSize/2;
    MOPIFirstAirGapXPosition = defaultMOPIFirstAirGapXPosition;
    
    G4double defaultMOPIFirstAirGapYPosition = 0.0 *mm;
    MOPIFirstAirGapYPosition = defaultMOPIFirstAirGapYPosition;
    
    G4double defaultMOPIFirstAirGapZPosition = 0.0 *mm;
    MOPIFirstAirGapZPosition = defaultMOPIFirstAirGapZPosition;
    
    // Cathode of MOPI
    G4double defaultMOPICathodeXSize = 25.0 *um;
    MOPICathodeXSize = defaultMOPICathodeXSize;
    
    G4double defaultMOPICathodeYSize = 30.0 *cm;
    MOPICathodeYSize = defaultMOPICathodeYSize;
    
    G4double defaultMOPICathodeZSize = 30.0 *cm;
    MOPICathodeZSize = defaultMOPICathodeZSize;
    
    G4double defaultMOPICathodeXPosition =
    MOPIFirstAirGapXPosition + MOPIFirstAirGapXSize/2 + MOPICathodeXSize/2;
    MOPICathodeXPosition = defaultMOPICathodeXPosition;
    
    G4double defaultMOPICathodeYPosition = 0.0 *mm;
    MOPICathodeYPosition = defaultMOPICathodeYPosition;
    
    G4double defaultMOPICathodeZPosition = 0.0 *mm;
    MOPICathodeZPosition = defaultMOPICathodeZPosition;
    
    // Second Air gap of MOPI
    G4double defaultMOPISecondAirGapXSize = 6000 *um;
    MOPISecondAirGapXSize = defaultMOPISecondAirGapXSize;
    
    G4double defaultMOPISecondAirGapYSize = 30 *cm;
    MOPISecondAirGapYSize = defaultMOPISecondAirGapYSize;
    
    G4double defaultMOPISecondAirGapZSize = 30 *cm;
    MOPISecondAirGapZSize = defaultMOPISecondAirGapZSize;
    
    G4double defaultMOPISecondAirGapXPosition =
    MOPICathodeXPosition + MOPICathodeXSize/2 + MOPISecondAirGapXSize/2;
    MOPISecondAirGapXPosition = defaultMOPISecondAirGapXPosition;
    
    G4double defaultMOPISecondAirGapYPosition = 0.0 *mm;
    MOPISecondAirGapYPosition = defaultMOPISecondAirGapYPosition;
    
    G4double defaultMOPISecondAirGapZPosition = 0.0 *mm;
    MOPISecondAirGapZPosition = defaultMOPISecondAirGapZPosition;
    
    //Second Aluminum  Layer of MOPI
    G4double defaultMOPISecondAluminumLayerXSize = 15 *um;
    MOPISecondAluminumLayerXSize = defaultMOPISecondAluminumLayerXSize;
    
    G4double defaultMOPISecondAluminumLayerYSize = 30 *cm;
    MOPISecondAluminumLayerYSize = defaultMOPISecondAluminumLayerYSize;
    
    G4double defaultMOPISecondAluminumLayerZSize = 30 *cm;
    MOPISecondAluminumLayerZSize = defaultMOPISecondAluminumLayerZSize;
    
    G4double defaultMOPISecondAluminumLayerXPosition =
    MOPISecondAirGapXPosition + MOPISecondAirGapXSize/2 + MOPISecondAluminumLayerXSize/2;
    MOPISecondAluminumLayerXPosition = defaultMOPISecondAluminumLayerXPosition;
    
    G4double defaultMOPISecondAluminumLayerYPosition = 0.0 *mm;
    MOPISecondAluminumLayerYPosition = defaultMOPISecondAluminumLayerYPosition;
    
    G4double defaultMOPISecondAluminumLayerZPosition = 0.0 *mm;
    MOPISecondAluminumLayerZPosition = defaultMOPISecondAluminumLayerZPosition;
    
    // Second Kapton Layer of MOPI
    G4double defaultMOPISecondKaptonLayerXSize = 35 *um;
    MOPISecondKaptonLayerXSize = defaultMOPISecondKaptonLayerXSize;
    
    G4double defaultMOPISecondKaptonLayerYSize = 30 *cm;
    MOPISecondKaptonLayerYSize = defaultMOPISecondKaptonLayerYSize;
    
    G4double defaultMOPISecondKaptonLayerZSize = 30 *cm;
    MOPISecondKaptonLayerZSize = defaultMOPISecondKaptonLayerZSize;
    
    G4double defaultMOPISecondKaptonLayerXPosition =
    MOPISecondAluminumLayerXPosition + MOPISecondAluminumLayerXSize/2 + MOPISecondKaptonLayerXSize/2;
    MOPISecondKaptonLayerXPosition = defaultMOPISecondKaptonLayerXPosition;
    
    G4double defaultMOPISecondKaptonLayerYPosition = 0.0 *mm;
    MOPISecondKaptonLayerYPosition = defaultMOPISecondKaptonLayerYPosition;
    
    G4double defaultMOPISecondKaptonLayerZPosition = 0.0 *mm;
    MOPISecondKaptonLayerZPosition = defaultMOPISecondKaptonLayerZPosition;
    
    
    // FINAL COLLIMATOR: is the collimator giving the final transversal shape
    // of the beam
    G4double defaultinnerRadiusFinalCollimator = 7.5 *mm;
    innerRadiusFinalCollimator = defaultinnerRadiusFinalCollimator;
    
    // DEFAULT DEFINITION OF THE MATERIALS
    // All elements and compound definition follows the NIST database
    
    // ELEMENTS
    G4bool isotopes = false;
    G4Material* aluminumNist = G4NistManager::Instance()->FindOrBuildMaterial("G4_Al", isotopes);
    G4Material* tantalumNist = G4NistManager::Instance()->FindOrBuildMaterial("G4_Ta", isotopes);
    G4Material* copperNistAsMaterial = G4NistManager::Instance()->FindOrBuildMaterial("G4_Cu", isotopes);
    G4Element* zincNist = G4NistManager::Instance()->FindOrBuildElement("Zn");
    G4Element* copperNist = G4NistManager::Instance()->FindOrBuildElement("Cu");
    
    // COMPOUND
    G4Material* airNist =  G4NistManager::Instance()->FindOrBuildMaterial("G4_AIR", isotopes);
    G4Material* kaptonNist = G4NistManager::Instance()->FindOrBuildMaterial("G4_KAPTON", isotopes);
    G4Material* galacticNist = G4NistManager::Instance()->FindOrBuildMaterial("G4_Galactic", isotopes);
    G4Material* PMMANist = G4NistManager::Instance()->FindOrBuildMaterial("G4_PLEXIGLASS", isotopes);
    G4Material* mylarNist = G4NistManager::Instance()->FindOrBuildMaterial("G4_MYLAR", isotopes);
    
    G4double d; // Density
    G4int nComponents;// Number of components
    G4double fractionmass; // Fraction in mass of an element in a material
    
    d = 8.40*g/cm3;
    nComponents = 2;
    G4Material* brass = new G4Material("Brass", d, nComponents);
    brass -> AddElement(zincNist, fractionmass = 30 *perCent);
    brass -> AddElement(copperNist, fractionmass = 70 *perCent);
    
    
    //***************************** PW ***************************************
    
    // DetectorROGeometry Material
    new G4Material("dummyMat", 1., 1.*g/mole, 1.*g/cm3);
    
    //***************************** PW ***************************************
    
    
    
    // MATERIAL ASSIGNMENT
    // Range shifter
    rangeShifterMaterial = airNist;
    
    // Support of the beam line
    beamLineSupportMaterial = aluminumNist;
    
    // Vacuum pipe
    vacuumZoneMaterial = galacticNist;
    
    // Material of the fisrt scattering foil
    firstScatteringFoilMaterial = tantalumNist;
    
    // Material of kapton window
    kaptonWindowMaterial = kaptonNist;
    
    // Material of the stopper
    stopperMaterial = brass;
    
    // Material of the second scattering foil
    secondScatteringFoilMaterial = tantalumNist;
    
    // Materials of the collimators
    firstCollimatorMaterial = PMMANist;
    holeFirstCollimatorMaterial = airNist;
    
    // Box containing the modulator wheel
    modulatorBoxMaterial = aluminumNist;
    holeModulatorBoxMaterial = airNist;
    
    // Materials of the monitor chamber
    layer1MonitorChamberMaterial = kaptonNist;
    layer2MonitorChamberMaterial = copperNistAsMaterial;
    layer3MonitorChamberMaterial = airNist;
    layer4MonitorChamberMaterial = copperNistAsMaterial;
    
    // Mother volume of the MOPI detector
    MOPIMotherVolumeMaterial = airNist;
    MOPIFirstKaptonLayerMaterial = kaptonNist;
    MOPIFirstAluminumLayerMaterial = aluminumNist;
    MOPIFirstAirGapMaterial = airNist;
    MOPICathodeMaterial = mylarNist;
    MOPISecondAirGapMaterial = airNist;
    MOPISecondAluminumLayerMaterial = aluminumNist;
    MOPISecondKaptonLayerMaterial = kaptonNist;
    
    // material of the final nozzle
    nozzleSupportMaterial = PMMANist;
    brassTubeMaterial = brassTube2Material = brassTube3Material = brass;
    holeNozzleSupportMaterial = airNist;
    
    // Material of the final collimator
    finalCollimatorMaterial = brass;
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::ConstructPassiveProtonBeamLine()
{
    // -----------------------------
    // Treatment room - World volume
    //------------------------------
    // Treatment room sizes
    const G4double worldX = 400.0 *cm;
    const G4double worldY = 400.0 *cm;
    const G4double worldZ = 400.0 *cm;
    G4bool isotopes = false;
    
    G4Material* airNist =  G4NistManager::Instance()->FindOrBuildMaterial("G4_AIR", isotopes);
    G4Box* treatmentRoom = new G4Box("TreatmentRoom",worldX,worldY,worldZ);
    G4LogicalVolume* logicTreatmentRoom = new G4LogicalVolume(treatmentRoom,
                                                              airNist,
                                                              "logicTreatmentRoom",
                                                              0,0,0);
    physicalTreatmentRoom = new G4PVPlacement(0,
                                              G4ThreeVector(),
                                              "physicalTreatmentRoom",
                                              logicTreatmentRoom,
                                              0,false,0);
    
    
    // The treatment room is invisible in the Visualisation
    //logicTreatmentRoom -> SetVisAttributes(G4VisAttributes::Invisible);
    
    // Components of the Passive Proton Beam Line
    HadrontherapyBeamLineSupport();
    HadrontherapyBeamScatteringFoils();
    HadrontherapyRangeShifter();
    HadrontherapyBeamCollimators();
    HadrontherapyBeamMonitoring();
    HadrontherapyMOPIDetector();
    HadrontherapyBeamNozzle();
    HadrontherapyBeamFinalCollimator();
    
    // The following lines construc a typical modulator wheel inside the Passive Beam line.
    // Please remember to set the nodulator material (default is air, i.e. no modulator!)
    // in the HadrontherapyModulator.cc file
    modulator = new HadrontherapyModulator();
    modulator -> BuildModulator(physicalTreatmentRoom);
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyBeamLineSupport()
{
    // ------------------//
    // BEAM LINE SUPPORT //
    //-------------------//
    const G4double beamLineSupportXSize = 1.5*m;
    const G4double beamLineSupportYSize = 20.*mm;
    const G4double beamLineSupportZSize = 600.*mm;
    
    const G4double beamLineSupportXPosition = -1745.09 *mm;
    const G4double beamLineSupportYPosition = -230. *mm;
    const G4double beamLineSupportZPosition = 0.*mm;
    
  G4Box* beamLineSupport = new G4Box("BeamLineSupport",
                                       beamLineSupportXSize,
                                       beamLineSupportYSize,
                                       beamLineSupportZSize);
    
    G4LogicalVolume* logicBeamLineSupport = new G4LogicalVolume(beamLineSupport,
                                                                beamLineSupportMaterial,
                                                                "BeamLineSupport");
    physiBeamLineSupport = new G4PVPlacement(0, G4ThreeVector(beamLineSupportXPosition,
                                                              beamLineSupportYPosition,
                                                              beamLineSupportZPosition),
                                             "BeamLineSupport",
                                             logicBeamLineSupport,
                                             physicalTreatmentRoom, false, 0);
    
    // Visualisation attributes of the beam line support
    
    logicBeamLineSupport -> SetVisAttributes(gray);

    //---------------------------------//
    //  Beam line cover 1 (left panel) //
    //---------------------------------//
    const G4double beamLineCoverXSize = 1.5*m;
    const G4double beamLineCoverYSize = 750.*mm;
    const G4double beamLineCoverZSize = 10.*mm;
    
    const G4double beamLineCoverXPosition = -1745.09 *mm;
    const G4double beamLineCoverYPosition = -1000.*mm;
    const G4double beamLineCoverZPosition = 600.*mm;
    
   G4Box* beamLineCover = new G4Box("BeamLineCover",
                                     beamLineCoverXSize,
                                     beamLineCoverYSize,
                                     beamLineCoverZSize);
    
    G4LogicalVolume* logicBeamLineCover = new G4LogicalVolume(beamLineCover,
                                                              beamLineSupportMaterial,
                                                              "BeamLineCover");
    
    physiBeamLineCover = new G4PVPlacement(0, G4ThreeVector(beamLineCoverXPosition,
                                                            beamLineCoverYPosition,
                                                            beamLineCoverZPosition),
                                           "BeamLineCover",
                                           logicBeamLineCover,
                                           physicalTreatmentRoom,
                                           false,
                                           0);
    
    // ---------------------------------//
    //  Beam line cover 2 (rigth panel) //
    // ---------------------------------//
    // It has the same characteristic of beam line cover 1 but set in a different position
    physiBeamLineCover2 = new G4PVPlacement(0, G4ThreeVector(beamLineCoverXPosition,
                                                             beamLineCoverYPosition,
                                                             - beamLineCoverZPosition),
                                            "BeamLineCover2",
                                            logicBeamLineCover,
                                            physicalTreatmentRoom,
                                            false,
                                            0);
    
    logicBeamLineCover -> SetVisAttributes(blue);

    }

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyBeamScatteringFoils()
{
   // ------------//
    // VACUUM PIPE //
    //-------------//
    //
    // First track of the beam line is inside vacuum;
    // The PIPE contains the FIRST SCATTERING FOIL and the KAPTON WINDOW
    G4Box* vacuumZone = new G4Box("VacuumZone", vacuumZoneXSize, vacuumZoneYSize, vacuumZoneZSize);
    G4LogicalVolume* logicVacuumZone = new G4LogicalVolume(vacuumZone, vacuumZoneMaterial, "VacuumZone");
    G4VPhysicalVolume* physiVacuumZone = new G4PVPlacement(0, G4ThreeVector(vacuumZoneXPosition, 0., 0.),
                                                           "VacuumZone", logicVacuumZone, physicalTreatmentRoom, false, 0);
    // --------------------------//
    // THE FIRST SCATTERING FOIL //
    // --------------------------//
    // A thin foil performing a first scattering
    // of the original beam
    firstScatteringFoil = new G4Box("FirstScatteringFoil",
                                    firstScatteringFoilXSize,
                                    firstScatteringFoilYSize,
                                    firstScatteringFoilZSize);
    
    G4LogicalVolume* logicFirstScatteringFoil = new G4LogicalVolume(firstScatteringFoil,
                                                                    firstScatteringFoilMaterial,
                                                                    "FirstScatteringFoil");
    
    physiFirstScatteringFoil = new G4PVPlacement(0, G4ThreeVector(firstScatteringFoilXPosition, 0.,0.),
                                                 "FirstScatteringFoil", logicFirstScatteringFoil, physiVacuumZone,
                                                 false, 0);
    
    logicFirstScatteringFoil -> SetVisAttributes(skyBlue);
    // -------------------//
    // THE KAPTON WINDOWS //
    //--------------------//
    //It prmits the passage of the beam from vacuum to air
 
    G4Box* solidKaptonWindow = new G4Box("KaptonWindow",
                                         kaptonWindowXSize,
                                         kaptonWindowYSize,
                                         kaptonWindowZSize);
    
    G4LogicalVolume* logicKaptonWindow = new G4LogicalVolume(solidKaptonWindow,
                                                             kaptonWindowMaterial,
                                                             "KaptonWindow");
    
    physiKaptonWindow = new G4PVPlacement(0, G4ThreeVector(kaptonWindowXPosition, 0., 0.),
                                          "KaptonWindow", logicKaptonWindow,
                                          physiVacuumZone, false,	0);
    
    logicKaptonWindow -> SetVisAttributes(darkOrange3);
    
    // ------------//
    // THE STOPPER //
    //-------------//
    // Is a small cylinder able to stop the central component
    // of the beam (having a gaussian shape). It is connected to the SECON SCATTERING FOIL
    // and represent the second element of the scattering system
    G4double phi = 90. *deg;
    // Matrix definition for a 90 deg rotation with respect to Y axis
    G4RotationMatrix rm;
    rm.rotateY(phi);
    
    solidStopper = new G4Tubs("Stopper",
                              innerRadiusStopper,
                              outerRadiusStopper,
                              heightStopper,
                              startAngleStopper,
                              spanningAngleStopper);
    
    logicStopper = new G4LogicalVolume(solidStopper,
                                       stopperMaterial,
                                       "Stopper",
                                       0, 0, 0);
    
    physiStopper = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector(stopperXPosition,
                                                                     stopperYPosition,
                                                                     stopperZPosition)),
                                     "Stopper",
                                     logicStopper,
                                     physicalTreatmentRoom,
                                     false,
                                     0);
    
    logicStopper -> SetVisAttributes(red);
    
    // ---------------------------//
    // THE SECOND SCATTERING FOIL //
    // ---------------------------//
    // It is another thin foil and provides the
    // final diffusion of the beam. It represents the third element of the scattering
    // system;
    
    secondScatteringFoil = new G4Box("SecondScatteringFoil",
                                     secondScatteringFoilXSize,
                                     secondScatteringFoilYSize,
                                     secondScatteringFoilZSize);
    
    G4LogicalVolume* logicSecondScatteringFoil = new G4LogicalVolume(secondScatteringFoil,
                                                                     secondScatteringFoilMaterial,
                                                                     "SecondScatteringFoil");
    
    physiSecondScatteringFoil = new G4PVPlacement(0, G4ThreeVector(secondScatteringFoilXPosition,
                                                                   secondScatteringFoilYPosition,
                                                                   secondScatteringFoilZPosition),
                                                  "SeconScatteringFoil",
                                                  logicSecondScatteringFoil,
                                                  physicalTreatmentRoom,
                                                  false,
                                                  0);
    
    logicSecondScatteringFoil -> SetVisAttributes(skyBlue);
 
 
}
/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyRangeShifter()
{
    // ---------------------------- //
    //         THE RANGE SHIFTER    //
    // -----------------------------//
    // It is a slab of PMMA acting as energy degreader of
    // primary beam
 
    
    solidRangeShifterBox = new G4Box("RangeShifterBox",
                                     rangeShifterXSize,
                                     rangeShifterYSize,
                                     rangeShifterZSize);
    
    logicRangeShifterBox = new G4LogicalVolume(solidRangeShifterBox,
                                               rangeShifterMaterial,
                                               "RangeShifterBox");
    physiRangeShifterBox = new G4PVPlacement(0,
                                             G4ThreeVector(rangeShifterXPosition, 0., 0.),
                                             "RangeShifterBox",
                                             logicRangeShifterBox,
                                             physicalTreatmentRoom,
                                             false,
                                             0);
    
    
    logicRangeShifterBox -> SetVisAttributes(yellow);
     
    
}
/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyBeamCollimators()
{
    
    
    // -----------------//
    // FIRST COLLIMATOR //
    // -----------------//
    // It is a slab of PMMA with an hole in its center
    const G4double firstCollimatorXSize = 20.*mm;
    const G4double firstCollimatorYSize = 100.*mm;
    const G4double firstCollimatorZSize = 100.*mm;
    
    const G4double firstCollimatorXPosition = -2673.00*mm;
    const G4double firstCollimatorYPosition = 0.*mm;
    const G4double firstCollimatorZPosition = 0.*mm;
    
    
    G4Box* solidFirstCollimator = new G4Box("FirstCollimator",
                                            firstCollimatorXSize,
                                            firstCollimatorYSize,
                                            firstCollimatorZSize);
    
    G4LogicalVolume* logicFirstCollimator = new G4LogicalVolume(solidFirstCollimator,
                                                                firstCollimatorMaterial,
                                                                "FirstCollimator");
    
    physiFirstCollimator = new G4PVPlacement(0, G4ThreeVector(firstCollimatorXPosition,
                                                              firstCollimatorYPosition,
                                                              firstCollimatorZPosition),
                                             "FirstCollimator",
                                             logicFirstCollimator,
                                             physicalTreatmentRoom,
                                             false,
                                             0);
    // ----------------------------//
    // Hole of the first collimator//
    //-----------------------------//
    G4double innerRadiusHoleFirstCollimator   = 0.*mm;
    G4double outerRadiusHoleFirstCollimator   = 15.*mm;
    G4double hightHoleFirstCollimator         = 20.*mm;
    G4double startAngleHoleFirstCollimator    = 0.*deg;
    G4double spanningAngleHoleFirstCollimator = 360.*deg;
    
    G4Tubs* solidHoleFirstCollimator = new G4Tubs("HoleFirstCollimator",
                                                  innerRadiusHoleFirstCollimator,
                                                  outerRadiusHoleFirstCollimator,
                                                  hightHoleFirstCollimator,
                                                  startAngleHoleFirstCollimator,
                                                  spanningAngleHoleFirstCollimator);
    
    G4LogicalVolume* logicHoleFirstCollimator = new G4LogicalVolume(solidHoleFirstCollimator,
                                                                    holeFirstCollimatorMaterial,
                                                                    "HoleFirstCollimator",
                                                                    0, 0, 0);
    G4double phi = 90. *deg;
    // Matrix definition for a 90 deg rotation. Also used for other volumes
    G4RotationMatrix rm;
    rm.rotateY(phi);
    
    physiHoleFirstCollimator = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector()),
                                                 "HoleFirstCollimator",
                                                 logicHoleFirstCollimator,
                                                 physiFirstCollimator,
                                                 false,
                                                 0);
    // ------------------//
    // SECOND COLLIMATOR //
    //-------------------//
    // It is a slab of PMMA with an hole in its center
    const G4double secondCollimatorXPosition = -1900.00*mm;
    const G4double secondCollimatorYPosition =  0*mm;
    const G4double secondCollimatorZPosition =  0*mm;
    
    physiSecondCollimator = new G4PVPlacement(0, G4ThreeVector(secondCollimatorXPosition,
                                                               secondCollimatorYPosition,
                                                               secondCollimatorZPosition),
                                              "SecondCollimator",
                                              logicFirstCollimator,
                                              physicalTreatmentRoom,
                                              false,
                                              0);
    
    // ------------------------------//
    // Hole of the second collimator //
    // ------------------------------//
    physiHoleSecondCollimator = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector()),
                                                  "HoleSecondCollimator",
                                                  logicHoleFirstCollimator,
                                                  physiSecondCollimator,
                                                  false,
                                                  0);
    
    // --------------------------------------//
    // FIRST SIDE OF THE MODULATOR BOX      //
    // --------------------------------------//
    // The modulator box is an aluminum box in which
    // the range shifter and the energy modulator are located
    // In this example only the entrance and exit
    // faces of the box are simulated.
    // Each face is an aluminum slab with an hole in its center
    
    const G4double firstCollimatorModulatorXSize = 10.*mm;
    const G4double firstCollimatorModulatorYSize = 200.*mm;
    const G4double firstCollimatorModulatorZSize = 200.*mm;
    
    const G4double firstCollimatorModulatorXPosition = -2523.00*mm;
    const G4double firstCollimatorModulatorYPosition = 0.*mm;
    const G4double firstCollimatorModulatorZPosition = 0.*mm;
    
   G4Box* solidFirstCollimatorModulatorBox = new G4Box("FirstCollimatorModulatorBox",
                                                        firstCollimatorModulatorXSize,
                                                        firstCollimatorModulatorYSize,
                                                        firstCollimatorModulatorZSize);
    
    G4LogicalVolume* logicFirstCollimatorModulatorBox = new G4LogicalVolume(solidFirstCollimatorModulatorBox,
                                                                            modulatorBoxMaterial,
                                                                            "FirstCollimatorModulatorBox");
    
    physiFirstCollimatorModulatorBox = new G4PVPlacement(0, G4ThreeVector(firstCollimatorModulatorXPosition,
                                                                          firstCollimatorModulatorYPosition,
                                                                          firstCollimatorModulatorZPosition),
                                                         "FirstCollimatorModulatorBox",
                                                         logicFirstCollimatorModulatorBox,
                                                         physicalTreatmentRoom, false, 0);
    
    // ----------------------------------------------------//
    //   Hole of the first collimator of the modulator box //
    // ----------------------------------------------------//
    const G4double innerRadiusHoleFirstCollimatorModulatorBox = 0.*mm;
    const G4double outerRadiusHoleFirstCollimatorModulatorBox = 31.*mm;
    const G4double hightHoleFirstCollimatorModulatorBox = 10.*mm;
    const G4double startAngleHoleFirstCollimatorModulatorBox = 0.*deg;
    const G4double spanningAngleHoleFirstCollimatorModulatorBox = 360.*deg;
    
    G4Tubs* solidHoleFirstCollimatorModulatorBox  = new G4Tubs("HoleFirstCollimatorModulatorBox",
                                                               innerRadiusHoleFirstCollimatorModulatorBox,
                                                               outerRadiusHoleFirstCollimatorModulatorBox,
                                                               hightHoleFirstCollimatorModulatorBox ,
                                                               startAngleHoleFirstCollimatorModulatorBox,
                                                               spanningAngleHoleFirstCollimatorModulatorBox);
    
    G4LogicalVolume* logicHoleFirstCollimatorModulatorBox = new G4LogicalVolume(solidHoleFirstCollimatorModulatorBox,
                                                                                holeModulatorBoxMaterial,
                                                                                "HoleFirstCollimatorModulatorBox",
                                                                                0, 0, 0);
    
    physiHoleFirstCollimatorModulatorBox = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector()),
                                                             "HoleFirstCollimatorModulatorBox",
                                                             logicHoleFirstCollimatorModulatorBox,
                                                             physiFirstCollimatorModulatorBox, false, 0);
    
    // --------------------------------------------------//
    //       SECOND SIDE OF THE MODULATOR BOX            //
    // --------------------------------------------------//
    const G4double secondCollimatorModulatorXSize = 10.*mm;
    const G4double secondCollimatorModulatorYSize = 200.*mm;
    const G4double secondCollimatorModulatorZSize = 200.*mm;
    
    const G4double secondCollimatorModulatorXPosition = -1953.00 *mm;
    
    const G4double secondCollimatorModulatorYPosition = 0.*mm;
    const G4double secondCollimatorModulatorZPosition = 0.*mm;
    
    G4Box* solidSecondCollimatorModulatorBox = new G4Box("SecondCollimatorModulatorBox",
                                                         secondCollimatorModulatorXSize,
                                                         secondCollimatorModulatorYSize,
                                                         secondCollimatorModulatorZSize);
    
    G4LogicalVolume* logicSecondCollimatorModulatorBox = new G4LogicalVolume(solidSecondCollimatorModulatorBox,
                                                                             modulatorBoxMaterial,
                                                                             "SecondCollimatorModulatorBox");
    
    physiSecondCollimatorModulatorBox = new G4PVPlacement(0, G4ThreeVector(secondCollimatorModulatorXPosition,
                                                                           secondCollimatorModulatorYPosition,
                                                                           secondCollimatorModulatorZPosition),
                                                          "SecondCollimatorModulatorBox",
                                                          logicSecondCollimatorModulatorBox,
                                                          physicalTreatmentRoom, false, 0);
    
    // ----------------------------------------------//
    //   Hole of the second collimator modulator box //
    // ----------------------------------------------//
    const G4double innerRadiusHoleSecondCollimatorModulatorBox = 0.*mm;
    const G4double outerRadiusHoleSecondCollimatorModulatorBox = 31.*mm;
    const G4double hightHoleSecondCollimatorModulatorBox = 10.*mm;
    const G4double startAngleHoleSecondCollimatorModulatorBox = 0.*deg;
    const G4double spanningAngleHoleSecondCollimatorModulatorBox = 360.*deg;
    
    G4Tubs* solidHoleSecondCollimatorModulatorBox  = new G4Tubs("HoleSecondCollimatorModulatorBox",
                                                                innerRadiusHoleSecondCollimatorModulatorBox,
                                                                outerRadiusHoleSecondCollimatorModulatorBox,
                                                                hightHoleSecondCollimatorModulatorBox ,
                                                                startAngleHoleSecondCollimatorModulatorBox,
                                                                spanningAngleHoleSecondCollimatorModulatorBox);
    
    G4LogicalVolume* logicHoleSecondCollimatorModulatorBox = new G4LogicalVolume(solidHoleSecondCollimatorModulatorBox,
                                                                                 holeModulatorBoxMaterial,
                                                                                 "HoleSecondCollimatorModulatorBox",
                                                                                 0, 0, 0);
    
    physiHoleSecondCollimatorModulatorBox = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector()),
                                                              "HoleSecondCollimatorModulatorBox",
                                                              logicHoleSecondCollimatorModulatorBox,
                                                              physiSecondCollimatorModulatorBox, false, 0);
    
    logicFirstCollimator -> SetVisAttributes(yellow);
    logicFirstCollimatorModulatorBox -> SetVisAttributes(blue);
    logicSecondCollimatorModulatorBox -> SetVisAttributes(blue);


  
  }

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyBeamMonitoring()
{
    // ----------------------------
    //   THE FIRST MONITOR CHAMBER
    // ----------------------------
    // A monitor chamber is a free-air  ionisation chamber
    // able to measure do proton fluence during the treatment.
    // Here its responce is not simulated in terms of produced
    // charge but only the energy losses are taked into account.
    // Each chamber consist of 9 mm of air in a box
    // that has two layers one of kapton and one
    // of copper
    const G4double monitor1XSize = 4.525022*mm;
    const G4double monitor2XSize = 0.000011*mm;
    const G4double monitor3XSize = 4.5*mm;
    const G4double monitorYSize = 10.*cm;
    const G4double monitorZSize = 10.*cm;
    const G4double monitor1XPosition = -1262.47498 *mm;
    const G4double monitor2XPosition = -4.500011*mm;
    const G4double monitor4XPosition = 4.500011*mm;
  
    

    G4Box* solidFirstMonitorLayer1 = new G4Box("FirstMonitorLayer1",
                                               monitor1XSize,
                                               monitorYSize,
                                               monitorZSize);
    
    G4LogicalVolume* logicFirstMonitorLayer1 = new G4LogicalVolume(solidFirstMonitorLayer1,
                                                                   layer1MonitorChamberMaterial,
                                                                   "FirstMonitorLayer1");
    
    physiFirstMonitorLayer1 = new G4PVPlacement(0,
                                                G4ThreeVector(monitor1XPosition,0.*cm,0.*cm),
                                                "FirstMonitorLayer1",
                                                logicFirstMonitorLayer1,
                                                physicalTreatmentRoom,
                                                false,
                                                0);
    
    G4Box* solidFirstMonitorLayer2 = new G4Box("FirstMonitorLayer2",
                                               monitor2XSize,
                                               monitorYSize,
                                               monitorZSize);
    
    G4LogicalVolume* logicFirstMonitorLayer2 = new G4LogicalVolume(solidFirstMonitorLayer2,
                                                                   layer2MonitorChamberMaterial,
                                                                   "FirstMonitorLayer2");
    
    physiFirstMonitorLayer2 = new G4PVPlacement(0, G4ThreeVector(monitor2XPosition,0.*cm,0.*cm),
                                                "FirstMonitorLayer2",
                                                logicFirstMonitorLayer2,
                                                physiFirstMonitorLayer1,
                                                false,
                                                0);
    
    G4Box* solidFirstMonitorLayer3 = new G4Box("FirstMonitorLayer3",
                                               monitor3XSize,
                                               monitorYSize,
                                               monitorZSize);
    
    G4LogicalVolume* logicFirstMonitorLayer3 = new G4LogicalVolume(solidFirstMonitorLayer3,
                                                                   layer3MonitorChamberMaterial,
                                                                   "FirstMonitorLayer3");
    
    physiFirstMonitorLayer3 = new G4PVPlacement(0,
                                                G4ThreeVector(0.*mm,0.*cm,0.*cm),
                                                "MonitorLayer3",
                                                logicFirstMonitorLayer3,
                                                physiFirstMonitorLayer1,
                                                false,
                                                0);
    
    G4Box* solidFirstMonitorLayer4 = new G4Box("FirstMonitorLayer4",
                                               monitor2XSize,
                                               monitorYSize,
                                               monitorZSize);
    
    G4LogicalVolume* logicFirstMonitorLayer4 = new G4LogicalVolume(solidFirstMonitorLayer4,
                                                                   layer4MonitorChamberMaterial,
                                                                   "FirstMonitorLayer4");
    
    physiFirstMonitorLayer4 = new G4PVPlacement(0, G4ThreeVector(monitor4XPosition,0.*cm,0.*cm),
                                                "FirstMonitorLayer4",
                                                logicFirstMonitorLayer4,
                                                physiFirstMonitorLayer1, false, 0);
    // ----------------------------//
    // THE SECOND MONITOR CHAMBER  //
    // ----------------------------//
    physiSecondMonitorLayer1 = new G4PVPlacement(0, G4ThreeVector(-1131.42493 *mm,0.*cm,0.*cm),
                                                 "SecondMonitorLayer1", logicFirstMonitorLayer1,physicalTreatmentRoom, false, 0);
    
    physiSecondMonitorLayer2 = new G4PVPlacement(0, G4ThreeVector( monitor2XPosition,0.*cm,0.*cm), "SecondMonitorLayer2",
                                                 logicFirstMonitorLayer2, physiSecondMonitorLayer1, false, 0);
    
    physiSecondMonitorLayer3 = new G4PVPlacement(0, G4ThreeVector(0.*mm,0.*cm,0.*cm), "MonitorLayer3",
                                                 logicFirstMonitorLayer3, physiSecondMonitorLayer1, false, 0);
    
    physiSecondMonitorLayer4 = new G4PVPlacement(0, G4ThreeVector(monitor4XPosition,0.*cm,0.*cm), "SecondMonitorLayer4",
                                                 logicFirstMonitorLayer4, physiSecondMonitorLayer1, false, 0);
    
    logicFirstMonitorLayer3 -> SetVisAttributes(white);
    

    
     }
/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyMOPIDetector()
{

    // --------------------------------//
    //        THE MOPI DETECTOR        //
    // --------------------------------//
    // MOPI DETECTOR: two orthogonal microstrip gas detectors developed
    // by the INFN Section of Turin in collaboration with some
    // of the author of this example. It permits the
    // on-line check of the beam simmetry via the signal
    // integration of the collected charge for each strip.
    //
    // In this example it is simulated as:
    // 1. First anode: 35 mu of kapton + 15 mu of aluminum,
    // 2. First air gap: 6 mm of air,
    // 3. The cathode: 1 mu Al + 25 mu mylar + 1 mu Al
    //    (in common with the two air gap),
    // 4. Second air gap: 6 mm of air,
    // 5  Second anode: 15 mu Al + 35 mu kapton
    // Color used in the graphical output
    
    
    // Mother volume
    solidMOPIMotherVolume = new G4Box("MOPIMotherVolume",
                                      MOPIMotherVolumeXSize/2,
                                      MOPIMotherVolumeYSize/2,
                                      MOPIMotherVolumeYSize/2);
    
    logicMOPIMotherVolume = new G4LogicalVolume(solidMOPIMotherVolume,
                                                MOPIMotherVolumeMaterial,
                                                "MOPIMotherVolume");
    physiMOPIMotherVolume = new G4PVPlacement(0,
                                              G4ThreeVector(MOPIMotherVolumeXPosition,
                                                            MOPIMotherVolumeYPosition,
                                                            MOPIMotherVolumeZPosition),
                                              "MOPIMotherVolume",
                                              logicMOPIMotherVolume,
                                              physicalTreatmentRoom,
                                              false,
                                              0);
    
    // First Kapton layer
    solidMOPIFirstKaptonLayer = new G4Box("MOPIFirstKaptonLayer",
                                          MOPIFirstKaptonLayerXSize/2,
                                          MOPIFirstKaptonLayerYSize/2 ,
                                          MOPIFirstKaptonLayerZSize/2);
    
    logicMOPIFirstKaptonLayer = new G4LogicalVolume(solidMOPIFirstKaptonLayer,
                                                    MOPIFirstKaptonLayerMaterial,
                                                    "MOPIFirstKaptonLayer");
    
    physiMOPIFirstKaptonLayer = new G4PVPlacement(0,
                                                  G4ThreeVector(MOPIFirstKaptonLayerXPosition,
                                                                MOPIFirstKaptonLayerYPosition ,
                                                                MOPIFirstKaptonLayerZPosition),
                                                  "MOPIFirstKaptonLayer",
                                                  logicMOPIFirstKaptonLayer,
                                                  physiMOPIMotherVolume,
                                                  false,
                                                  0);
    
    // First Aluminum layer
    solidMOPIFirstAluminumLayer = new G4Box("MOPIFirstAluminumLayer",
                                            MOPIFirstAluminumLayerXSize/2,
                                            MOPIFirstAluminumLayerYSize/2 ,
                                            MOPIFirstAluminumLayerZSize/2);
    
    logicMOPIFirstAluminumLayer = new G4LogicalVolume(solidMOPIFirstAluminumLayer,
                                                      MOPIFirstAluminumLayerMaterial,
                                                      "MOPIFirstAluminumLayer");
    
    physiMOPIFirstAluminumLayer = new G4PVPlacement(0,
                                                    G4ThreeVector(MOPIFirstAluminumLayerXPosition,
                                                                  MOPIFirstAluminumLayerYPosition ,
                                                                  MOPIFirstAluminumLayerZPosition),
                                                    "MOPIFirstAluminumLayer",
                                                    logicMOPIFirstAluminumLayer, physiMOPIMotherVolume, false, 0);
    
    // First Air GAP
    solidMOPIFirstAirGap = new G4Box("MOPIFirstAirGap",
                                     MOPIFirstAirGapXSize/2,
                                     MOPIFirstAirGapYSize/2,
                                     MOPIFirstAirGapZSize/2);
    
    logicMOPIFirstAirGap = new G4LogicalVolume(solidMOPIFirstAirGap,
                                               MOPIFirstAirGapMaterial,
                                               "MOPIFirstAirgap");
    
    physiMOPIFirstAirGap = new G4PVPlacement(0,
                                             G4ThreeVector(MOPIFirstAirGapXPosition,
                                                           MOPIFirstAirGapYPosition ,
                                                           MOPIFirstAirGapZPosition),
                                             "MOPIFirstAirGap",
                                             logicMOPIFirstAirGap, physiMOPIMotherVolume, false, 0);
    
    
    // The Cathode
    solidMOPICathode = new G4Box("MOPICathode",
                                 MOPICathodeXSize/2,
                                 MOPICathodeYSize/2,
                                 MOPICathodeZSize/2);
    
    logicMOPICathode = new G4LogicalVolume(solidMOPICathode,
                                           MOPICathodeMaterial,
                                           "MOPICathode");
    
    physiMOPICathode = new G4PVPlacement(0,
                                         G4ThreeVector(MOPICathodeXPosition,
                                                       MOPICathodeYPosition ,
                                                       MOPICathodeZPosition),
                                         "MOPICathode",
                                         logicMOPICathode,
                                         physiMOPIMotherVolume, false, 0);
    
    // Second Air GAP
    solidMOPISecondAirGap = new G4Box("MOPISecondAirGap",
                                      MOPISecondAirGapXSize/2,
                                      MOPISecondAirGapYSize/2,
                                      MOPISecondAirGapZSize/2);
    
    logicMOPISecondAirGap = new G4LogicalVolume(solidMOPISecondAirGap,
                                                MOPISecondAirGapMaterial,
                                                "MOPISecondAirgap");
    
    physiMOPISecondAirGap = new G4PVPlacement(0,
                                              G4ThreeVector(MOPISecondAirGapXPosition,
                                                            MOPISecondAirGapYPosition ,
                                                            MOPISecondAirGapZPosition),
                                              "MOPISecondAirGap",
                                              logicMOPISecondAirGap, physiMOPIMotherVolume, false, 0);
    
    // Second Aluminum layer
    solidMOPISecondAluminumLayer = new G4Box("MOPISecondAluminumLayer",
                                             MOPISecondAluminumLayerXSize/2,
                                             MOPISecondAluminumLayerYSize/2 ,
                                             MOPISecondAluminumLayerZSize/2);
    
    logicMOPISecondAluminumLayer = new G4LogicalVolume(solidMOPISecondAluminumLayer,
                                                       MOPISecondAluminumLayerMaterial,
                                                       "MOPISecondAluminumLayer");
    
    physiMOPISecondAluminumLayer = new G4PVPlacement(0,
                                                     G4ThreeVector(MOPISecondAluminumLayerXPosition,
                                                                   MOPISecondAluminumLayerYPosition ,
                                                                   MOPISecondAluminumLayerZPosition),
                                                     "MOPISecondAluminumLayer",
                                                     logicMOPISecondAluminumLayer,
                                                     physiMOPIMotherVolume,
                                                     false,
                                                     0);
    
    // Second Kapton layer
    solidMOPISecondKaptonLayer = new G4Box("MOPISecondKaptonLayer",
                                           MOPISecondKaptonLayerXSize/2,
                                           MOPISecondKaptonLayerYSize/2 ,
                                           MOPISecondKaptonLayerZSize/2);
    
    logicMOPISecondKaptonLayer = new G4LogicalVolume(solidMOPISecondKaptonLayer,
                                                     MOPIFirstKaptonLayerMaterial,
                                                     "MOPISecondKaptonLayer");
    
    physiMOPISecondKaptonLayer = new G4PVPlacement(0,
                                                   G4ThreeVector(MOPISecondKaptonLayerXPosition,
                                                                 MOPISecondKaptonLayerYPosition ,
                                                                 MOPISecondKaptonLayerZPosition),
                                                   "MOPISecondKaptonLayer",
                                                   logicMOPISecondKaptonLayer,
                                                   physiMOPIMotherVolume,
                                                   false,
                                                   0);
    
    logicMOPIFirstAirGap -> SetVisAttributes(darkGreen);
    logicMOPISecondAirGap -> SetVisAttributes(darkGreen);
    
    
    }
/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyBeamNozzle()
{
    // ------------------------------//
    // THE FINAL TUBE AND COLLIMATOR //
    //-------------------------------//
    // The last part of the transport beam line consists of
    // a 59 mm thick PMMA slab (to stop all the diffused radiation), a 370 mm brass tube
    // (to well collimate the proton beam) and a final collimator with 25 mm diameter
    // aperture (that provide the final trasversal shape of the beam)
    
    // -------------------//
    //     PMMA SUPPORT   //
    // -------------------//
    const G4double nozzleSupportXSize = 29.5 *mm;
    const G4double nozzleSupportYSize = 180. *mm;
    const G4double nozzleSupportZSize = 180. *mm;
    
    const G4double nozzleSupportXPosition = -397.50 *mm;
    
    G4double phi = 90. *deg;
    // Matrix definition for a 90 deg rotation. Also used for other volumes
    G4RotationMatrix rm;
    rm.rotateY(phi);
    
    G4Box* solidNozzleSupport = new G4Box("NozzleSupport",
                                          nozzleSupportXSize,
                                          nozzleSupportYSize,
                                          nozzleSupportZSize);
    
    G4LogicalVolume* logicNozzleSupport = new G4LogicalVolume(solidNozzleSupport,
                                                              nozzleSupportMaterial,
                                                              "NozzleSupport");
    
    physiNozzleSupport = new G4PVPlacement(0, G4ThreeVector(nozzleSupportXPosition,0., 0.),
                                           "NozzleSupport",
                                           logicNozzleSupport,
                                           physicalTreatmentRoom,
                                           false,
                                           0);
    
    logicNozzleSupport -> SetVisAttributes(yellow);
    
    
    
    //------------------------------------//
    // HOLE IN THE SUPPORT                //
    //------------------------------------//
    const G4double innerRadiusHoleNozzleSupport = 0.*mm;
    const G4double outerRadiusHoleNozzleSupport = 21.5*mm;
    const G4double hightHoleNozzleSupport = 29.5 *mm;
    const G4double startAngleHoleNozzleSupport = 0.*deg;
    const G4double spanningAngleHoleNozzleSupport = 360.*deg;
    
    G4Tubs* solidHoleNozzleSupport = new G4Tubs("HoleNozzleSupport",
                                                innerRadiusHoleNozzleSupport,
                                                outerRadiusHoleNozzleSupport,
                                                hightHoleNozzleSupport,
                                                startAngleHoleNozzleSupport,
                                                spanningAngleHoleNozzleSupport);
    
    G4LogicalVolume* logicHoleNozzleSupport = new G4LogicalVolume(solidHoleNozzleSupport,
                                                                  holeNozzleSupportMaterial,
                                                                  "HoleNozzleSupport",
                                                                  0,
                                                                  0,
                                                                  0);
    
    
    physiHoleNozzleSupport = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector()),
                                               "HoleNozzleSupport",
                                               logicHoleNozzleSupport,
                                               physiNozzleSupport,
                                               false, 0);
    
    logicHoleNozzleSupport -> SetVisAttributes(darkOrange3);
    
    // ---------------------------------//
    //     BRASS TUBE 1 (phantom side)    //
    // ---------------------------------//
    const G4double innerRadiusBrassTube= 18.*mm;
    const G4double outerRadiusBrassTube = 21.5 *mm;
    const G4double hightBrassTube = 140.5*mm;
    const G4double startAngleBrassTube = 0.*deg;
    const G4double spanningAngleBrassTube = 360.*deg;
    
    const G4double brassTubeXPosition = -227.5 *mm;
    
    G4Tubs* solidBrassTube = new G4Tubs("BrassTube",
                                        innerRadiusBrassTube,
                                        outerRadiusBrassTube,
                                        hightBrassTube,
                                        startAngleBrassTube,
                                        spanningAngleBrassTube);
    
    G4LogicalVolume* logicBrassTube = new G4LogicalVolume(solidBrassTube,
                                                          brassTubeMaterial,
                                                          "BrassTube",
                                                          0, 0, 0);
    
    physiBrassTube = new G4PVPlacement(G4Transform3D(rm,
                                                     G4ThreeVector(brassTubeXPosition,
                                                                   0.,
                                                                   0.)),
                                       "BrassTube",
                                       logicBrassTube,
                                       physicalTreatmentRoom,
                                       false,
                                       0);
    
    logicBrassTube -> SetVisAttributes(darkOrange3);
    
    // ----------------------------------------------//
    //     BRASS TUBE 2 (inside the PMMA support)    //
    // ----------------------------------------------//
    const G4double innerRadiusBrassTube2= 18.*mm;
    const G4double outerRadiusBrassTube2 = 21.5 *mm;
    const G4double hightBrassTube2 = 29.5*mm;
    const G4double startAngleBrassTube2 = 0.*deg;
    const G4double spanningAngleBrassTube2 = 360.*deg;
    
    
    G4Tubs* solidBrassTube2 = new G4Tubs("BrassTube2",
                                         innerRadiusBrassTube2,
                                         outerRadiusBrassTube2,
                                         hightBrassTube2,
                                         startAngleBrassTube2,
                                         spanningAngleBrassTube2);
    
    G4LogicalVolume* logicBrassTube2 = new G4LogicalVolume(solidBrassTube2,
                                                           brassTube2Material,
                                                           "BrassTube2",
                                                           0, 0, 0);
    
    physiBrassTube2 = new G4PVPlacement(0,
                                        G4ThreeVector(0,0.,0.),
                                        logicBrassTube2,
                                        "BrassTube2",
                                        logicHoleNozzleSupport,
                                        false,
                                        0);
    
    logicBrassTube2 -> SetVisAttributes(darkOrange3);
    
    
    // --------------------------------------//
    //     BRASS TUBE 3 (beam line side)    //
    // -------------------------------------//
    const G4double innerRadiusBrassTube3= 18.*mm;
    const G4double outerRadiusBrassTube3 = 21.5 *mm;
    const G4double hightBrassTube3 = 10.0 *mm;
    const G4double startAngleBrassTube3 = 0.*deg;
    const G4double spanningAngleBrassTube3 = 360.*deg;
    
    const G4double brassTube3XPosition = -437 *mm;
    
    G4Tubs* solidBrassTube3 = new G4Tubs("BrassTube3",
                                         innerRadiusBrassTube3,
                                         outerRadiusBrassTube3,
                                         hightBrassTube3,
                                         startAngleBrassTube3,
                                         spanningAngleBrassTube3);
    
    G4LogicalVolume* logicBrassTube3 = new G4LogicalVolume(solidBrassTube3,
                                                           brassTube3Material,
                                                           "BrassTube3",
                                                           0, 0, 0);
    
    physiBrassTube3 = new G4PVPlacement(G4Transform3D(rm,
                                                      G4ThreeVector(brassTube3XPosition,
                                                                    0.,
                                                                    0.)),
                                        "BrassTube3",
                                        logicBrassTube3,
                                        physicalTreatmentRoom,
                                        false,
                                        0);
    
    logicBrassTube3 -> SetVisAttributes(darkOrange3);
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::HadrontherapyBeamFinalCollimator()
{
    // -----------------------//
    //     FINAL COLLIMATOR   //
    //------------------------//
    const G4double outerRadiusFinalCollimator = 21.5*mm;
    const G4double hightFinalCollimator = 3.5*mm;
    const G4double startAngleFinalCollimator = 0.*deg;
    const G4double spanningAngleFinalCollimator = 360.*deg;
    const G4double finalCollimatorXPosition = -83.5 *mm;
    
    G4double phi = 90. *deg;
    
    // Matrix definition for a 90 deg rotation. Also used for other volumes
    G4RotationMatrix rm;
    rm.rotateY(phi);
    
    solidFinalCollimator = new G4Tubs("FinalCollimator",
                                      innerRadiusFinalCollimator,
                                      outerRadiusFinalCollimator,
                                      hightFinalCollimator,
                                      startAngleFinalCollimator,
                                      spanningAngleFinalCollimator);
    
    G4LogicalVolume* logicFinalCollimator = new G4LogicalVolume(solidFinalCollimator,
                                                                finalCollimatorMaterial,
                                                                "FinalCollimator",
                                                                0,
                                                                0,
                                                                0);
    
    physiFinalCollimator = new G4PVPlacement(G4Transform3D(rm, G4ThreeVector(finalCollimatorXPosition,0.,0.)),
                                             "FinalCollimator", logicFinalCollimator, physicalTreatmentRoom, false, 0);
    
    logicFinalCollimator -> SetVisAttributes(yellow);
}
/////////////////////////// MESSENGER ///////////////////////////////////////
/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetRangeShifterXPosition(G4double value)
{
    physiRangeShifterBox -> SetTranslation(G4ThreeVector(value, 0., 0.));
    G4RunManager::GetRunManager() -> GeometryHasBeenModified();
    G4cout << "The Range Shifter is translated to"<< value/mm <<"mm along the X axis" <<G4endl;
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetRangeShifterXSize(G4double value)
{
    solidRangeShifterBox -> SetXHalfLength(value) ;
    G4cout << "RangeShifter size X (mm): "<< ((solidRangeShifterBox -> GetXHalfLength())*2.)/mm
    << G4endl;
    G4RunManager::GetRunManager() -> GeometryHasBeenModified();
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetFirstScatteringFoilXSize(G4double value)
{
    firstScatteringFoil -> SetXHalfLength(value);
    G4RunManager::GetRunManager() -> GeometryHasBeenModified();
    G4cout <<"The X size of the first scattering foil is (mm):"<<
    ((firstScatteringFoil -> GetXHalfLength())*2.)/mm
    << G4endl;
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetSecondScatteringFoilXSize(G4double value)
{
    secondScatteringFoil -> SetXHalfLength(value);
    G4RunManager::GetRunManager() -> GeometryHasBeenModified();
    G4cout <<"The X size of the second scattering foil is (mm):"<<
    ((secondScatteringFoil -> GetXHalfLength())*2.)/mm
    << G4endl;
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetOuterRadiusStopper(G4double value)
{
    solidStopper -> SetOuterRadius(value);
    G4RunManager::GetRunManager() -> GeometryHasBeenModified();
    G4cout << "OuterRadius od the Stopper is (mm):"
    << solidStopper -> GetOuterRadius()/mm
    << G4endl;
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetInnerRadiusFinalCollimator(G4double value)
{
    solidFinalCollimator -> SetInnerRadius(value);
    G4RunManager::GetRunManager() -> GeometryHasBeenModified();
    G4cout<<"Inner Radius of the final collimator is (mm):"
	<< solidFinalCollimator -> GetInnerRadius()/mm
	<< G4endl;
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetRSMaterial(G4String materialChoice)
{
    if (G4Material* pttoMaterial = G4NistManager::Instance()->FindOrBuildMaterial(materialChoice, false) )
    {
        if (pttoMaterial)
        {
            rangeShifterMaterial  = pttoMaterial;
            logicRangeShifterBox -> SetMaterial(pttoMaterial);
            G4cout << "The material of the Range Shifter has been changed to " << materialChoice << G4endl;
        }
    }
    else
    {
        G4cout << "WARNING: material \"" << materialChoice << "\" doesn't exist in NIST elements/materials"
	    " table [located in $G4INSTALL/source/materials/src/G4NistMaterialBuilder.cc]" << G4endl;
        G4cout << "Use command \"/parameter/nist\" to see full materials list!" << G4endl;
    }
}

/////////////////////////////////////////////////////////////////////////////
void PassiveProtonBeamLine::SetModulatorAngle(G4double value)
{
    modulator -> SetModulatorAngle(value);
    //G4RunManager::GetRunManager() -> GeometryHasBeenModified();
}
/////////////////////////////////////////////////////////////////////////////
```

[Go Top](#GoTop)<a id='SomeNotesOnDoseLET_RBE_etc'></a>
## Some notes about Dose, LET, RBE etc calculations.
[Go Top](#GoTop)<a id='DoseCalculations'></a>
### About [Dose Calculations](#DoseCalculations)

* **Output:** Dose.out
```
KPAd's FunPrompt $ grep -in Dose.out src/*
src/HadrontherapyAnalysisFileMessenger.cc:51:    DoseMatrixCmd->SetDefaultValue("Dose.out");
src/HadrontherapyMatrix.cc:89:stdFile("Dose.out"),
src/HadrontherapyMatrix.cc:326:        StoreMatrix(ionStore[i].name + "_Dose.out", ionStore[i].dose, sizeof(G4double));
KPAd's FunPrompt $ 
```
* The name **Dose.out** is stored in **stdFile** variable of G4String type (through the initializer list passed to the constructor of [HadrontherapyMatrix](#HadrontherapyMatrix.cc)).
    ```cpp 
HadrontherapyMatrix::HadrontherapyMatrix(G4int voxelX, G4int voxelY, G4int voxelZ, G4double mass): stdFile("Dose.out"), doseUnit(gray) { ... }
    ```
* The dose data is written out in a file that is opened using the **filename** variable, which in turn gets it's value in the following manner.
    * From the statement **filename = (file=="") ? stdFile:file;**, it is clear that if **file** is empty (==NULL) or not-defined, then **stdFile** is used, else **file** is used to assign to **filename**.
        * Below, **stdFile** is used in [**Conditional ternary operator ( ? )**](http://www.cplusplus.com/doc/tutorial/operators/) where **condition ? result1 : result2** means this whole expression will return (evaluate to) either result1 or result2 if the condition is true or false respectively. (Examples: '7==5 ? 4 : 3' evaluates to 3, '5>3 ? a : b' evaluates to a etc.)
* File opening and dose writing is done as follows inside **void [HadrontherapyMatrix](#HadrontherapyMatrix.cc)::StoreDoseFluenceAscii(G4String file)**.
```cpp
////////////////////////////////////////////////////////////////////////
// Store dose into a single file or in histograms. 
// Please note that this function is called via messenger commands
// defined in the HadrontherapyAnalysisFileMessenger.cc class file
void HadrontherapyMatrix::StoreDoseFluenceAscii(G4String file)
{
#define width 15L
    filename = (file=="") ? stdFile:file; //kp: if 'file' is empty, use 'stdFile', else 'file' for 'filename
    
    // Sort like periodic table   
    std::sort(ionStore.begin(), ionStore.end());
    G4cout << "Dose is being written to " << filename << G4endl;
    ofs.open(filename, std::ios::out);
```
* **Dose** value is written out as follows: **ofs << std::setw(width) << (matrix[n]/massOfVoxel)/doseUnit;**. In otherwords, **matrix[n]/massOfVoxel** is what we call as the Dose.
* The value of Dose or matrix is assigned as follows in L412 of [HadrontherapyMatrix](#HadrontherapyMatrix.cc)): **matrix[Index(i,j,k)] += energyDeposit;**
```cpp
void HadrontherapyMatrix::Fill(G4int i, G4int j, G4int k,
                               G4double energyDeposit)
{
    if (matrix)
        matrix[Index(i,j,k)] += energyDeposit;
    
    // Store the energy deposit in the matrix element corresponding 
    // to the phantom voxel  
}
```
* Above Fill() method is called from inside the method **void HadrontherapyEventAction::EndOfEventAction(const G4Event* evt)** in L94 of [**HadrontherapyEventAction.cc**](#HadrontherapyEventAction.cc) as follows:                   **matrix -> Fill(i, j, k, energyDeposit/MeV);**
    * Just 1 line above that, there is **G4double energyDeposit = ((`*CHC`)`[h]`) -> GetEdep();**
    * In the same method, we have the following lines that precedes above line.
        *    `G4HCofThisEvent* HCE = evt -> GetHCofThisEvent();`
        *    `HadrontherapyDetectorHitsCollection* CHC = (HadrontherapyDetectorHitsCollection*)(HCE -> GetHC(hitsCollectionID));`
        
```cpp
/////////////////////////////////////////////////////////////////////////////
void HadrontherapyEventAction::EndOfEventAction(const G4Event* evt)
{
    if(hitsCollectionID < 0)
        return;
    G4HCofThisEvent* HCE = evt -> GetHCofThisEvent();
    
    // Clear voxels hit list
    HadrontherapyMatrix* matrix = HadrontherapyMatrix::GetInstance();
    if (matrix) matrix -> ClearHitTrack();
    
    if(HCE)
    {
        HadrontherapyDetectorHitsCollection* CHC = (HadrontherapyDetectorHitsCollection*)(HCE -> GetHC(hitsCollectionID));
        if(CHC)
        {
            if(matrix)
            {
                // Fill the matrix with the information: voxel and associated energy deposit
                // in the detector at the end of the event
                
                G4int HitCount = CHC -> entries();
                for (G4int h=0; h<HitCount; h++)
                {
                    G4int i = ((*CHC)[h]) -> GetXID();
                    G4int j = ((*CHC)[h]) -> GetYID();
                    G4int k = ((*CHC)[h]) -> GetZID();
                    G4double energyDeposit = ((*CHC)[h]) -> GetEdep();
                    matrix -> Fill(i, j, k, energyDeposit/MeV);
                }
            }
        }
    }
}
```
* It seems, above one is the only instance when **GetEdep()** method is invoked. Which means, either this is the only source of Dose for entire 'hadrontherapy' program or it may have been forgotten to be called again if we have an independent module (such as RBE-calculation part??). May be RBE-calculation also takes the **matrix`[n]`** or it is supposed to make similar method calls separately.



[Go Top](#GoTop)<a id='LETCalculations'></a>
### About [LET Calculations](#DoseCalculations)
8/19/19,

file:///Users/kpadhikari/KpInstalls/Geant4/ExamplesBld/advanced/hadrontherapy/src/HadrontherapyLet.cc
Default output filename ‘Let.out’ defined in the initializer-list of the constructor:
HadrontherapyLet::HadrontherapyLet(HadrontherapyDetectorConstruction* pDet)
:filename("Let.out"),matrix(0) // Default output filename


This is also a singleton-object class
The output file is opened and written to in void HadrontherapyLet::StoreLetAscii(), then it is closed.

The first three columns are voxel indices, LDT is totalLetD[v]/(keV/um), LTT is totalLetT[v]/(keV/um), and the rest are list of particles/ions. (Here, D & T indicates Track averaged LET and Dose averaged LET.)

```
KPAd's FunPrompt $ m TmpBkp_defMac/Let.out
i       j       k       LDT            LTT            proton_1_D     proton_1_T     proton_D       proton_T       N15_D          N15_T          

0       0       0       1.13758        1.13722        1.13758        1.13722        0              0              0              0              
1       0       0       1.13905        1.13906        1.13905        1.13906        0              0              0              0              
2       0       0       1.14112        1.141          1.14112        1.141          0              0              0              0              
3       0       0       1.14297        1.14289        1.14297        1.14289        0              0              0              0              
4       0       0       1.14478        1.14468        1.14478        1.14468        0              0              0              0              
5       0       0       1.14675        1.14654        1.14675        1.14654        0              0              0              0              
6       0       0       1.14839        1.14837        1.14839        1.14837        0              0              0              0              
7       0       0       1.15054        1.15019        1.15054        1.15019        0              0              0              0              
8       0       0       1.15207        1.15208        1.15207        1.15208        0              0              0              0              
9
```

[Go Top](#GoTop)<a id='RBECalculations'></a>
### About [RBE Calculations](#RBECalculations)

Part of what I see from **'more RBE.out'**.
```
Dose(Gy)         ln(S)        Survival      DoseB(Gy)            RBE   depth(slice)
0              0              1              0            nan              0
0              0              1              0            nan              1
0              0              1              0            nan              2
0              0              1              0            nan              3
0              0              1              0            nan              4
...
...
0              0              1              0            nan           5996
0              0              1              0            nan           5997
0              0              1              0            nan           5998
0              0              1              0            nan           5999
```

### Tracking RBE calculations
**First of Tracking how RBE.out is produced**
 
* **‘./hadrontherapy RBE-62MeV-proton.mac'** causes it to produce **RBE.out**
* **RBE.out** is defined in [HadrontherapyRBE.hh](#HadrontherapyRBE.hh) as the value of **private** data member **fRBEPath** of the class. (It also defines "AlphaAndBeta.out" similarly as value of fAlphaBetaPath).
    * Please remember, **HadrontherapyRBE is also a singleton class**, so only one object of it will be created during the entire run. In various plances (methods of different classes, a local pointer such as 'rbe' is created through GetInstance() method of the same class, but we should remember that each of them will always point to the same object residing in the memory.
* The **opening and writing of both** output files are done in **StoreAlphaAndBeta()** and **StoreRBE()** methods in the [HadrontherapyRBE.cc](#HadrontherapyRBE.cc) file. RBE values are written looping over **fRBE[i]**.
    * The StoreRBE() method is called at L144 (from inside **EndOfRunAction(const G4Run*)** method) of **[HadrontherapyRunAction.cc](#HadrontherapyRunAction.cc)** as **rbe->StoreRBE();** (likewise for StoreAlphaAndBeta())
    * **EndOfRunAction(..)** doesn't show up being called directly in any of the src or include files, and because I see that **HadrontherapyRunAction** inherits from **G4UserRunAction**, I am assuming that the EndOfRunAction(..) is called (by the kernel?) automatically internally (when a run ends).

<span style="color:red"> So, the **Chain of events** on the **writing out** part is as follwos:</span>
* Call of **HadrontherapyRunAction::EndOfRunAction(const G4Run*)**
* Call of **rbe->StoreRBE();**
* Opening of **RBE.out** file.
* Writing out RBE values by looping over the elements of **fRBE** matrix.


#### Next tracking how fRBE matrix gets values.
* The RBE values (5th column) are all **'nan'** because it’s defined as **fRBE = fDoseX / fDose; (matrix or array division)** and the Dose (which comes from fDose) seem to be all zeros (first column).
* Upon digging through: **“src/HadrontherapyRBE.cc:501: fDose = eDep * (fDoseScale / fMassOfVoxel);”** which tells me either **eDep or fDoseScale** is making it zero. But,another grepping shows **fDoseScale=scale=1**. So, the **culprit for 'NANs' is eDep**. 
* Hunting for eDep: I find **fDose = eDep * (fDoseScale / fMassOfVoxel);** inside the method **void HadrontherapyRBE::SetEnergyDeposit(const std::valarray`<G4double>` eDep)** of the same class.
    - Similarly, I find **fDose = eDep * (fDoseScale / fMassOfVoxel);** and **fDose += eDep * (fDoseScale / fMassOfVoxel);** inside another method of the same class **void HadrontherapyRBE::AddEnergyDeposit(const std::valarray`<G4double>` eDep))**
* That tells me that **eDep** comes through as **input arg** in SetEnergyDeposit or AddEnergyDeposit methods. Now I grep for **SetEnergyDeposit** and **AddEnergyDeposit** and I find '**rbe->SetEnergyDeposit([fRBEAccumulable](#HadrontherapyRBEAccumulable.hh).GetEnergyDeposit());'** and **'rbe->AddEnergyDeposit([fRBEAccumulable](#HadrontherapyRBEAccumulable.hh).GetEnergyDeposit());**' inside the method **[HadrontherapyRunAction](#HadrontherapyRunAction.cc)::EndOfRunAction(const G4Run*)**.
    * Note that **fRBEAccumulable** is private data member of **[HadrontherapyRunAction](#HadrontherapyRunAction.hh)** class (the member is declared as "**[HadrontherapyRBEAccumulable](#HadrontherapyRBEAccumulable.hh) fRBEAccumulable;"**).
    * Method **const [HadrontherapyRBEAccumulable](#HadrontherapyRBEAccumulable.hh)::array_type HadrontherapyRBEAccumulable::GetEnergyDeposit() const** simply returns **fEnergyDeposit;**.
        * Inside **void HadrontherapyRBEAccumulable::Initialize()**, there is a line **fEnergyDeposit = array_type(0.0, fVoxels);**.
            - Tried replacing 0.0 with 1.0 in the line. This time, Dose column had a value of 0.116825 for all rows and the RBE came out 0, instead of NAN.
        * Inside **void HadrontherapyRBEAccumulable::Merge(const G4VAccumulable& rhs)**, there is a line **fEnergyDeposit += other.fEnergyDeposit;**
            - Tried commenting out this method, and it produced compilation errors, indicating that it's essential or being used.
        * Inside **void HadrontherapyRBEAccumulable::Reset()**, there is **fEnergyDeposit = 0.0;**.
        * Inside **void HadrontherapyRBEAccumulable::Accumulate(G4double E, G4double energyDeposit, G4double dX, G4int Z, G4int i, G4int j, G4int k)**, there is a line **fEnergyDeposit[n] += energyDeposit;**.
            - Because I searched for the call of Accumulate method and didn't find one anywhere, I supsected that this may not have been used at all, so I simply commented out this whole method and I found that it didn't produce and compilation or runtime error, thus confirming what I had suspected.
    * Note that [HadrontherapyRBEAccumulable](#HadrontherapyRBEAccumulable.hh) class inherits from **G4VAccumulable** (see [G4VAccumulable.hh](http://hurel.hanyang.ac.kr/Geant4/Doxygen/10.03.p01/html/_g4_v_accumulable_8hh_source.html) and [G4VAccumulable.cc](http://nuclear.korea.ac.kr/~lamps/geant4/G4AccumulableManager_8cc_source.html)).

8/13-14/19,

The name of the output file for **RBE module** run which produces **RBE.out** (using **‘./hadrontherapy RBE-62MeV-proton.mac'**) is defined in [**include/HadrontherapyRBE.hh**](#HadrontherapyRBE.hh)

```
    KPAd's FunPrompt $ grep -in 'out' include/* | grep -in rbe.out
    27:include/HadrontherapyRBE.hh:155:    G4String fRBEPath { "RBE.out" };
    KPAd's FunPrompt $ pwd
    /Users/kpadhikari/KpInstalls/Geant4/ExamplesBld/advanced/hadrontherapy
    KPAd's FunPrompt $
 ```  
 
Some info about macro commands is also found at the top of that file.
This run also produces **AlphaAndBeta.out**, which is also defined in the same file. See the corresponding code in that .hh file:
```
  // Output paths (TODO: Change to analysis tools)
    G4String fAlphaBetaPath { "AlphaAndBeta.out" };
    G4String fRBEPath { "RBE.out" };
```
The opening and writing of both output files are done  in the corresponding Store methods in the [HadrontherapyRBE.cc](#HadrontherapyRBE.cc) file:

```cpp
void HadrontherapyRBE::StoreAlphaAndBeta()
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Writing alpha and beta..." << G4endl;
    }
    ofstream ofs(fAlphaBetaPath);

    ComputeAlphaAndBeta();

    if (ofs.is_open())
    {
        ofs << "alpha" << std::setw(width) << "beta " << std::setw(width) << "depth(slice)" << G4endl;
        for (G4int i = 0; i < fNumberOfVoxelsAlongX * fNumberOfVoxelsAlongY * fNumberOfVoxelsAlongZ; i++)
            ofs << fAlpha[i]*gray << std::setw(15L) << fBeta[i]*pow(gray, 2.0) << std::setw(15L) << i << G4endl;
    }
    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: Alpha and beta written to " << fAlphaBetaPath << G4endl;
    }
}

void HadrontherapyRBE::StoreRBE()
{
    ofstream ofs(fRBEPath);

    // TODO: only if not yet calculated. Or in RunAction???
    ComputeRBE();

    if (ofs.is_open())
    {
        ofs << "Dose(Gy)" << std::setw(width) << "ln(S) " << std::setw(width) << "Survival"  << std::setw(width) << "DoseB(Gy)" << std::setw(width) << "RBE" <<  std::setw(width) << "depth(slice)" << G4endl;

        for (G4int i = 0; i < fNumberOfVoxelsAlongX * fNumberOfVoxelsAlongY * fNumberOfVoxelsAlongZ; i++)

            ofs << (fDose[i] / gray) << std::setw(width) << fLnS[i] << std::setw(width) << fSurvival[i]
                << std::setw(width) << fDoseX[i] / gray << std::setw(width) << fRBE[i] << std::setw(width)  << i << G4endl;
    }
    if (fVerboseLevel > 0)
    {
        G4cout << "RBE: RBE written to " << fRBEPath << G4endl;
    }
}
```

The RBE values are zero because it’s defined as **fRBE = fDoseX / fDose; (matrix division)** and various elements of fRBE is being printed in the RBE column of the output file. As we can see in the same output file, the first column is for Dose (which comes from fDose) and they seem to be all zeros, that’s the reason why they fRBE is coming out all ‘nan’.

So, I am digging for how ‘fDose’ gets value. When I do the grepping for it, I find **“src/HadrontherapyRBE.cc:501:    fDose = eDep * (fDoseScale / fMassOfVoxel);”** which tells me either **eDep or fDoseScale** is making it zero. But, I found from another grepping that **fDoseScale=scale=1. So, the culprit is eDep**. Now I am grepping about eDep in both src and include directories and I find nowhere it says eDep = something.

Now, I go back to the [HadrontherapyRBE.cc](#HadrontherapyRBE.cc) file and find the following method:

```cpp
void HadrontherapyRBE::SetEnergyDeposit(const std::valarray<G4double> eDep)
{
    if (fVerboseLevel > 1)
    {
        G4cout << "RBE: Setting dose..." << G4endl;
    }
    fDose = eDep * (fDoseScale / fMassOfVoxel);
}
```
That tells me that eDep is coming from the SetEnergyDeposit method as an input. Now I search for that method and I find the following saying it’s coming from [**HadrontherapyRunAction.cc**](#HadrontherapyRunAction.cc).

```
KPAd's FunPrompt $ grep -in setEnergyDeposit src/* 
src/HadrontherapyRBE.cc:495:void HadrontherapyRBE::SetEnergyDeposit(const std::valarray<G4double> eDep) 
src/HadrontherapyRBE.cc~:495:void HadrontherapyRBE::SetEnergyDeposit(const std::valarray<G4double> eDep) 
src/HadrontherapyRunAction.cc:140:           rbe->SetEnergyDeposit(fRBEAccumulable.GetEnergyDeposit());
KPAd's FunPrompt $
```

Copying the 3rd one to make easy navigation links:

src/[HadrontherapyRunAction.cc](#HadrontherapyRunAction.cc):140:            rbe->SetEnergyDeposit([fRBEAccumulable](#HadrontherapyRBEAccumulable.hh).GetEnergyDeposit());

**About fDoseX**
```
KPAd's FunPrompt $ grep -in fDoseX src/*
src/HadrontherapyRBE.cc:378:    fDoseX.resize(fNumberOfVoxels);
src/HadrontherapyRBE.cc:385:            fDoseX[i] = 0.0;
src/HadrontherapyRBE.cc:390:            fDoseX[i] = sqrt((-fLnS[i] / fBetaX) + pow((fAlphaX / (2 * fBetaX)), 2.0)) - (fAlphaX / (2 * fBetaX));
src/HadrontherapyRBE.cc:398:            fDoseX[i] = ( (-fLnS[i] + ln_Scut) / smax ) + fDoseCut;
src/HadrontherapyRBE.cc:401:    fRBE = fDoseX / fDose; //kpa: matrix division operation
src/HadrontherapyRBE.cc:560:                << std::setw(width) << fDoseX[i] / gray << std::setw(width) << fRBE[i] << std::setw(width)  << i << G4endl;
KPAd's FunPrompt $
```

8/7/19,

It seems HadrontherapyRBE.cc also defines a singleton class because it has **static CreateInstance and GetInstance methods with no public constructor, just like we saw “    G4ScoringManager *scoringManager = G4ScoringManager::GetScoringManager();” in the main file of the hadrontherapy example.

I realized today 8/13/19 that HadrontherapyRBE is also a singleton-object type class. It uses CreateInstance and GetInstance methods** together with a **private static data member ‘instance’** (see line **“static HadrontherapyRBE* instance;”** in .hh file) and **“HadrontherapyRBE *rbe = HadrontherapyRBE::GetInstance();”** in [HadrontherapyRunAction.cc](#HadrontherapyRunAction.cc). Now I am realizing that Geant4 is using lots of singleton-object type classes where you can produce only one object/instance of that type.